# Baroque Ceiling Paintings - DuckDB Analysis

This notebook loads the Parquet database created from NFDI4Culture and CbDD data into a **persistent DuckDB database** for collaborative analysis.

## Data Sources
- **CbDD (Primary)**: NFDI4Culture Knowledge Graph (SPARQL endpoint)
  - 6,228 baroque ceiling paintings with metadata
  - Enrichment from CbDD graphData.json (persons, buildings, rooms)
  - Subject Resolution via ICONCLASS & Getty AAT
  
- **Bildindex (Secondary)**: Historical art photographs from Bildindex (n4c:E6161)
  - Cross-referenced via shared GND identifiers (painters & buildings)
  - Enables linking ceiling paintings to historical photographs

## Tech Stack
- **DuckDB**: Persistent analytical database with SQL interface
- **Polars**: High-performance DataFrame operations
- **Altair**: Declarative visualizations

## Table Overview
| Prefix | Source | Description |
|--------|--------|-------------|
| (none) | CbDD | Core tables: paintings, persons, buildings, rooms, subjects, etc. |
| `bi_` | Bildindex | Cross-reference tables: bi_items, bi_buildings, bi_painters, etc. |

In [5]:
# =============================================================================
# CELL 1: Setup & Configuration
# =============================================================================
import duckdb
import polars as pl
import altair as alt
import re
from pathlib import Path
from typing import Tuple, Optional
from IPython.display import display as ipython_display

# Configuration
PARQUET_DIR = Path(r"c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories")
DUCKDB_PATH = PARQUET_DIR / "baroque.duckdb"
PREFIX = "baroque_"
BILDINDEX_PREFIX = "bildindex_"

# CbDD Table names (from baroque_ parquet files)
CBDD_TABLES = [
    'paintings', 'persons', 'buildings', 'rooms', 'ensembles',
    'subjects', 'painting_persons', 'painting_subjects',
    'building_persons', 'room_persons'
]

# Bildindex Table names (from bildindex_ parquet files)
BILDINDEX_TABLES = [
    'items', 'buildings', 'painters', 'subjects', 'gnd_overlaps'
]

# Combined for backward compatibility
TABLES = CBDD_TABLES

# Enable Altair for larger datasets
alt.data_transformers.disable_max_rows()

print(f"📁 Parquet directory: {PARQUET_DIR}")
print(f"🗄️ DuckDB database: {DUCKDB_PATH}")
print(f"📊 CbDD tables: {len(CBDD_TABLES)}")
print(f"📊 Bildindex tables: {len(BILDINDEX_TABLES)}")

📁 Parquet directory: c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories
🗄️ DuckDB database: c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories/baroque.duckdb
📊 CbDD tables: 10
📊 Bildindex tables: 5


In [2]:
# =============================================================================
# CELL 2: German Year Parsing Utility
# =============================================================================
# Handles formats like:
#   - "1732" → (1732, 1732, False)
#   - "1732-1742" → (1732, 1742, False)
#   - "um 1732" / "ca. 1579" → (1732, 1732, True)
#   - "um 1732-1742" → (1732, 1742, True)
#   - "17. Jh" / "17. Jahrhundert" → (1600, 1699, False)
#   - "2. Hälfte 17. Jahrhundert" → (1650, 1699, False)
#   - "1. Hälfte 18. Jh." → (1700, 1749, False)
#   - "Anfang 18. Jh." → (1700, 1730, True)
#   - "Ende 17. Jh." → (1670, 1699, True)
#   - "Mitte 18. Jahrhundert" → (1740, 1760, True)
#   - "vor 1750" → (None, 1750, True)
#   - "nach 1700" → (1700, None, True)
#   - "bis 1716" → (None, 1716, False)

def parse_german_year(year_str: Optional[str]) -> Tuple[Optional[int], Optional[int], bool, bool]:
    """
    Parse German year strings into structured data.
    
    Returns:
        (year_start, year_end, is_approximate, is_parsed)
        - year_start: Earliest possible year (or None)
        - year_end: Latest possible year (or None)
        - is_approximate: True if "um", "ca.", "circa", etc.
        - is_parsed: True if successfully parsed, False if unparseable
    """
    if year_str is None or str(year_str).strip() == '':
        return (None, None, False, False)
    
    text = str(year_str).strip().lower()
    original_text = text
    
    # Track approximation markers
    is_approx = False
    approx_markers = ['um ', 'ca. ', 'ca ', 'circa ', 'etwa ', 'wohl ', 'vermutlich ']
    for marker in approx_markers:
        if marker in text:
            is_approx = True
            text = text.replace(marker, '')
    
    # Remove common noise
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    text = text.replace('jhd', 'jh').replace('jahrhundert', 'jh')
    
    # Pattern 1: Simple year "1732"
    match = re.search(r'^(\d{4})$', text.strip())
    if match:
        year = int(match.group(1))
        return (year, year, is_approx, True)
    
    # Pattern 2: Year range "1732-1742" or "1732–1742" or "1732/1742"
    match = re.search(r'(\d{4})\s*[-–/]\s*(\d{4})', text)
    if match:
        return (int(match.group(1)), int(match.group(2)), is_approx, True)
    
    # Pattern 3: Century "17. jh" or "17.jh."
    match = re.search(r'(\d{1,2})\.?\s*jh', text)
    if match:
        century = int(match.group(1))
        base_year = (century - 1) * 100
        
        # Check for half-century modifiers
        if '2. hälfte' in text or 'zweite hälfte' in text:
            return (base_year + 50, base_year + 99, is_approx, True)
        elif '1. hälfte' in text or 'erste hälfte' in text:
            return (base_year, base_year + 49, is_approx, True)
        elif 'anfang' in text or 'beginn' in text or 'früh' in text:
            return (base_year, base_year + 30, True, True)
        elif 'ende' in text or 'spät' in text:
            return (base_year + 70, base_year + 99, True, True)
        elif 'mitte' in text:
            return (base_year + 40, base_year + 60, True, True)
        else:
            return (base_year, base_year + 99, is_approx, True)
    
    # Pattern 4: "vor 1750" (before)
    match = re.search(r'vor\s+(\d{4})', text)
    if match:
        return (None, int(match.group(1)), True, True)
    
    # Pattern 5: "nach 1700" (after) or "ab 1700"
    match = re.search(r'(?:nach|ab)\s+(\d{4})', text)
    if match:
        return (int(match.group(1)), None, True, True)
    
    # Pattern 6: "bis 1716" (until)
    match = re.search(r'bis\s+(\d{4})', text)
    if match:
        return (None, int(match.group(1)), is_approx, True)
    
    # Pattern 7: Extract first 4-digit year as fallback
    match = re.search(r'(\d{4})', text)
    if match:
        year = int(match.group(1))
        # Check if there's a second year
        all_years = re.findall(r'(\d{4})', text)
        if len(all_years) >= 2:
            years = [int(y) for y in all_years]
            return (min(years), max(years), is_approx, True)
        return (year, year, is_approx, True)
    
    # Unparseable
    return (None, None, False, False)


# Test the parser
test_cases = [
    "1732",
    "1732-1742",
    "um 1732-1742",
    "ca. 1579",
    "17. Jh",
    "2. Hälfte 17. Jahrhundert",
    "Anfang 18. Jh.",
    "vor 1750",
    "nach 1700",
    "bis 1716, Weihe 1736",
    "1697-1749, 1782",
    "wohl 18. Jahrhundert",
    "mittelalterlich, 1730-1734",
    None,
    ""
]

print("🧪 Year Parser Test Results:")
print("=" * 80)
print(f"{'Input':<40} {'Start':>6} {'End':>6} {'Approx':>7} {'Parsed':>7}")
print("-" * 80)
for tc in test_cases:
    start, end, approx, parsed = parse_german_year(tc)
    display = str(tc)[:38] if tc else "(None)"
    start_s = str(start) if start else "-"
    end_s = str(end) if end else "-"
    print(f"{display:<40} {start_s:>6} {end_s:>6} {str(approx):>7} {str(parsed):>7}")

🧪 Year Parser Test Results:
Input                                     Start    End  Approx  Parsed
--------------------------------------------------------------------------------
1732                                       1732   1732   False    True
1732-1742                                  1732   1742   False    True
um 1732-1742                               1732   1742    True    True
ca. 1579                                   1579   1579    True    True
17. Jh                                     1600   1699   False    True
2. Hälfte 17. Jahrhundert                  1650   1699   False    True
Anfang 18. Jh.                             1700   1730    True    True
vor 1750                                      -   1750    True    True
nach 1700                                  1700      -    True    True
bis 1716, Weihe 1736                          -   1716   False    True
1697-1749, 1782                            1697   1749   False    True
wohl 18. Jahrhundert                   

In [3]:
# =============================================================================
# CELL 1b: Reconnect to Existing Database (skip if creating fresh)
# =============================================================================
# Use this cell to reconnect to an existing database without rebuilding

if DUCKDB_PATH.exists():
    con = duckdb.connect(str(DUCKDB_PATH), read_only=True)
    print(f"✅ Reconnected to existing database: {DUCKDB_PATH.name} (read-only)")
    
    # Show table counts grouped by source
    tables_df = con.execute("SHOW TABLES").pl()
    print(f"📋 Found {len(tables_df)} tables:")
    
    cbdd_tables = []
    bi_tables = []
    for row in tables_df.iter_rows(named=True):
        count = con.execute(f"SELECT COUNT(*) FROM {row['name']}").fetchone()[0]
        if row['name'].startswith('bi_'):
            bi_tables.append((row['name'], count))
        else:
            cbdd_tables.append((row['name'], count))
    
    print("\n   📘 CbDD Tables:")
    for name, count in cbdd_tables:
        print(f"      {name:<25}: {count:>6,} rows")
    
    if bi_tables:
        print("\n   📗 Bildindex Tables:")
        for name, count in bi_tables:
            print(f"      {name:<25}: {count:>6,} rows")
else:
    print(f"⚠️ Database not found. Run cells 2-5 to create it.")
    con = None

⚠️ Database not found. Run cells 2-5 to create it.


In [4]:
# =============================================================================
# CELL 3: Create Persistent DuckDB Database
# =============================================================================
# This creates/opens the baroque.duckdb file for persistent storage

# Close any existing connection first
try:
    con.close()
    print("🔌 Closed existing connection")
except:
    pass

# Remove existing database to recreate fresh (comment out to append)
if DUCKDB_PATH.exists():
    try:
        DUCKDB_PATH.unlink()
        print(f"🗑️ Removed existing database: {DUCKDB_PATH.name}")
    except PermissionError:
        print(f"⚠️ Could not remove existing database (in use). Will overwrite tables.")

# Connect to persistent database
con = duckdb.connect(str(DUCKDB_PATH))
print(f"✅ Connected to DuckDB: {DUCKDB_PATH.name}")

# Register the year parser as Python UDFs with special null handling
# null_handling='special' allows the function to receive and return NULL values
con.create_function(
    'parse_year_start', 
    lambda x: parse_german_year(x)[0], 
    ['VARCHAR'], 
    'INTEGER',
    null_handling='special'
)
con.create_function(
    'parse_year_end', 
    lambda x: parse_german_year(x)[1], 
    ['VARCHAR'], 
    'INTEGER',
    null_handling='special'
)
con.create_function(
    'parse_year_approx', 
    lambda x: parse_german_year(x)[2], 
    ['VARCHAR'], 
    'BOOLEAN',
    null_handling='special'
)
con.create_function(
    'parse_year_success', 
    lambda x: parse_german_year(x)[3], 
    ['VARCHAR'], 
    'BOOLEAN',
    null_handling='special'
)

print("✅ Registered year parsing UDFs (with null handling)")

IOException: IO Error: Cannot open file "/Users/yomo/DataStory_Docker/course-data-stories-baroque/src/docs/story/CbDD/c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories/baroque.duckdb": No such file or directory

In [ ]:
# =============================================================================
# CELL 4: Load Parquet Tables into DuckDB with Year Parsing
# =============================================================================

def load_parquet_table(con: duckdb.DuckDBPyConnection, table_name: str, prefix: str = PREFIX) -> int:
    """Load a Parquet file into DuckDB as a table."""
    parquet_path = PARQUET_DIR / f"{prefix}{table_name}.parquet"
    
    if not parquet_path.exists():
        print(f"   ⚠️ {table_name}: File not found at {parquet_path}")
        return 0
    
    # Determine the target table name in DuckDB
    # For bildindex tables, prefix with "bi_" to avoid name collisions
    if prefix == BILDINDEX_PREFIX:
        db_table_name = f"bi_{table_name}"
    else:
        db_table_name = table_name
    
    # For paintings table, add parsed year columns
    if table_name == 'paintings' and prefix == PREFIX:
        con.execute(f"""
            CREATE TABLE {db_table_name} AS
            SELECT 
                *,
                parse_year_start(year) AS year_start,
                parse_year_end(year) AS year_end,
                parse_year_approx(year) AS year_is_approximate,
                parse_year_success(year) AS year_parsed
            FROM read_parquet('{parquet_path}')
        """)
    else:
        con.execute(f"""
            CREATE TABLE {db_table_name} AS
            SELECT * FROM read_parquet('{parquet_path}')
        """)
    
    row_count = con.execute(f"SELECT COUNT(*) FROM {db_table_name}").fetchone()[0]
    return row_count


print("📥 Loading CbDD Parquet tables into DuckDB...")
print("=" * 60)

table_stats = {}
for table in CBDD_TABLES:
    row_count = load_parquet_table(con, table, PREFIX)
    table_stats[table] = row_count
    status = "✅" if row_count > 0 else "⚠️"
    print(f"   {status} {table:<25}: {row_count:>8,} rows")

print("=" * 60)
total_rows = sum(table_stats.values())
print(f"📊 Total: {total_rows:,} rows across {len([t for t in table_stats if table_stats[t] > 0])} tables")

# =============================================================================
# Load Bildindex Tables
# =============================================================================
print("\n📥 Loading Bildindex Parquet tables into DuckDB...")
print("=" * 60)

bildindex_stats = {}
for table in BILDINDEX_TABLES:
    row_count = load_parquet_table(con, table, BILDINDEX_PREFIX)
    bildindex_stats[table] = row_count
    db_table_name = f"bi_{table}"
    status = "✅" if row_count > 0 else "⚠️"
    print(f"   {status} {db_table_name:<25}: {row_count:>8,} rows")

print("=" * 60)
bi_total = sum(bildindex_stats.values())
print(f"📊 Bildindex Total: {bi_total:,} rows across {len([t for t in bildindex_stats if bildindex_stats[t] > 0])} tables")

# Combine stats
all_table_stats = {**table_stats, **{f"bi_{k}": v for k, v in bildindex_stats.items()}}

📥 Loading CbDD Parquet tables into DuckDB...
   ⚠️ paintings: File not found at c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories/baroque_paintings.parquet
   ⚠️ paintings                :        0 rows
   ⚠️ persons: File not found at c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories/baroque_persons.parquet
   ⚠️ persons                  :        0 rows
   ⚠️ buildings: File not found at c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories/baroque_buildings.parquet
   ⚠️ buildings                :        0 rows
   ⚠️ rooms: File not found at c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories/baroque_rooms.parquet
   ⚠️ rooms                    :        0 rows
   ⚠️ ensembles: File not found at c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories/baroque_ensembles.parquet
   ⚠️ ensembles                :        0 rows
   ⚠️ subjects: File not found at c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories/baroque_subjects.parq

In [ ]:
# =============================================================================
# CELL 5: Create Indexes for Performance
# =============================================================================

print("🔧 Creating indexes for CbDD tables...")

indexes = [
    # Paintings table
    ("idx_paintings_nfdi", "paintings", "nfdi_uri"),
    ("idx_paintings_cbdd", "paintings", "cbdd_painting_id"),
    ("idx_paintings_building", "paintings", "building_id"),
    ("idx_paintings_room", "paintings", "room_id"),
    ("idx_paintings_year_start", "paintings", "year_start"),
    ("idx_paintings_state", "paintings", "location_state"),
    
    # Junction tables
    ("idx_pp_nfdi", "painting_persons", "nfdi_uri"),
    ("idx_pp_person", "painting_persons", "person_id"),
    ("idx_ps_nfdi", "painting_subjects", "nfdi_uri"),
    ("idx_ps_subject", "painting_subjects", "subject_uri"),
    ("idx_bp_building", "building_persons", "building_id"),
    ("idx_bp_person", "building_persons", "person_id"),
    ("idx_rp_room", "room_persons", "room_id"),
    ("idx_rp_person", "room_persons", "person_id"),
    
    # Entity tables
    ("idx_persons_id", "persons", "person_id"),
    ("idx_buildings_id", "buildings", "building_id"),
    ("idx_rooms_id", "rooms", "room_id"),
    ("idx_subjects_uri", "subjects", "subject_uri"),
]

for idx_name, table, column in indexes:
    try:
        con.execute(f"CREATE INDEX {idx_name} ON {table}({column})")
        print(f"   ✅ {idx_name}")
    except Exception as e:
        if "already exists" in str(e).lower():
            print(f"   ⏭️ {idx_name} (exists)")
        elif "does not exist" in str(e).lower() or "not found" in str(e).lower():
            print(f"   ⚠️ {idx_name} - table/column not found")
        else:
            print(f"   ❌ {idx_name}: {e}")

# =============================================================================
# Bildindex Indexes
# =============================================================================
print("\n🔧 Creating indexes for Bildindex tables...")

bi_indexes = [
    # bi_items - main Bildindex items table
    ("idx_bi_items_uri", "bi_items", "bildindex_uri"),
    ("idx_bi_items_url", "bi_items", "bildindex_url"),
    ("idx_bi_items_tier", "bi_items", "tier"),
    
    # bi_buildings - Bildindex items linked to buildings via GND
    ("idx_bi_buildings_uri", "bi_buildings", "bildindex_uri"),
    ("idx_bi_buildings_gnd", "bi_buildings", "building_gnd"),
    
    # bi_painters - Bildindex items linked to painters via GND
    ("idx_bi_painters_uri", "bi_painters", "bildindex_uri"),
    ("idx_bi_painters_gnd", "bi_painters", "painter_gnd"),
    
    # bi_subjects - Bildindex items linked to ICONCLASS subjects
    ("idx_bi_subjects_uri", "bi_subjects", "bildindex_uri"),
    ("idx_bi_subjects_code", "bi_subjects", "iconclass_code"),
    
    # bi_gnd_overlaps - GND overlap summary
    ("idx_bi_gnd_overlaps_uri", "bi_gnd_overlaps", "gnd_uri"),
    ("idx_bi_gnd_overlaps_type", "bi_gnd_overlaps", "gnd_type"),
]

for idx_name, table, column in bi_indexes:
    try:
        con.execute(f"CREATE INDEX {idx_name} ON {table}({column})")
        print(f"   ✅ {idx_name}")
    except Exception as e:
        if "already exists" in str(e).lower():
            print(f"   ⏭️ {idx_name} (exists)")
        elif "does not exist" in str(e).lower() or "not found" in str(e).lower():
            print(f"   ⚠️ {idx_name} - table/column not found")
        else:
            print(f"   ❌ {idx_name}: {e}")

print("\n✅ Indexing complete")

🔧 Creating indexes for CbDD tables...
   ❌ idx_paintings_nfdi: 'NoneType' object has no attribute 'execute'
   ❌ idx_paintings_cbdd: 'NoneType' object has no attribute 'execute'
   ❌ idx_paintings_building: 'NoneType' object has no attribute 'execute'
   ❌ idx_paintings_room: 'NoneType' object has no attribute 'execute'
   ❌ idx_paintings_year_start: 'NoneType' object has no attribute 'execute'
   ❌ idx_paintings_state: 'NoneType' object has no attribute 'execute'
   ❌ idx_pp_nfdi: 'NoneType' object has no attribute 'execute'
   ❌ idx_pp_person: 'NoneType' object has no attribute 'execute'
   ❌ idx_ps_nfdi: 'NoneType' object has no attribute 'execute'
   ❌ idx_ps_subject: 'NoneType' object has no attribute 'execute'
   ❌ idx_bp_building: 'NoneType' object has no attribute 'execute'
   ❌ idx_bp_person: 'NoneType' object has no attribute 'execute'
   ❌ idx_rp_room: 'NoneType' object has no attribute 'execute'
   ❌ idx_rp_person: 'NoneType' object has no attribute 'execute'
   ❌ idx_perso

In [ ]:
# =============================================================================
# CELL 6: Year Parsing Validation
# =============================================================================

print("📅 Year Parsing Statistics:")
print("=" * 60)

year_stats = con.execute("""
    SELECT
        COUNT(*) as total,
        SUM(CASE WHEN year IS NOT NULL AND year != '' THEN 1 ELSE 0 END) as has_year_string,
        SUM(CASE WHEN year_parsed THEN 1 ELSE 0 END) as parsed_successfully,
        SUM(CASE WHEN year IS NOT NULL AND year != '' AND NOT year_parsed THEN 1 ELSE 0 END) as parse_failed,
        SUM(CASE WHEN year_is_approximate THEN 1 ELSE 0 END) as approximate_dates,
        MIN(year_start) as earliest_year,
        MAX(year_end) as latest_year
    FROM paintings
""").fetchone()

total, has_year, parsed, failed, approx, earliest, latest = year_stats

print(f"   Total paintings:        {total:>6}")
print(f"   Has year string:        {has_year:>6} ({100*has_year/total:.1f}%)")
print(f"   ✅ Parsed successfully: {parsed:>6} ({100*parsed/total:.1f}%)")
print(f"   ❌ Parse failed:        {failed:>6} ({100*failed/total:.1f}%)")
print(f"   ~ Approximate dates:    {approx:>6}")
print(f"   📆 Date range:          {earliest} - {latest}")

# Show unparseable years for review
if failed > 0:
    print("\n" + "=" * 60)
    print("⚠️ UNPARSEABLE YEAR STRINGS (for manual review):")
    print("=" * 60)
    
    unparsed = con.execute("""
        SELECT year, COUNT(*) as count
        FROM paintings
        WHERE year IS NOT NULL AND year != '' AND NOT year_parsed
        GROUP BY year
        ORDER BY count DESC
        LIMIT 20
    """).pl()
    
    for row in unparsed.iter_rows(named=True):
        print(f"   [{row['count']:>2}x] {row['year']}")

📅 Year Parsing Statistics:


AttributeError: 'NoneType' object has no attribute 'execute'

In [ ]:
# =============================================================================
# CELL 7: Data Quality Summary
# =============================================================================

print("📊 DATA QUALITY SUMMARY")
print("=" * 70)

# Row counts per table
print("\n📋 Table Row Counts:")
for table in TABLES:
    try:
        count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
        print(f"   {table:<25}: {count:>8,}")
    except:
        print(f"   {table:<25}: (not loaded)")

# Paintings table coverage
print("\n📋 Paintings Column Coverage:")
coverage = con.execute("""
    SELECT
        COUNT(*) as total,
        SUM(CASE WHEN label IS NOT NULL THEN 1 ELSE 0 END) as has_label,
        SUM(CASE WHEN year_parsed THEN 1 ELSE 0 END) as has_year,
        SUM(CASE WHEN lat IS NOT NULL THEN 1 ELSE 0 END) as has_coords,
        SUM(CASE WHEN imageUrl IS NOT NULL THEN 1 ELSE 0 END) as has_image,
        SUM(CASE WHEN cbdd_painting_id IS NOT NULL THEN 1 ELSE 0 END) as has_cbdd,
        SUM(CASE WHEN building_name IS NOT NULL THEN 1 ELSE 0 END) as has_building,
        SUM(CASE WHEN painters IS NOT NULL AND painters != '' THEN 1 ELSE 0 END) as has_painter,
        SUM(CASE WHEN location_state IS NOT NULL THEN 1 ELSE 0 END) as has_state
    FROM paintings
""").fetchone()

total = coverage[0]
cols = ['label', 'year (parsed)', 'coordinates', 'imageUrl', 'CbDD match', 
        'building_name', 'painters', 'location_state']
for i, col in enumerate(cols):
    val = coverage[i + 1]
    pct = 100 * val / total if total > 0 else 0
    bar = "█" * int(pct / 5) + "░" * (20 - int(pct / 5))
    print(f"   {col:<20}: {bar} {val:>5}/{total} ({pct:>5.1f}%)")

# Foreign key integrity
print("\n🔗 Foreign Key Integrity:")

fk_checks = [
    ("painting_persons → paintings", 
     "SELECT COUNT(*) FROM painting_persons pp LEFT JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri WHERE p.nfdi_uri IS NULL"),
    ("painting_subjects → paintings",
     "SELECT COUNT(*) FROM painting_subjects ps LEFT JOIN paintings p ON ps.nfdi_uri = p.nfdi_uri WHERE p.nfdi_uri IS NULL"),
    ("paintings → buildings",
     "SELECT COUNT(*) FROM paintings p LEFT JOIN buildings b ON p.building_id = b.building_id WHERE p.building_id IS NOT NULL AND b.building_id IS NULL"),
    ("paintings → rooms",
     "SELECT COUNT(*) FROM paintings p LEFT JOIN rooms r ON p.room_id = r.room_id WHERE p.room_id IS NOT NULL AND r.room_id IS NULL"),
]

for desc, query in fk_checks:
    try:
        orphans = con.execute(query).fetchone()[0]
        status = "✅" if orphans == 0 else "⚠️"
        print(f"   {status} {desc}: {orphans} orphan(s)")
    except Exception as e:
        print(f"   ⚠️ {desc}: check failed")

📊 DATA QUALITY SUMMARY

📋 Table Row Counts:
   paintings                : (not loaded)
   persons                  : (not loaded)
   buildings                : (not loaded)
   rooms                    : (not loaded)
   ensembles                : (not loaded)
   subjects                 : (not loaded)
   painting_persons         : (not loaded)
   painting_subjects        : (not loaded)
   building_persons         : (not loaded)
   room_persons             : (not loaded)

📋 Paintings Column Coverage:


AttributeError: 'NoneType' object has no attribute 'execute'

In [ ]:
# =============================================================================
# CELL 8: Save and Verify Database
# =============================================================================

# Checkpoint to ensure data is written
con.execute("CHECKPOINT")

# Verify database size
db_size = DUCKDB_PATH.stat().st_size / (1024 * 1024)

print("💾 DATABASE SAVED")
print("=" * 60)
print(f"   📁 Path: {DUCKDB_PATH}")
print(f"   📊 Size: {db_size:.2f} MB")

# Count CbDD tables
cbdd_count = len([t for t in CBDD_TABLES if table_stats.get(t, 0) > 0])
cbdd_rows = sum(table_stats.values())
print(f"\n   📘 CbDD Tables: {cbdd_count}")
print(f"      Total rows: {cbdd_rows:,}")

# Count Bildindex tables
bi_count = len([t for t in BILDINDEX_TABLES if bildindex_stats.get(t, 0) > 0])
bi_rows = sum(bildindex_stats.values())
print(f"\n   📗 Bildindex Tables: {bi_count}")
print(f"      Total rows: {bi_rows:,}")

# List all tables in the database
print("\n📋 All Tables in Database:")
tables_in_db = con.execute("SHOW TABLES").pl()
cbdd_list = []
bi_list = []
for row in tables_in_db.iter_rows(named=True):
    if row['name'].startswith('bi_'):
        bi_list.append(row['name'])
    else:
        cbdd_list.append(row['name'])

print("   CbDD: " + ", ".join(cbdd_list))
print("   Bildindex: " + ", ".join(bi_list))

print("\n✅ Database ready for collaborative use!")
print(f"   Share the file: {DUCKDB_PATH.name}")

AttributeError: 'NoneType' object has no attribute 'execute'

---
# Analysis Queries

Now that the database is loaded, we can run analytical queries using DuckDB SQL with Polars DataFrames as output.

## Data Sources
- **CbDD Tables**: `paintings`, `persons`, `buildings`, `rooms`, `ensembles`, `subjects`, and junction tables
- **Bildindex Tables**: `bi_items`, `bi_buildings`, `bi_painters`, `bi_subjects`, `bi_gnd_overlaps`

## Cross-Reference Capability
Join CbDD and Bildindex data via shared GNDs (German National Library identifiers):
- `paintings.creatorGnds` ↔ `bi_painters.painter_gnd`
- `paintings.locationGnds` ↔ `bi_buildings.building_gnd`

In [ ]:
# =============================================================================
# CELL 9: Geographic Analysis - Paintings by German State
# =============================================================================

print("🗺️ GEOGRAPHIC DISTRIBUTION")
print("=" * 60)

geo_df = con.execute("""
    SELECT 
        location_state,
        COUNT(*) as painting_count,
        COUNT(DISTINCT building_id) as building_count,
        MIN(year_start) as earliest,
        MAX(year_end) as latest,
        ROUND(AVG(year_start), 0) as avg_year
    FROM paintings
    WHERE location_state IS NOT NULL
    GROUP BY location_state
    ORDER BY painting_count DESC
""").pl()

print(geo_df)

# Visualization
if len(geo_df) > 0:
    chart = alt.Chart(geo_df.to_pandas()).mark_bar().encode(
        x=alt.X('painting_count:Q', title='Number of Paintings'),
        y=alt.Y('location_state:N', sort='-x', title='German State'),
        color=alt.Color('painting_count:Q', scale=alt.Scale(scheme='blues'), legend=None),
        tooltip=['location_state', 'painting_count', 'building_count', 'earliest', 'latest']
    ).properties(
        title='Baroque Ceiling Paintings by German State',
        width=500,
        height=400
    )
    ipython_display(chart)

🗺️ GEOGRAPHIC DISTRIBUTION


AttributeError: 'NoneType' object has no attribute 'execute'

In [ ]:
# =============================================================================
# CELL 10: Temporal Analysis - Paintings by Decade
# =============================================================================
# Filter by Getty AAT "ceiling paintings" to exclude buildings

print("📅 TEMPORAL DISTRIBUTION (Ceiling Paintings Only)")
print("=" * 60)

temporal_df = con.execute("""
    SELECT 
        CAST(FLOOR(p.year_start / 10) * 10 AS INTEGER) as decade,
        COUNT(*) as count,
        CAST(SUM(CASE WHEN p.year_is_approximate THEN 1 ELSE 0 END) AS INTEGER) as approximate_count
    FROM paintings p
    JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE p.year_start IS NOT NULL
      AND p.year_start >= 1500 AND p.year_start <= 1900
      AND s.subject_source = 'GETTY_AAT'
      AND s.subject_label = 'ceiling paintings'
    GROUP BY FLOOR(p.year_start / 10) * 10
    ORDER BY decade
""").pl()

print(temporal_df)

# Visualization
if len(temporal_df) > 0:
    chart = alt.Chart(temporal_df.to_pandas()).mark_bar().encode(
        x=alt.X('decade:O', title='Decade'),
        y=alt.Y('count:Q', title='Number of Paintings'),
        color=alt.Color('approximate_count:Q', 
                       scale=alt.Scale(scheme='oranges'),
                       title='Approximate Dates'),
        tooltip=['decade', 'count', 'approximate_count']
    ).properties(
        title='Baroque Ceiling Paintings by Decade',
        width=600,
        height=300
    )
    ipython_display(chart)

📅 TEMPORAL DISTRIBUTION (Ceiling Paintings Only)


AttributeError: 'NoneType' object has no attribute 'execute'

In [ ]:
# =============================================================================
# CELL 10b: Investigate 1540s Paintings
# =============================================================================
# Manual inspection of paintings dated to 1540s - seems unusually early for Baroque

print("🔍 INVESTIGATING 1540s PAINTINGS")
print("=" * 80)

paintings_1540s = con.execute("""
    SELECT 
        p.label,
        p.year,
        p.year_start,
        p.year_end,
        p.year_is_approximate,
        p.building_name,
        p.location_state,
        p.nfdi_uri
    FROM paintings p
    JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE p.year_start >= 1540 AND p.year_start < 1550
      AND s.subject_source = 'GETTY_AAT'
      AND s.subject_label = 'ceiling paintings'
    ORDER BY p.year_start, p.label
""").pl()

print(f"Found {len(paintings_1540s)} paintings from the 1540s:\n")

for i, row in enumerate(paintings_1540s.iter_rows(named=True), 1):
    print(f"{i}. {row['label']}")
    print(f"   📅 Year string: '{row['year']}' → parsed: {row['year_start']}-{row['year_end']} (approx: {row['year_is_approximate']})")
    print(f"   🏛️ {row['building_name']}, {row['location_state']}")
    print(f"   🔗 {row['nfdi_uri']}")
    print()

🔍 INVESTIGATING 1540s PAINTINGS
Found 375 paintings from the 1540s:

1. Christi Himmelfahrt
   📅 Year string: '1540-1542, 1780' → parsed: 1540-1542 (approx: False)
   🏛️ Harburg, Schlosskirche St. Michael, Bayern
   🔗 https://www.deckenmalerei.eu/f59c1628-8008-4e88-a63e-d31509950715

2. Diana und die Jagdhunde
   📅 Year string: '1540-1542' → parsed: 1540-1542 (approx: False)
   🏛️ Landshut, Stadtresidenz, Bayern
   🔗 https://www.deckenmalerei.eu/fe1d8777-2f6c-458f-8faf-d14f979c0cc9

3. Diana wird von Aktäon beim Bad überrascht
   📅 Year string: '1540-1542' → parsed: 1540-1542 (approx: False)
   🏛️ Landshut, Stadtresidenz, Bayern
   🔗 https://www.deckenmalerei.eu/5e2b1711-3732-4cbb-a2e0-58c9c7c50518

4. Die Auferstehung Christi (Supraporte)
   📅 Year string: '1540-1542' → parsed: 1540-1542 (approx: False)
   🏛️ Landshut, Stadtresidenz, Bayern
   🔗 https://www.deckenmalerei.eu/7bae7ccd-4086-4257-aeb4-d1abdffabfd0

5. Die Decken- und Wandmalerei im Gerichtszimmer
   📅 Year string: '1540, 

In [ ]:
# =============================================================================
# CELL 11: Top Painters Analysis
# =============================================================================
# Using the painting_persons junction table for accurate individual painter counts.
# The paintings.painters column contains pipe-separated values for collaborative
# works, which would incorrectly group "A | B" as a single painter.

print("🎨 TOP PAINTERS")
print("=" * 60)

painters_df = con.execute("""
    SELECT 
        pp.person_name,
        COUNT(DISTINCT pp.nfdi_uri) as painting_count,
        COUNT(DISTINCT p.building_id) as building_count,
        MIN(p.year_start) as active_from,
        MAX(p.year_end) as active_until
    FROM painting_persons pp
    JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
    WHERE pp.role = 'PAINTER'
    GROUP BY pp.person_name
    HAVING COUNT(DISTINCT pp.nfdi_uri) >= 2
    ORDER BY painting_count DESC
    LIMIT 25
""").pl()

print(painters_df)

# Visualization
if len(painters_df) > 0:
    chart = alt.Chart(painters_df.head(15).to_pandas()).mark_bar().encode(
        x=alt.X('painting_count:Q', title='Number of Paintings'),
        y=alt.Y('person_name:N', sort='-x', title='Painter'),
        color=alt.Color('building_count:Q', 
                       scale=alt.Scale(scheme='viridis'),
                       title='Buildings'),
        tooltip=['person_name', 'painting_count', 'building_count', 'active_from', 'active_until']
    ).properties(
        title='Most Prolific Baroque Ceiling Painters',
        width=500,
        height=400
    )
    ipython_display(chart)

🎨 TOP PAINTERS
shape: (25, 5)
┌──────────────────────────────┬────────────────┬────────────────┬─────────────┬──────────────┐
│ person_name                  ┆ painting_count ┆ building_count ┆ active_from ┆ active_until │
│ ---                          ┆ ---            ┆ ---            ┆ ---         ┆ ---          │
│ str                          ┆ i64            ┆ i64            ┆ i32         ┆ i32          │
╞══════════════════════════════╪════════════════╪════════════════╪═════════════╪══════════════╡
│ Harms, Johann Oswald         ┆ 119            ┆ 7              ┆ 1569        ┆ 1780         │
│ Lammers, Seivert             ┆ 82             ┆ 4              ┆ 1670        ┆ 1750         │
│ Castelli, Carlo Ludovico     ┆ 70             ┆ 8              ┆ 1683        ┆ 1770         │
│ Asam, Cosmas Damian          ┆ 63             ┆ 7              ┆ 1670        ┆ 1948         │
│ Ritter, Johann Heinrich      ┆ 58             ┆ 6              ┆ 1580        ┆ 1780         │
│ …       

alt.Chart(...)

In [ ]:
# =============================================================================
# CELL 12: Building Function Analysis
# =============================================================================
# Join with buildings table to get the function column

print("🏛️ BUILDING FUNCTION ANALYSIS")
print("=" * 60)

building_func_df = con.execute("""
    SELECT 
        b.function as building_function,
        COUNT(*) as painting_count,
        COUNT(DISTINCT p.building_id) as building_count,
        ROUND(AVG(p.year_start), 0) as avg_year
    FROM paintings p
    JOIN buildings b ON p.building_id = b.building_id
    WHERE b.function IS NOT NULL
    GROUP BY b.function
    ORDER BY painting_count DESC
    LIMIT 20
""").pl()

print(building_func_df)

# Visualization - Pie/Donut chart
if len(building_func_df) > 0:
    chart = alt.Chart(building_func_df.head(10).to_pandas()).mark_arc(innerRadius=50).encode(
        theta=alt.Theta('painting_count:Q'),
        color=alt.Color('building_function:N', title='Building Type'),
        tooltip=['building_function', 'painting_count', 'building_count']
    ).properties(
        title='Paintings by Building Function',
        width=400,
        height=400
    )
    ipython_display(chart)

🏛️ BUILDING FUNCTION ANALYSIS
shape: (20, 4)
┌─────────────────────────────────┬────────────────┬────────────────┬──────────┐
│ building_function               ┆ painting_count ┆ building_count ┆ avg_year │
│ ---                             ┆ ---            ┆ ---            ┆ ---      │
│ str                             ┆ i64            ┆ i64            ┆ f64      │
╞═════════════════════════════════╪════════════════╪════════════════╪══════════╡
│ Residenz                        ┆ 655            ┆ 22             ┆ 1616.0   │
│ Hauptresidenz                   ┆ 623            ┆ 42             ┆ 1680.0   │
│ Herrenhaus / Rittersitz / Guts… ┆ 424            ┆ 71             ┆ 1716.0   │
│ Schloss                         ┆ 380            ┆ 39             ┆ 1713.0   │
│ Lustschloss                     ┆ 164            ┆ 20             ┆ 1724.0   │
│ …                               ┆ …              ┆ …              ┆ …        │
│ Kapelle                         ┆ 40             ┆ 2          

alt.Chart(...)

In [ ]:
# =============================================================================
# CELL 13: Subject/Theme Analysis (ICONCLASS only)
# =============================================================================
# Filter: Only ceiling paintings (via Getty AAT), display only ICONCLASS subjects

print("📚 SUBJECT/THEME ANALYSIS (ICONCLASS)")
print("=" * 60)

subjects_df = con.execute("""
    SELECT 
        s.subject_label,
        COUNT(DISTINCT ps.nfdi_uri) as painting_count
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    -- Only include paintings that are classified as "ceiling paintings" by Getty AAT
    WHERE ps.nfdi_uri IN (
        SELECT ps2.nfdi_uri 
        FROM painting_subjects ps2
        JOIN subjects s2 ON ps2.subject_uri = s2.subject_uri
        WHERE s2.subject_source = 'GETTY_AAT' 
          AND s2.subject_label = 'ceiling paintings'
    )
    -- Only show ICONCLASS subjects (not Getty AAT)
    AND s.subject_source = 'ICONCLASS'
    AND s.subject_label IS NOT NULL
    GROUP BY s.subject_label
    ORDER BY painting_count DESC
    LIMIT 30
""").pl()

print(subjects_df)

# Visualization
if len(subjects_df) > 0:
    chart = alt.Chart(subjects_df.head(20).to_pandas()).mark_bar().encode(
        x=alt.X('painting_count:Q', title='Number of Paintings'),
        y=alt.Y('subject_label:N', sort='-x', title='ICONCLASS Subject'),
        color=alt.Color('painting_count:Q', scale=alt.Scale(scheme='tealblues'), legend=None),
        tooltip=['subject_label', 'painting_count']
    ).properties(
        title='Most Common ICONCLASS Subjects in Baroque Ceiling Paintings',
        width=800,
        height=400
    )
    ipython_display(chart)

📚 SUBJECT/THEME ANALYSIS (ICONCLASS)
shape: (30, 2)
┌─────────────────────────────────┬────────────────┐
│ subject_label                   ┆ painting_count │
│ ---                             ┆ ---            │
│ str                             ┆ i64            │
╞═════════════════════════════════╪════════════════╡
│ cupids: 'amores', 'amoretti', … ┆ 331            │
│ clouds                          ┆ 180            │
│ garland, wreath                 ┆ 80             │
│ trees                           ┆ 71             │
│ column, pillar ~ architecture   ┆ 62             │
│ …                               ┆ …              │
│ dog                             ┆ 36             │
│ sceptre, staff (symbol of sove… ┆ 36             │
│ helmet                          ┆ 36             │
│ winter, 'Hyems'; 'Inverno' (Ri… ┆ 35             │
│ (story of) Minerva (Pallas, At… ┆ 35             │
└─────────────────────────────────┴────────────────┘


alt.Chart(...)

In [ ]:
# =============================================================================
# CELL 13b: ICONCLASS Category Analysis
# =============================================================================
# Categorize ICONCLASS subjects by their top-level category (first digit)
# ICONCLASS hierarchy: https://iconclass.org/

# Category mapping (first digit of ICONCLASS notation)
ICONCLASS_CATEGORIES = {
    '0': 'Abstract, Non-representational Art',
    '1': 'Religion and Magic',
    '2': 'Nature',
    '3': 'Human Being, Man in General',
    '4': 'Society, Civilization, Culture',
    '5': 'Abstract Ideas and Concepts',
    '6': 'History',
    '7': 'Bible',
    '8': 'Literature',
    '9': 'Classical Mythology and Ancient History'
}

print("📊 ICONCLASS CATEGORY ANALYSIS")
print("=" * 70)

# Query to extract ICONCLASS code and categorize
category_df = con.execute("""
    WITH iconclass_parsed AS (
        SELECT 
            s.subject_label,
            s.subject_uri,
            -- Extract ICONCLASS code from URI (after iconclass.org/)
            REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code,
            COUNT(DISTINCT ps.nfdi_uri) as painting_count
        FROM painting_subjects ps
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        -- Only ceiling paintings
        WHERE ps.nfdi_uri IN (
            SELECT ps2.nfdi_uri 
            FROM painting_subjects ps2
            JOIN subjects s2 ON ps2.subject_uri = s2.subject_uri
            WHERE s2.subject_source = 'GETTY_AAT' 
              AND s2.subject_label = 'ceiling paintings'
        )
        AND s.subject_source = 'ICONCLASS'
        AND s.subject_label IS NOT NULL
        GROUP BY s.subject_label, s.subject_uri
    )
    SELECT 
        subject_label,
        iconclass_code,
        -- Extract first character for category
        SUBSTRING(iconclass_code, 1, 1) as category_digit,
        painting_count
    FROM iconclass_parsed
    WHERE iconclass_code IS NOT NULL
    ORDER BY painting_count DESC
""").pl()

# Add category names using Python
category_df = category_df.with_columns(
    pl.col('category_digit').replace(ICONCLASS_CATEGORIES).alias('category_name')
)

print(f"Total ICONCLASS subjects: {len(category_df)}")
print(f"\n📋 Category Distribution:")

# Summary by category
category_summary = category_df.group_by('category_digit', 'category_name').agg([
    pl.count().alias('subject_count'),
    pl.sum('painting_count').alias('total_paintings')
]).sort('total_paintings', descending=True)

print(category_summary)

# Prepare data for visualization
viz_df = category_df.head(50).to_pandas()  # Top 50 subjects

# Create a treemap-style visualization using layered bar chart
# Color by category, sorted by painting count within category

chart_bars = alt.Chart(viz_df).mark_bar().encode(
    x=alt.X('painting_count:Q', title='Number of Paintings'),
    y=alt.Y('subject_label:N', sort='-x', title='ICONCLASS Subject'),
    color=alt.Color('category_name:N', 
                   title='ICONCLASS Category',
                   scale=alt.Scale(scheme='category10')),
    tooltip=[
        alt.Tooltip('subject_label:N', title='Subject'),
        alt.Tooltip('iconclass_code:N', title='ICONCLASS Code'),
        alt.Tooltip('category_name:N', title='Category'),
        alt.Tooltip('painting_count:Q', title='Paintings')
    ]
).properties(
    title='Top 50 ICONCLASS Subjects by Category',
    width=700,
    height=800
)

ipython_display(chart_bars)

📊 ICONCLASS CATEGORY ANALYSIS
Total ICONCLASS subjects: 4080

📋 Category Distribution:
shape: (9, 4)
┌────────────────┬─────────────────────────────────┬───────────────┬─────────────────┐
│ category_digit ┆ category_name                   ┆ subject_count ┆ total_paintings │
│ ---            ┆ ---                             ┆ ---           ┆ ---             │
│ str            ┆ str                             ┆ u32           ┆ i64             │
╞════════════════╪═════════════════════════════════╪═══════════════╪═════════════════╡
│ 4              ┆ Society, Civilization, Culture  ┆ 1116          ┆ 4387            │
│ 2              ┆ Nature                          ┆ 580           ┆ 2777            │
│ 9              ┆ Classical Mythology and Ancien… ┆ 787           ┆ 2514            │
│ 1              ┆ Religion and Magic              ┆ 566           ┆ 1433            │
│ 7              ┆ Bible                           ┆ 457           ┆ 668             │
│ 3              ┆ Human Bein

C:\Users\thano\AppData\Local\Temp\ipykernel_48220\1495220835.py:68: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('subject_count'),


alt.Chart(...)

In [ ]:
# =============================================================================
# CELL 13c: ICONCLASS Category Summary Visualization
# =============================================================================
# Donut chart showing category distribution + faceted detail view

# Category summary donut chart
category_summary_pd = category_summary.to_pandas()

donut = alt.Chart(category_summary_pd).mark_arc(innerRadius=60, outerRadius=120).encode(
    theta=alt.Theta('total_paintings:Q', stack=True),
    color=alt.Color('category_name:N', 
                   title='ICONCLASS Category',
                   scale=alt.Scale(scheme='category10'),
                   legend=alt.Legend(orient='right', titleLimit=200, labelLimit=200)),
    tooltip=[
        alt.Tooltip('category_name:N', title='Category'),
        alt.Tooltip('subject_count:Q', title='Unique Subjects'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings')
    ]
).properties(
    title='ICONCLASS Categories in Baroque Ceiling Paintings',
    width=300,
    height=300
)

# Text labels in center showing totals
text_center = alt.Chart(category_summary_pd).mark_text(
    size=20, fontWeight='bold'
).encode(
    text=alt.value(f"{category_summary_pd['total_paintings'].sum():,}")
).properties(width=300, height=300)

text_label = alt.Chart(category_summary_pd).mark_text(
    size=12, dy=25
).encode(
    text=alt.value("total subject\noccurrences")
).properties(width=300, height=300)

donut_with_text = donut + text_center + text_label

# Top subjects per category (faceted view)
top_per_category = category_df.group_by('category_name').head(5).to_pandas()

faceted = alt.Chart(top_per_category).mark_bar().encode(
    x=alt.X('painting_count:Q', title='Paintings'),
    y=alt.Y('subject_label:N', sort='-x', title=None),
    color=alt.Color('category_name:N', legend=None),
    tooltip=['subject_label', 'iconclass_code', 'painting_count']
).properties(
    width=150,
    height=120
).facet(
    column=alt.Column('category_name:N', title=None, header=alt.Header(labelFontSize=10, labelAngle=0))
).properties(
    title='Top 5 Subjects per ICONCLASS Category'
).resolve_scale(
    y='independent'
)

ipython_display(donut_with_text)
ipython_display(faceted)

alt.LayerChart(...)

alt.FacetChart(...)

In [ ]:
# =============================================================================
# CELL 13d: Painter Topic Profile Analysis (Full ICONCLASS Hierarchy)
# =============================================================================
# Analyze each painter's thematic preferences using FULL ICONCLASS codes
# 
# ICONCLASS Structure: e.g., "92D1521" = 
#   9 (Mythology) > 92 (specific) > 92D (gods) > 92D1 (Apollo) > etc.
# 
# Approach:
# 1. For each painter, collect all FULL ICONCLASS codes from their paintings
# 2. Compute hierarchical similarity between painters using code prefix matching
# 3. Distance metric: weighted prefix similarity (deeper matches = higher weight)

import numpy as np
from collections import Counter

print("🎨 PAINTER TOPIC PROFILE ANALYSIS (Full ICONCLASS Hierarchy)")
print("=" * 70)

# Step 1: Get all (painter, iconclass_code) pairs for ceiling paintings
painter_subjects_df = con.execute("""
    SELECT 
        pp.person_name as painter,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code,
        s.subject_label
    FROM painting_persons pp
    JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
    JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE pp.role = 'PAINTER'
      AND s.subject_source = 'ICONCLASS'
      AND s.subject_label IS NOT NULL
      -- Only ceiling paintings
      AND p.nfdi_uri IN (
          SELECT ps2.nfdi_uri 
          FROM painting_subjects ps2
          JOIN subjects s2 ON ps2.subject_uri = s2.subject_uri
          WHERE s2.subject_source = 'GETTY_AAT' 
            AND s2.subject_label = 'ceiling paintings'
      )
""").pl()

print(f"📊 Total painter-subject pairs: {len(painter_subjects_df):,}")

# Step 2: Filter to painters with enough data (min 20 subject occurrences)
painter_counts = painter_subjects_df.group_by('painter').len()
active_painters = painter_counts.filter(pl.col('len') >= 20)['painter'].to_list()
print(f"📊 Painters with ≥20 subject occurrences: {len(active_painters)}")

# Filter to active painters
painter_subjects_filtered = painter_subjects_df.filter(pl.col('painter').is_in(active_painters))

# Step 3: Build ICONCLASS code frequency vectors for each painter
# Store as dict: painter -> {code: count}
painter_code_vectors = {}
for painter in active_painters:
    codes = painter_subjects_filtered.filter(pl.col('painter') == painter)['iconclass_code'].to_list()
    painter_code_vectors[painter] = Counter(codes)

# Also extract all unique codes for analysis
all_unique_codes = set()
for codes in painter_code_vectors.values():
    all_unique_codes.update(codes.keys())

print(f"📊 Unique ICONCLASS codes across all painters: {len(all_unique_codes)}")

# Step 4: Define hierarchical prefix extraction function
def get_all_prefixes(code):
    """
    Extract all hierarchical prefixes from an ICONCLASS code.
    e.g., "92D1521" -> ["9", "92", "92D", "92D1", "92D15", "92D152", "92D1521"]
    """
    if not code:
        return []
    prefixes = []
    current = ""
    for char in code:
        current += char
        # Only add meaningful prefixes (skip URL-encoded parts like %28)
        if not current.endswith('%'):
            prefixes.append(current)
    return prefixes

# Build prefix frequency vectors (weighted by depth)
# Deeper prefixes get exponentially higher weights
def build_prefix_vector(code_counter):
    """Build a prefix-weighted vector from ICONCLASS code frequencies"""
    prefix_weights = Counter()
    for code, count in code_counter.items():
        prefixes = get_all_prefixes(code)
        for i, prefix in enumerate(prefixes):
            # Weight increases with depth: 2^(depth-1)
            depth_weight = 2 ** i
            prefix_weights[prefix] += count * depth_weight
    return prefix_weights

painter_prefix_vectors = {p: build_prefix_vector(codes) for p, codes in painter_code_vectors.items()}

# Collect all prefixes across all painters
all_prefixes = set()
for pv in painter_prefix_vectors.values():
    all_prefixes.update(pv.keys())

print(f"📊 Total unique prefixes (all depths): {len(all_prefixes)}")

# Step 5: Build a matrix for painters x top-level categories (for overview)
ICONCLASS_CATEGORIES = {
    '0': 'Abstract, Non-representational Art',
    '1': 'Religion and Magic',
    '2': 'Nature',
    '3': 'Human Being, Man in General',
    '4': 'Society, Civilization, Culture',
    '5': 'Abstract Ideas and Concepts',
    '6': 'History',
    '7': 'Bible',
    '8': 'Literature',
    '9': 'Classical Mythology and Ancient History'
}

# Also build the top-level category matrix for visualization
def get_top_category(code):
    if code and len(code) > 0 and code[0].isdigit():
        return code[0]
    return None

painter_subjects_with_cat = painter_subjects_filtered.with_columns(
    pl.col('iconclass_code').map_elements(get_top_category, return_dtype=pl.Utf8).alias('category')
).filter(pl.col('category').is_not_null())

category_counts = painter_subjects_with_cat.group_by(['painter', 'category']).len().rename({'len': 'count'})
painter_matrix = category_counts.pivot(
    values='count',
    index='painter', 
    on='category'
).fill_null(0)

painter_totals = painter_subjects_with_cat.group_by('painter').len().rename({'len': 'total'})
painter_matrix = painter_matrix.join(painter_totals, on='painter')

# Show sample of the most common full codes per painter
print("\n📋 Sample: Top 5 most used ICONCLASS codes by top painter:")
top_painter = active_painters[0] if active_painters else None
if top_painter:
    top_codes = painter_code_vectors[top_painter].most_common(5)
    for code, count in top_codes:
        label = painter_subjects_filtered.filter(
            (pl.col('painter') == top_painter) & (pl.col('iconclass_code') == code)
        )['subject_label'].to_list()[0]
        print(f"  {code}: {label} ({count}x)")

🎨 PAINTER TOPIC PROFILE ANALYSIS (Full ICONCLASS Hierarchy)
📊 Total painter-subject pairs: 9,651
📊 Painters with ≥20 subject occurrences: 108
📊 Unique ICONCLASS codes across all painters: 2691
📊 Total unique prefixes (all depths): 10933

📋 Sample: Top 5 most used ICONCLASS codes by top painter:
  42: family, descendance (5x)
  42F5: house personnel, servants (3x)
  43C11124: hunt ~ hoofed animals (2x)
  24C17: Saturn (planet) (2x)
  46C131: riding a horse, ass, or mule; rider, horseman (2x)


In [ ]:
# =============================================================================
# CELL 13e: Painter Topic Metrics (Full Hierarchical Analysis)
# =============================================================================
# Compute diversity using full ICONCLASS codes, not just top-level categories

# Get category columns (digits 0-9) for top-level view
all_categories = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
available_categories = [c for c in all_categories if c in painter_matrix.columns]

# Convert to numpy for calculations
painters_list = painter_matrix['painter'].to_list()
totals = painter_matrix['total'].to_numpy()

# Build percentage matrix for top-level categories (for visualization)
pct_data = {}
for cat in available_categories:
    if cat in painter_matrix.columns:
        pct_data[cat] = (painter_matrix[cat].to_numpy() / totals * 100).round(1)
    else:
        pct_data[cat] = np.zeros(len(painters_list))

# =============================================================================
# NEW: Hierarchical Diversity Score using FULL ICONCLASS codes
# =============================================================================
# Instead of just top-level category entropy, we compute:
# 1. Code-level entropy: diversity across unique full ICONCLASS codes
# 2. Depth-weighted diversity: considers how deeply specialized the painter is

def shannon_entropy(probs):
    """Compute Shannon entropy from probability distribution"""
    probs = np.array(probs)
    probs = probs[probs > 0]
    if len(probs) == 0:
        return 0
    probs = probs / probs.sum()
    return -np.sum(probs * np.log2(probs))

def compute_hierarchical_diversity(code_counter):
    """
    Compute hierarchical diversity score that considers:
    1. Number of unique codes at each hierarchy level
    2. How evenly distributed codes are at each level
    """
    if not code_counter:
        return 0, 0, 0
    
    # Group codes by prefix length (hierarchy depth)
    level_codes = {}  # depth -> set of prefixes at that depth
    for code in code_counter.keys():
        prefixes = get_all_prefixes(code)
        for depth, prefix in enumerate(prefixes):
            if depth not in level_codes:
                level_codes[depth] = Counter()
            level_codes[depth][prefix] += code_counter[code]
    
    # Compute entropy at each level
    level_entropies = []
    level_unique_counts = []
    for depth in sorted(level_codes.keys()):
        counts = list(level_codes[depth].values())
        level_entropies.append(shannon_entropy(counts))
        level_unique_counts.append(len(level_codes[depth]))
    
    # Aggregate metrics
    total_unique_codes = len(code_counter)
    avg_entropy = np.mean(level_entropies) if level_entropies else 0
    max_depth = max(level_codes.keys()) if level_codes else 0
    
    return total_unique_codes, avg_entropy, max_depth

# Compute metrics for each painter
hierarchical_metrics = []
for painter in painters_list:
    code_counter = painter_code_vectors.get(painter, Counter())
    unique_codes, avg_entropy, max_depth = compute_hierarchical_diversity(code_counter)
    
    # Also compute top-level category stats
    top_counts = [painter_matrix[cat].to_numpy()[painters_list.index(painter)] 
                  if cat in painter_matrix.columns else 0 
                  for cat in available_categories]
    
    top_level_entropy = shannon_entropy(top_counts)
    
    # Dominant category
    if sum(top_counts) > 0:
        max_idx = np.argmax(top_counts)
        dominant_cat = available_categories[max_idx]
        dominant_pct = top_counts[max_idx] / sum(top_counts) * 100
    else:
        dominant_cat = None
        dominant_pct = 0
    
    hierarchical_metrics.append({
        'painter': painter,
        'unique_codes': unique_codes,
        'avg_level_entropy': avg_entropy,
        'max_depth': max_depth,
        'top_level_entropy': top_level_entropy,
        'dominant_category': dominant_cat,
        'dominant_pct': dominant_pct
    })

metrics_df = pl.DataFrame(hierarchical_metrics)

# Normalize diversity scores
max_unique = metrics_df['unique_codes'].max()
max_top_entropy = np.log2(len(available_categories))

# Create final profiles dataframe
painter_profiles = pl.DataFrame({
    'painter': painters_list,
    'total_subjects': totals,
}).join(metrics_df, on='painter')

# Add combined diversity score (weighted combination of metrics)
painter_profiles = painter_profiles.with_columns([
    (pl.col('unique_codes') / max_unique * 50 + 
     pl.col('avg_level_entropy') / 5 * 30 +  # Avg entropy rarely exceeds 5
     pl.col('top_level_entropy') / max_top_entropy * 20
    ).alias('diversity_score'),
    pl.col('dominant_category').replace(ICONCLASS_CATEGORIES).alias('dominant_category_name')
])

# Normalize to 0-100
max_div = painter_profiles['diversity_score'].max()
painter_profiles = painter_profiles.with_columns(
    (pl.col('diversity_score') / max_div * 100).alias('diversity_normalized')
)

# Sort by total subjects
painter_profiles = painter_profiles.sort('total_subjects', descending=True)

# Also keep entropies list for compatibility with heatmap
entropies = painter_profiles['diversity_score'].to_list()

print("📊 PAINTER TOPIC PROFILES (Full Hierarchical Analysis)")
print("=" * 100)
print(painter_profiles.select([
    'painter', 'total_subjects', 'unique_codes', 'dominant_category_name', 
    'dominant_pct', 'diversity_normalized'
]).head(20))

print(f"\n📈 Hierarchical Diversity Statistics:")
print(f"   Unique ICONCLASS codes range: {metrics_df['unique_codes'].min()} - {metrics_df['unique_codes'].max()}")
print(f"   Average unique codes per painter: {metrics_df['unique_codes'].mean():.1f}")
most_diverse = painter_profiles.sort('diversity_normalized', descending=True).head(1)
most_focused = painter_profiles.sort('diversity_normalized').head(1)
print(f"   Most diverse painter: {most_diverse['painter'][0]} ({most_diverse['unique_codes'][0]} unique codes)")
print(f"   Most focused painter: {most_focused['painter'][0]} ({most_focused['unique_codes'][0]} unique codes)")

📊 PAINTER TOPIC PROFILES (Full Hierarchical Analysis)
shape: (20, 6)
┌─────────────────┬────────────────┬──────────────┬────────────────┬──────────────┬────────────────┐
│ painter         ┆ total_subjects ┆ unique_codes ┆ dominant_categ ┆ dominant_pct ┆ diversity_norm │
│ ---             ┆ ---            ┆ ---          ┆ ory_name       ┆ ---          ┆ alized         │
│ str             ┆ u32            ┆ i64          ┆ ---            ┆ f64          ┆ ---            │
│                 ┆                ┆              ┆ str            ┆              ┆ f64            │
╞═════════════════╪════════════════╪══════════════╪════════════════╪══════════════╪════════════════╡
│ Hermann, Franz  ┆ 556            ┆ 298          ┆ Society,       ┆ 48.021583    ┆ 100.0          │
│ Georg           ┆                ┆              ┆ Civilization,  ┆              ┆                │
│                 ┆                ┆              ┆ Culture        ┆              ┆                │
│ Castelli, Carlo ┆ 37

In [ ]:
# =============================================================================
# CELL 13f: Painter Topic Heatmap Visualization
# =============================================================================
# Create a heatmap showing topic distribution for top painters

# Prepare data for heatmap - top 25 painters by total subjects
top_n = 25
top_painters = painter_profiles.head(top_n)['painter'].to_list()

# Build long-form data for Altair
heatmap_data = []
for painter in top_painters:
    idx = painters_list.index(painter)
    total = totals[idx]
    diversity = entropies[idx]
    
    for cat in available_categories:
        if cat in painter_matrix.columns:
            count = painter_matrix.filter(pl.col('painter') == painter)[cat].to_numpy()[0]
            pct = count / total * 100 if total > 0 else 0
        else:
            pct = 0
        
        heatmap_data.append({
            'painter': painter,
            'category': cat,
            'category_name': ICONCLASS_CATEGORIES.get(cat, cat),
            'percentage': round(pct, 1),
            'count': int(count) if cat in painter_matrix.columns else 0,
            'diversity_score': round(diversity, 2)
        })

heatmap_df = pl.DataFrame(heatmap_data).to_pandas()

# Sort painters by total subjects (already sorted in painter_profiles)
painter_order = top_painters

# Create heatmap
heatmap = alt.Chart(heatmap_df).mark_rect().encode(
    x=alt.X('category_name:N', 
            title='ICONCLASS Category',
            sort=['Abstract, Non-representational Art', 'Religion and Magic', 'Nature',
                  'Human Being, Man in General', 'Society, Civilization, Culture',
                  'Abstract Ideas and Concepts', 'History', 'Bible', 'Literature',
                  'Classical Mythology and Ancient History'],
            axis=alt.Axis(labelAngle=-45, labelLimit=150)),
    y=alt.Y('painter:N', title='Painter', sort=painter_order),
    color=alt.Color('percentage:Q', 
                   scale=alt.Scale(scheme='viridis', domain=[0, 60]),
                   title='% of Subjects'),
    tooltip=[
        alt.Tooltip('painter:N', title='Painter'),
        alt.Tooltip('category_name:N', title='Category'),
        alt.Tooltip('percentage:Q', title='Percentage'),
        alt.Tooltip('count:Q', title='Count'),
        alt.Tooltip('diversity_score:Q', title='Diversity Score')
    ]
).properties(
    title=f'Topic Distribution Heatmap (Top {top_n} Painters by Subject Count)',
    width=600,
    height=600
)

# Add diversity score as a separate bar chart
diversity_bars = alt.Chart(
    painter_profiles.head(top_n).to_pandas()
).mark_bar(color='steelblue').encode(
    x=alt.X('diversity_normalized:Q', title='Diversity Score (0-100)', scale=alt.Scale(domain=[0, 100])),
    y=alt.Y('painter:N', title=None, sort=painter_order, axis=None),
    tooltip=[
        alt.Tooltip('painter:N', title='Painter'),
        alt.Tooltip('diversity_normalized:Q', title='Diversity %'),
        alt.Tooltip('dominant_category_name:N', title='Dominant Category')
    ]
).properties(
    width=150,
    height=600,
    title='Diversity'
)

# Combine horizontally
combined = alt.hconcat(heatmap, diversity_bars).resolve_scale(y='shared')

ipython_display(combined)

alt.HConcatChart(...)

In [ ]:
# =============================================================================
# CELL 13g: Painter Similarity Analysis (Hierarchical ICONCLASS Distance)
# =============================================================================
# Compute distance using FULL ICONCLASS hierarchy with weighted prefix matching
# 
# Key insight: Two painters who both depict "92D15" (Apollo with lyre) are MORE
# similar than two who share only "9" (both mythology) 

import numpy as np

print("🔗 PAINTER SIMILARITY ANALYSIS (Full Hierarchical Distance)")
print("=" * 70)

# =============================================================================
# Hierarchical Jaccard Similarity with Depth Weighting
# =============================================================================
def hierarchical_similarity(codes1, codes2, max_weight_depth=8):
    """
    Compute similarity between two sets of ICONCLASS codes using hierarchical prefix matching.
    
    For each pair of codes, similarity is based on the longest common prefix.
    Deeper shared prefixes indicate higher similarity.
    
    Returns a value between 0 (no overlap) and 1 (identical code sets).
    """
    if not codes1 or not codes2:
        return 0.0
    
    # Get all prefixes from both code sets (with counts)
    prefixes1 = Counter()
    prefixes2 = Counter()
    
    for code, count in codes1.items():
        for i, prefix in enumerate(get_all_prefixes(code)[:max_weight_depth]):
            depth_weight = (i + 1) ** 2  # Quadratic depth weighting
            prefixes1[prefix] += count * depth_weight
    
    for code, count in codes2.items():
        for i, prefix in enumerate(get_all_prefixes(code)[:max_weight_depth]):
            depth_weight = (i + 1) ** 2
            prefixes2[prefix] += count * depth_weight
    
    # Compute weighted Jaccard-like similarity
    all_prefixes = set(prefixes1.keys()) | set(prefixes2.keys())
    
    intersection_sum = 0
    union_sum = 0
    
    for prefix in all_prefixes:
        v1 = prefixes1.get(prefix, 0)
        v2 = prefixes2.get(prefix, 0)
        intersection_sum += min(v1, v2)
        union_sum += max(v1, v2)
    
    return intersection_sum / union_sum if union_sum > 0 else 0.0

# Compute pairwise hierarchical similarity for top painters
top_n_sim = 20
top_painters_sim = painter_profiles.head(top_n_sim)['painter'].to_list()

print(f"Computing hierarchical similarity for top {top_n_sim} painters...")
print(f"Painters: {top_painters_sim}")

# Build similarity matrix
hierarchical_sim_matrix = np.zeros((top_n_sim, top_n_sim))

for i, p1 in enumerate(top_painters_sim):
    for j, p2 in enumerate(top_painters_sim):
        if i == j:
            hierarchical_sim_matrix[i, j] = 1.0
        elif j > i:
            sim = hierarchical_similarity(
                painter_code_vectors.get(p1, {}),
                painter_code_vectors.get(p2, {})
            )
            hierarchical_sim_matrix[i, j] = sim
            hierarchical_sim_matrix[j, i] = sim

print("✅ Similarity matrix computed!")

# Prepare data for heatmap
sim_data = []
for i, p1 in enumerate(top_painters_sim):
    for j, p2 in enumerate(top_painters_sim):
        sim_data.append({
            'painter_1': p1,
            'painter_2': p2,
            'similarity': float(hierarchical_sim_matrix[i, j])  # Ensure it's a Python float
        })

sim_df = pl.DataFrame(sim_data).to_pandas()

# Debug: Check data types
print(f"\nDataFrame dtypes:\n{sim_df.dtypes}")
print(f"\nSample data:\n{sim_df.head()}")

# Create similarity heatmap - using simplest possible encoding
similarity_heatmap = alt.Chart(sim_df).mark_rect().encode(
    x=alt.X('painter_1:N', title='Painter', sort=top_painters_sim,
            axis=alt.Axis(labelAngle=-45, labelLimit=150)),
    y=alt.Y('painter_2:N', title='Painter', sort=top_painters_sim,
            axis=alt.Axis(labelLimit=150)),
    color=alt.Color('similarity:Q', 
                   scale=alt.Scale(scheme='redyellowgreen', domain=[0, 1]),
                   title='Similarity')
).properties(
    title='Painter Topic Similarity (Weighted Hierarchical ICONCLASS Distance)',
    width=550,
    height=550
)

ipython_display(similarity_heatmap)

# Find most similar and most different pairs
sim_pairs = []
for i in range(len(top_painters_sim)):
    for j in range(i+1, len(top_painters_sim)):
        sim_pairs.append({
            'painter_1': top_painters_sim[i],
            'painter_2': top_painters_sim[j],
            'similarity': hierarchical_sim_matrix[i, j]
        })

sim_pairs_df = pl.DataFrame(sim_pairs).sort('similarity', descending=True)

print("\n🔗 Most Similar Painter Pairs (by full ICONCLASS hierarchy):")
print("="*70)
for row in sim_pairs_df.head(5).iter_rows(named=True):
    # Find shared codes
    codes1 = set(painter_code_vectors.get(row['painter_1'], {}).keys())
    codes2 = set(painter_code_vectors.get(row['painter_2'], {}).keys())
    shared = len(codes1 & codes2)
    print(f"  {row['painter_1'][:25]:<28} ↔ {row['painter_2'][:25]:<28}")
    print(f"      Similarity: {row['similarity']:.3f} | Shared codes: {shared}")

print("\n🔀 Most Different Painter Pairs:")
print("="*70)
for row in sim_pairs_df.tail(5).iter_rows(named=True):
    codes1 = set(painter_code_vectors.get(row['painter_1'], {}).keys())
    codes2 = set(painter_code_vectors.get(row['painter_2'], {}).keys())
    shared = len(codes1 & codes2)
    print(f"  {row['painter_1'][:25]:<28} ↔ {row['painter_2'][:25]:<28}")
    print(f"      Similarity: {row['similarity']:.3f} | Shared codes: {shared}")

🔗 PAINTER SIMILARITY ANALYSIS (Full Hierarchical Distance)
Computing hierarchical similarity for top 20 painters...
Painters: ['Hermann, Franz Georg', 'Castelli, Carlo Ludovico', 'Asam, Cosmas Damian', 'Harms, Johann Oswald', 'Wolcker, Johann Georg', 'Ritter, Johann Heinrich', 'Kilian, Magnus', 'Höder, Johann Wilhelm', 'Asam, Hans Georg', 'Asam, Maria Theresia', 'Lammers, Seivert', 'Paduano, Alessandro', 'Kuen, Franz Martin', 'Wohlhaupter, Emanuel', 'Huber, Johann Joseph Anton', 'Spiegler, Franz Josef', 'Schönfeld, Johann Heinrich', 'Vaillant, Jacques', 'Rode, Bernhard', 'Speer, Martin']
✅ Similarity matrix computed!

DataFrame dtypes:
painter_1         str
painter_2         str
similarity    float64
dtype: object

Sample data:
              painter_1                 painter_2  similarity
0  Hermann, Franz Georg      Hermann, Franz Georg    1.000000
1  Hermann, Franz Georg  Castelli, Carlo Ludovico    0.141900
2  Hermann, Franz Georg       Asam, Cosmas Damian    0.200194
3  Hermann, Fr

alt.Chart(...)


🔗 Most Similar Painter Pairs (by full ICONCLASS hierarchy):
  Asam, Hans Georg             ↔ Asam, Maria Theresia        
      Similarity: 0.962 | Shared codes: 127
  Asam, Cosmas Damian          ↔ Asam, Maria Theresia        
      Similarity: 0.459 | Shared codes: 121
  Asam, Cosmas Damian          ↔ Asam, Hans Georg            
      Similarity: 0.452 | Shared codes: 121
  Hermann, Franz Georg         ↔ Asam, Cosmas Damian         
      Similarity: 0.200 | Shared codes: 70
  Asam, Cosmas Damian          ↔ Wolcker, Johann Georg       
      Similarity: 0.196 | Shared codes: 35

🔀 Most Different Painter Pairs:
  Höder, Johann Wilhelm        ↔ Paduano, Alessandro         
      Similarity: 0.024 | Shared codes: 1
  Ritter, Johann Heinrich      ↔ Speer, Martin               
      Similarity: 0.022 | Shared codes: 3
  Höder, Johann Wilhelm        ↔ Vaillant, Jacques           
      Similarity: 0.018 | Shared codes: 0
  Höder, Johann Wilhelm        ↔ Speer, Martin               
    

In [ ]:
# =============================================================================
# CELL 13h: Individual Painter Topic Profiles vs Average (with Full Hierarchy Details)
# =============================================================================
# Compare individual painters against the overall "average" + show specific ICONCLASS codes

# For top-level category comparison, use normalized matrix
category_columns = [col for col in painter_matrix.columns if col in ICONCLASS_CATEGORIES]
matrix_np = painter_matrix.select(category_columns).to_numpy().astype(float)
row_sums = matrix_np.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1
matrix_normalized = matrix_np / row_sums

avg_profile = matrix_normalized.mean(axis=0)
avg_profile_pct = avg_profile * 100

# Select featured painters (most prolific + most unique based on our analysis)
featured_painters = [
    "Hermann, Franz Georg",        # Most subjects
    "Asam, Cosmas Damian",         # Famous Baroque painter
    "Asam, Hans Georg",            # High similarity cluster
    "Pictorius, Johann Martin",    # Highly diverse
    "Meil, Johann Ludwig",         # Most focused
]

# Only include painters that exist in our data
featured_painters = [p for p in featured_painters if p in painters_list]

# Build comparison data for top-level categories
comparison_data = []

# Add average profile
for idx, cat in enumerate(category_columns):
    comparison_data.append({
        'painter': '📊 AVERAGE (all painters)',
        'category': cat,
        'category_name': ICONCLASS_CATEGORIES.get(cat, cat),
        'percentage': round(avg_profile_pct[idx], 2),
        'is_average': True
    })

# Add individual painter profiles
for painter in featured_painters:
    p_idx = painters_list.index(painter)
    profile_pct = matrix_normalized[p_idx] * 100
    
    for idx, cat in enumerate(category_columns):
        comparison_data.append({
            'painter': painter,
            'category': cat,
            'category_name': ICONCLASS_CATEGORIES.get(cat, cat),
            'percentage': round(profile_pct[idx], 2),
            'is_average': False
        })

comparison_df = pl.DataFrame(comparison_data).to_pandas()

# Create faceted bar chart
category_order = ['Abstract, Non-representational Art', 'Religion and Magic', 'Nature',
                  'Human Being, Man in General', 'Society, Civilization, Culture',
                  'Abstract Ideas and Concepts', 'History', 'Bible', 'Literature',
                  'Classical Mythology and Ancient History']
category_order = [c for c in category_order if c in comparison_df['category_name'].unique()]

profile_chart = alt.Chart(comparison_df).mark_bar().encode(
    x=alt.X('category_name:N', title='Topic Category', sort=category_order,
            axis=alt.Axis(labelAngle=-45, labelLimit=150)),
    y=alt.Y('percentage:Q', title='Percentage of Subjects', scale=alt.Scale(domain=[0, 70])),
    color=alt.Color('is_average:N', 
                   scale=alt.Scale(domain=[False, True], range=['steelblue', 'orange']),
                   legend=alt.Legend(title='Type', labelExpr="datum.value ? 'Average' : 'Individual'")),
    tooltip=[
        alt.Tooltip('painter:N', title='Painter'),
        alt.Tooltip('category_name:N', title='Category'),
        alt.Tooltip('percentage:Q', title='Percentage', format='.1f')
    ]
).properties(
    width=200,
    height=200
).facet(
    facet=alt.Facet('painter:N', title=None, 
                    sort=['📊 AVERAGE (all painters)'] + featured_painters,
                    header=alt.Header(labelFontSize=11, labelLimit=200)),
    columns=3
).properties(
    title='Painter Topic Profiles vs Overall Average (Top-Level Categories)'
)

ipython_display(profile_chart)

# =============================================================================
# NEW: Show FULL ICONCLASS code signatures for each painter
# =============================================================================
print("\n" + "=" * 80)
print("🎨 PAINTER ICONCLASS SIGNATURES (Full Hierarchical Codes)")
print("=" * 80)
print("Each painter's top 8 most frequently used ICONCLASS codes:\n")

for painter in featured_painters:
    codes = painter_code_vectors.get(painter, Counter())
    total_codes = sum(codes.values())
    unique_codes = len(codes)
    
    # Get profile info
    profile_row = painter_profiles.filter(pl.col('painter') == painter)
    diversity = profile_row['diversity_normalized'][0] if len(profile_row) > 0 else 0
    
    print(f"📌 {painter}")
    print(f"   Unique codes: {unique_codes} | Diversity: {diversity:.1f}%")
    print(f"   Top codes:")
    
    top_codes = codes.most_common(8)
    for code, count in top_codes:
        # Get label for this code
        label_rows = painter_subjects_filtered.filter(
            (pl.col('painter') == painter) & (pl.col('iconclass_code') == code)
        )['subject_label'].to_list()
        label = label_rows[0] if label_rows else "Unknown"
        pct = count / total_codes * 100
        print(f"      {code:<15} ({pct:5.1f}%) {label[:45]}")
    print()

# Compute hierarchical distance from "average painter"
print("=" * 80)
print("📏 HIERARCHICAL DISTANCE FROM AVERAGE")
print("=" * 80)

# Build average code vector (aggregate of all painters)
avg_code_vector = Counter()
for painter in active_painters:
    for code, count in painter_code_vectors.get(painter, {}).items():
        avg_code_vector[code] += count

# Normalize by number of painters
for code in avg_code_vector:
    avg_code_vector[code] = avg_code_vector[code] / len(active_painters)

print(f"\nDistance from average code distribution (0 = identical, lower = more typical):\n")
for painter in featured_painters:
    sim = hierarchical_similarity(
        painter_code_vectors.get(painter, {}),
        avg_code_vector
    )
    distance = 1 - sim
    
    # Get unique codes count
    unique = len(painter_code_vectors.get(painter, {}))
    
    bar = "█" * int(distance * 40)
    print(f"  {painter:<35} {distance:.3f} {bar}")

alt.FacetChart(...)


🎨 PAINTER ICONCLASS SIGNATURES (Full Hierarchical Codes)
Each painter's top 8 most frequently used ICONCLASS codes:

📌 Hermann, Franz Georg
   Unique codes: 298 | Diversity: 100.0%
   Top codes:
      26A             (  2.9%) clouds
      92D1916         (  2.0%) cupids: 'amores', 'amoretti', 'putti'
      49MM32          (  1.8%) book - MM - book open
      11G192          (  1.6%) angel(s) represented as youth(s) or adult(s)
      41D263          (  1.4%) walking-stick, staff, cane
      48C161          (  1.4%) column, pillar ~ architecture
      11G191          (  1.4%) angel(s) represented as child(ren)
      41B31           (  1.3%) candle

📌 Asam, Cosmas Damian
   Unique codes: 272 | Diversity: 92.0%
   Top codes:
      26A             (  1.6%) clouds
      92D1916         (  1.4%) cupids: 'amores', 'amoretti', 'putti'
      44B192          (  1.1%) sceptre, staff (symbol of sovereignty)
      44B191          (  1.1%) crown (symbol of sovereignty)
      11H%28John%20Nepomuk%294

In [ ]:
# =============================================================================
# CELL 14: Painter Collaboration Network
# =============================================================================

print("🤝 PAINTER COLLABORATIONS")
print("=" * 60)
print("(Painters who worked on the same painting)")

collabs_df = con.execute("""
    SELECT 
        pp1.person_name AS painter1,
        pp2.person_name AS painter2,
        COUNT(DISTINCT pp1.nfdi_uri) AS collaborations
    FROM painting_persons pp1
    JOIN painting_persons pp2 
        ON pp1.nfdi_uri = pp2.nfdi_uri 
        AND pp1.person_id < pp2.person_id
    WHERE pp1.role = 'PAINTER' AND pp2.role = 'PAINTER'
    GROUP BY pp1.person_name, pp2.person_name
    HAVING COUNT(DISTINCT pp1.nfdi_uri) >= 1
    ORDER BY collaborations DESC
    LIMIT 20
""").pl()

print(collabs_df)

# Visualization - Collaboration pairs
if len(collabs_df) > 0 and collabs_df['collaborations'].max() > 0:
    df_viz = collabs_df.head(15).to_pandas()
    df_viz['pair'] = df_viz['painter1'].str[:20] + ' + ' + df_viz['painter2'].str[:20]
    
    chart = alt.Chart(df_viz).mark_bar().encode(
        x=alt.X('collaborations:Q', title='Number of Collaborations'),
        y=alt.Y('pair:N', sort='-x', title='Painter Pair'),
        color=alt.Color('collaborations:Q', scale=alt.Scale(scheme='purples'), legend=None),
        tooltip=['painter1', 'painter2', 'collaborations']
    ).properties(
        title='Painter Collaborations',
        width=500,
        height=400
    )
    ipython_display(chart)

🤝 PAINTER COLLABORATIONS
(Painters who worked on the same painting)
shape: (20, 3)
┌────────────────────────────┬────────────────────────────┬────────────────┐
│ painter1                   ┆ painter2                   ┆ collaborations │
│ ---                        ┆ ---                        ┆ ---            │
│ str                        ┆ str                        ┆ i64            │
╞════════════════════════════╪════════════════════════════╪════════════════╡
│ Richter, Johann Wilhelm    ┆ Wentzing, Heinrich Richard ┆ 30             │
│ Asam, Hans Georg           ┆ Asam, Maria Theresia       ┆ 29             │
│ Wentzing, Heinrich Richard ┆ Lammers, Seivert           ┆ 28             │
│ Richter, Johann Wilhelm    ┆ Lammers, Seivert           ┆ 28             │
│ Asam, Cosmas Damian        ┆ Asam, Maria Theresia       ┆ 28             │
│ …                          ┆ …                          ┆ …              │
│ Zimmermann, Franz Michael  ┆ Zimmermann, Johann Baptist ┆ 7         

alt.Chart(...)

---
# Bildindex Cross-Reference Analysis

The following cells demonstrate how to join CbDD ceiling paintings with Bildindex historical photographs using shared GND identifiers.

In [ ]:
# =============================================================================
# CELL: Bildindex Overview Statistics
# =============================================================================

print("📗 BILDINDEX DATA OVERVIEW")
print("=" * 60)

try:
    # Check if Bildindex tables exist
    bi_tables = con.execute("""
        SELECT table_name FROM information_schema.tables 
        WHERE table_name LIKE 'bi_%'
    """).fetchall()
    
    if not bi_tables:
        print("⚠️ Bildindex tables not loaded. Run the database creation cells first.")
    else:
        # Basic counts
        bi_items_count = con.execute("SELECT COUNT(*) FROM bi_items").fetchone()[0]
        bi_buildings_count = con.execute("SELECT COUNT(*) FROM bi_buildings").fetchone()[0]
        bi_painters_count = con.execute("SELECT COUNT(*) FROM bi_painters").fetchone()[0]
        bi_subjects_count = con.execute("SELECT COUNT(*) FROM bi_subjects").fetchone()[0]
        
        print(f"   📷 Bildindex items:           {bi_items_count:>6,}")
        print(f"   🏛️  Building connections:     {bi_buildings_count:>6,}")
        print(f"   🎨 Painter connections:       {bi_painters_count:>6,}")
        print(f"   📜 Subject connections:       {bi_subjects_count:>6,}")
        
        # GND overlap summary
        print("\n📊 GND Overlap Summary:")
        gnd_summary = con.execute("""
            SELECT 
                gnd_type,
                COUNT(*) as shared_gnds,
                SUM(cbdd_count) as cbdd_items,
                SUM(bildindex_count) as bildindex_items
            FROM bi_gnd_overlaps
            GROUP BY gnd_type
        """).pl()
        print(gnd_summary)
        
        # Collection method breakdown
        print("\n📈 Collection Method:")
        method_df = con.execute("""
            SELECT 
                collection_method,
                tier,
                COUNT(*) as item_count
            FROM bi_items
            GROUP BY collection_method, tier
            ORDER BY tier, item_count DESC
        """).pl()
        print(method_df)
        
except Exception as e:
    print(f"⚠️ Error querying Bildindex tables: {e}")

📗 BILDINDEX DATA OVERVIEW
   📷 Bildindex items:              968
   🏛️  Building connections:        215
   🎨 Painter connections:          753
   📜 Subject connections:          964

📊 GND Overlap Summary:
shape: (2, 4)
┌──────────┬─────────────┬───────────────┬─────────────────┐
│ gnd_type ┆ shared_gnds ┆ cbdd_items    ┆ bildindex_items │
│ ---      ┆ ---         ┆ ---           ┆ ---             │
│ str      ┆ i64         ┆ decimal[38,0] ┆ decimal[38,0]   │
╞══════════╪═════════════╪═══════════════╪═════════════════╡
│ building ┆ 69          ┆ 784           ┆ 215             │
│ painter  ┆ 364         ┆ 2294          ┆ 753             │
└──────────┴─────────────┴───────────────┴─────────────────┘

📈 Collection Method:
shape: (2, 3)
┌───────────────────┬──────┬────────────┐
│ collection_method ┆ tier ┆ item_count │
│ ---               ┆ ---  ┆ ---        │
│ str               ┆ i64  ┆ i64        │
╞═══════════════════╪══════╪════════════╡
│ building_gnd      ┆ 1    ┆ 215        │
│ p

In [ ]:
# =============================================================================
# CELL: Cross-Reference: CbDD Paintings with Bildindex Images
# =============================================================================
# Find CbDD paintings that have related Bildindex images via shared painter GNDs

print("🔗 CROSS-REFERENCE: CbDD Paintings ↔ Bildindex Images")
print("=" * 60)

try:
    # Find paintings with Bildindex connections via painter GND
    cross_ref_df = con.execute("""
        WITH cbdd_painter_gnds AS (
            -- Extract individual GNDs from the piped string in paintings
            SELECT DISTINCT
                p.nfdi_uri,
                p.label as painting_label,
                p.painters,
                p.building_name,
                TRIM(gnd.value) as painter_gnd
            FROM paintings p,
                 UNNEST(STRING_SPLIT(p.creatorGnds, '|')) as gnd(value)
            WHERE p.creatorGnds IS NOT NULL AND p.creatorGnds != ''
        )
        SELECT 
            cpg.painting_label,
            cpg.painters as cbdd_painter,
            cpg.building_name,
            bi.title as bildindex_title,
            bi.bildindex_url,
            bi.tier as collection_tier
        FROM cbdd_painter_gnds cpg
        JOIN bi_painters bp ON cpg.painter_gnd = bp.painter_gnd
        JOIN bi_items bi ON bp.bildindex_uri = bi.bildindex_uri
        ORDER BY cpg.painters, cpg.painting_label
        LIMIT 50
    """).pl()
    
    if len(cross_ref_df) > 0:
        print(f"✅ Found {len(cross_ref_df)} cross-references (showing max 50)")
        print("\n" + "-" * 60)
        ipython_display(cross_ref_df)
    else:
        print("⚠️ No cross-references found. This may mean:")
        print("   - The creatorGnds column is empty in paintings table")
        print("   - No shared painter GNDs between CbDD and Bildindex")
        
except Exception as e:
    print(f"⚠️ Error in cross-reference query: {e}")

🔗 CROSS-REFERENCE: CbDD Paintings ↔ Bildindex Images
✅ Found 50 cross-references (showing max 50)

------------------------------------------------------------


painting_label,cbdd_painter,building_name,bildindex_title,bildindex_url,collection_tier
str,str,str,str,str,i64
"""Die drei Jünglinge im Feuerofe…","""Aloisi, Andrea""","""Bad Iburg, Schloss Iburg""","""Innendekoration des Rittersaal…","""http://www.bildindex.de/docume…",2
"""Die drei Jünglinge im Feuerofe…","""Aloisi, Andrea""","""Bad Iburg, Schloss Iburg""","""Bischofssitz und Benediktinera…","""http://www.bildindex.de/docume…",2
"""Die drei Jünglinge im Feuerofe…","""Aloisi, Andrea""","""Bad Iburg, Schloss Iburg""","""Innendekoration des Rittersaal…","""http://www.bildindex.de/docume…",2
"""Jupiter""","""Aloisi, Andrea""","""Bad Iburg, Schloss Iburg""","""Bischofssitz und Benediktinera…","""http://www.bildindex.de/docume…",2
"""Jupiter""","""Aloisi, Andrea""","""Bad Iburg, Schloss Iburg""","""Deckendekoration im Rittersaal…","""http://www.bildindex.de/docume…",2
…,…,…,…,…,…
"""Die vier Evangelisten in blaue…","""Appiani, Joseph Ignaz""","""Altshausen, Kirche St. Michael""","""Erzengel Michael""","""http://www.bildindex.de/docume…",2
"""Die vier Evangelisten in blaue…","""Appiani, Joseph Ignaz""","""Altshausen, Kirche St. Michael""","""Heilige in der Glorie""","""http://www.bildindex.de/docume…",2
"""Die vier Evangelisten in blaue…","""Appiani, Joseph Ignaz""","""Altshausen, Kirche St. Michael""","""Altar""","""http://www.bildindex.de/docume…",2


In [ ]:
# =============================================================================
# CELL: Cross-Reference: Buildings with Most Bildindex Coverage
# =============================================================================

print("🏛️ BUILDINGS WITH MOST BILDINDEX COVERAGE")
print("=" * 60)

try:
    building_coverage_df = con.execute("""
        WITH cbdd_building_gnds AS (
            SELECT DISTINCT
                p.building_id,
                p.building_name,
                TRIM(gnd.value) as location_gnd,
                COUNT(DISTINCT p.nfdi_uri) as cbdd_paintings
            FROM paintings p,
                 UNNEST(STRING_SPLIT(p.locationGnds, '|')) as gnd(value)
            WHERE p.locationGnds IS NOT NULL AND p.locationGnds != ''
            GROUP BY p.building_id, p.building_name, TRIM(gnd.value)
        )
        SELECT 
            cbg.building_name,
            cbg.cbdd_paintings,
            COUNT(DISTINCT bi.bildindex_uri) as bildindex_images,
            cbg.location_gnd
        FROM cbdd_building_gnds cbg
        JOIN bi_buildings bb ON cbg.location_gnd = bb.building_gnd
        JOIN bi_items bi ON bb.bildindex_uri = bi.bildindex_uri
        GROUP BY cbg.building_name, cbg.cbdd_paintings, cbg.location_gnd
        ORDER BY bildindex_images DESC
        LIMIT 20
    """).pl()
    
    if len(building_coverage_df) > 0:
        print(f"✅ Found {len(building_coverage_df)} buildings with Bildindex coverage:")
        ipython_display(building_coverage_df)
        
        # Visualization
        if len(building_coverage_df) > 0:
            df_viz = building_coverage_df.head(15).to_pandas()
            chart = alt.Chart(df_viz).mark_bar().encode(
                x=alt.X('bildindex_images:Q', title='Bildindex Images'),
                y=alt.Y('building_name:N', sort='-x', title='Building'),
                color=alt.Color('cbdd_paintings:Q', 
                               scale=alt.Scale(scheme='greens'),
                               title='CbDD Paintings'),
                tooltip=['building_name', 'cbdd_paintings', 'bildindex_images']
            ).properties(
                title='Buildings with Bildindex Coverage',
                width=500,
                height=400
            )
            ipython_display(chart)
    else:
        print("⚠️ No building cross-references found.")
        
except Exception as e:
    print(f"⚠️ Error in building coverage query: {e}")

🏛️ BUILDINGS WITH MOST BILDINDEX COVERAGE
✅ Found 20 buildings with Bildindex coverage:


building_name,cbdd_paintings,bildindex_images,location_gnd
str,i64,i64,str
"""Rastatt, Residenzschloss""",1,8,"""https://d-nb.info/gnd/4132604-…"
"""Potsdam, Schloss ""Sanssouci""""",29,8,"""https://d-nb.info/gnd/4132604-…"
"""Landshut, Stadtresidenz""",1,8,"""https://d-nb.info/gnd/4132604-…"
"""Burgk, Residenzschloss""",13,7,"""https://d-nb.info/gnd/4496763-…"
"""Regensburg, Weitholdstraße 6""",1,7,"""https://d-nb.info/gnd/4496763-…"
…,…,…,…
"""Berlin, Schloss Charlottenburg""",1,6,"""https://d-nb.info/gnd/11531528…"
"""Mannheim, Kurfürstliches Resid…",4,6,"""https://d-nb.info/gnd/4629677-…"
"""Landshut, Burg Trausnitz""",1,6,"""https://d-nb.info/gnd/11531528…"


alt.Chart(...)

In [ ]:
# =============================================================================
# CELL: Shared ICONCLASS Subjects Between CbDD and Bildindex
# =============================================================================

print("📜 SHARED ICONCLASS SUBJECTS")
print("=" * 60)

try:
    # Find ICONCLASS codes that appear in both datasets
    shared_subjects_df = con.execute("""
        SELECT 
            s.subject_uri,
            s.subject_label,
            COUNT(DISTINCT ps.nfdi_uri) as cbdd_paintings,
            COUNT(DISTINCT bis.bildindex_uri) as bildindex_items
        FROM subjects s
        JOIN painting_subjects ps ON s.subject_uri = ps.subject_uri
        JOIN bi_subjects bis ON s.subject_uri = bis.iconclass_code
        WHERE s.subject_source = 'ICONCLASS'
        GROUP BY s.subject_uri, s.subject_label
        HAVING cbdd_paintings > 0 AND bildindex_items > 0
        ORDER BY (cbdd_paintings + bildindex_items) DESC
        LIMIT 30
    """).pl()
    
    if len(shared_subjects_df) > 0:
        print(f"✅ Found {len(shared_subjects_df)} shared ICONCLASS subjects:")
        ipython_display(shared_subjects_df)
        
        # Scatter plot: CbDD vs Bildindex coverage per subject
        df_viz = shared_subjects_df.to_pandas()
        chart = alt.Chart(df_viz).mark_circle(size=100).encode(
            x=alt.X('cbdd_paintings:Q', title='CbDD Paintings', scale=alt.Scale(type='log')),
            y=alt.Y('bildindex_items:Q', title='Bildindex Items', scale=alt.Scale(type='log')),
            color=alt.value('#8856a7'),
            tooltip=['subject_label', 'cbdd_paintings', 'bildindex_items']
        ).properties(
            title='Shared ICONCLASS Subjects: CbDD vs Bildindex',
            width=500,
            height=400
        )
        ipython_display(chart)
    else:
        print("⚠️ No shared ICONCLASS subjects found.")
        
except Exception as e:
    print(f"⚠️ Error in shared subjects query: {e}")

📜 SHARED ICONCLASS SUBJECTS
✅ Found 30 shared ICONCLASS subjects:


subject_uri,subject_label,cbdd_paintings,bildindex_items
str,str,i64,i64
"""https://iconclass.org/43C11""","""hunting, chase""",38,3
"""https://iconclass.org/44B192""","""sceptre, staff (symbol of sove…",36,2
"""https://iconclass.org/92C2""","""(story of) Minerva (Pallas, At…",35,2
"""https://iconclass.org/23D41""","""winter, 'Hyems'; 'Inverno' (Ri…",35,1
"""https://iconclass.org/92D1""","""(story of) Cupid, Amor (Eros)""",32,3
…,…,…,…
"""https://iconclass.org/23U22""","""hourglass""",13,2
"""https://iconclass.org/25I153""","""obelisk, needle""",13,1
"""https://iconclass.org/35""","""pastorals, Arcadian scenes""",13,1


alt.Chart(...)

# Top 3 Painters with GND Overlap - Comprehensive Biographies

This section identifies the painters with the highest cross-reference counts between CbDD and Bildindex datasets, then builds comprehensive biographical profiles showing:
- **Personal information** (birth, death, GND identifier)
- **All paintings** (from CbDD)
- **All Bildindex images** (historical photographs)
- **Buildings worked in** (from both datasets)
- **Collaborators** (persons they worked with)
- **Timeline visualization**

Data source indicators:
- 🟦 **CbDD** (Ceiling Paintings Database Germany)
- 🟧 **Bildindex** (Historical Image Archive)

In [ ]:
# =============================================================================
# CELL: Identify Top 3 Painters with Highest GND Overlap
# =============================================================================
# Find painters that appear in BOTH CbDD and Bildindex datasets via GND matching

print("🎨 TOP 3 PAINTERS WITH HIGHEST CBDD ↔ BILDINDEX GND OVERLAP")
print("=" * 70)

# First, let's examine the relevant table schemas
print("\n📋 Examining table schemas...")
print("\n   painting_persons columns:")
pp_schema = con.execute("DESCRIBE painting_persons").pl()
for row in pp_schema.iter_rows(named=True):
    print(f"      - {row['column_name']}: {row['column_type']}")

print("\n   persons columns:")
p_schema = con.execute("DESCRIBE persons").pl()
for row in p_schema.iter_rows(named=True):
    print(f"      - {row['column_name']}: {row['column_type']}")

print("\n   bi_painters columns:")
bp_schema = con.execute("DESCRIBE bi_painters").pl()
for row in bp_schema.iter_rows(named=True):
    print(f"      - {row['column_name']}: {row['column_type']}")

print("\n" + "=" * 70)

try:
    # The paintings table has creatorGnds column with pipe-separated GND URIs
    # We need to match these with bi_painters.painter_gnd
    # But first, let's verify there's data in creatorGnds
    
    gnd_check = con.execute("""
        SELECT 
            COUNT(*) as total,
            COUNT(CASE WHEN creatorGnds IS NOT NULL AND creatorGnds != '' THEN 1 END) as with_gnd
        FROM paintings
    """).fetchone()
    
    print(f"\n📊 Paintings GND coverage:")
    print(f"   Total paintings: {gnd_check[0]}")
    print(f"   With creatorGnds: {gnd_check[1]}")
    
    if gnd_check[1] == 0:
        print("\n⚠️ No creatorGnds found in paintings table!")
        print("   Falling back to matching via person names...")
        
        # Alternative approach: match by person name
        # First get all Bildindex painter GNDs 
        # Then find matching persons in CbDD by looking at the persons table
        
        top_painters_overlap = con.execute("""
            WITH bildindex_painter_gnds AS (
                -- Get unique painter GNDs from Bildindex
                SELECT DISTINCT painter_gnd
                FROM bi_painters
            ),
            cbdd_persons_with_gnd AS (
                -- Find CbDD persons that might match (we need to extract GND from their NFDI URI or name)
                SELECT 
                    ps.person_id,
                    ps.person_name,
                    pp.nfdi_uri,
                    pp.role
                FROM persons ps
                JOIN painting_persons pp ON ps.person_id = pp.person_id
                WHERE pp.role = 'PAINTER'
            ),
            -- Count how many paintings and bildindex items per painter
            painter_counts AS (
                SELECT 
                    pp.person_id,
                    pp.person_name,
                    COUNT(DISTINCT pp.nfdi_uri) as cbdd_painting_count
                FROM painting_persons pp
                WHERE pp.role = 'PAINTER'
                GROUP BY pp.person_id, pp.person_name
                ORDER BY cbdd_painting_count DESC
                LIMIT 10
            )
            SELECT * FROM painter_counts
        """).pl()
        
        print("\n📋 Top painters by CbDD painting count (no GND matching available):")
        print(top_painters_overlap)
        
        TOP_3_PAINTERS = []
        for row in top_painters_overlap.head(3).iter_rows(named=True):
            TOP_3_PAINTERS.append({
                'person_id': row['person_id'],
                'person_name': row['person_name'],
                'cbdd_painting_count': row['cbdd_painting_count'],
                'bildindex_item_count': 0,  # No GND matching available
                'painter_gnd': None
            })
        
    else:
        # GND-based matching
        top_painters_overlap = con.execute("""
            WITH cbdd_painter_gnds AS (
                -- Get all painter GNDs from CbDD paintings (from creatorGnds column)
                SELECT DISTINCT
                    TRIM(gnd.value) as painter_gnd,
                    p.nfdi_uri,
                    p.label as painting_label,
                    pp.person_name,
                    pp.person_id
                FROM paintings p,
                     UNNEST(STRING_SPLIT(p.creatorGnds, '|')) as gnd(value)
                JOIN painting_persons pp ON p.nfdi_uri = pp.nfdi_uri AND pp.role = 'PAINTER'
                WHERE p.creatorGnds IS NOT NULL AND p.creatorGnds != ''
            ),
            gnd_overlap_counts AS (
                -- Count how many Bildindex items each painter GND has
                SELECT 
                    cpg.painter_gnd,
                    cpg.person_name,
                    cpg.person_id,
                    COUNT(DISTINCT cpg.nfdi_uri) as cbdd_painting_count,
                    COUNT(DISTINCT bp.bildindex_uri) as bildindex_item_count
                FROM cbdd_painter_gnds cpg
                JOIN bi_painters bp ON cpg.painter_gnd = bp.painter_gnd
                GROUP BY cpg.painter_gnd, cpg.person_name, cpg.person_id
            )
            SELECT 
                painter_gnd,
                person_name,
                person_id,
                cbdd_painting_count,
                bildindex_item_count,
                (cbdd_painting_count + bildindex_item_count) as total_items
            FROM gnd_overlap_counts
            ORDER BY total_items DESC
            LIMIT 3
        """).pl()
        
        if len(top_painters_overlap) > 0:
            print(f"\n✅ Found {len(top_painters_overlap)} painters with cross-dataset presence:\n")
            for i, row in enumerate(top_painters_overlap.iter_rows(named=True), 1):
                print(f"  {i}. {row['person_name']}")
                print(f"     GND: {row['painter_gnd']}")
                print(f"     🟦 CbDD Paintings: {row['cbdd_painting_count']}")
                print(f"     🟧 Bildindex Items: {row['bildindex_item_count']}")
                print(f"     Total cross-references: {row['total_items']}")
                print()
            
            # Store for use in subsequent cells
            TOP_3_PAINTERS = top_painters_overlap.to_dicts()
            print("📌 Stored painter data in TOP_3_PAINTERS for biography generation")
            
            # Display as table
            ipython_display(top_painters_overlap)
        else:
            print("⚠️ No painters with GND overlap found between datasets.")
            TOP_3_PAINTERS = []
        
except Exception as e:
    print(f"⚠️ Error finding top painters: {e}")
    import traceback
    traceback.print_exc()
    TOP_3_PAINTERS = []

🎨 TOP 3 PAINTERS WITH HIGHEST CBDD ↔ BILDINDEX GND OVERLAP

📋 Examining table schemas...

   painting_persons columns:
      - nfdi_uri: VARCHAR
      - cbdd_painting_id: VARCHAR
      - person_id: VARCHAR
      - person_name: VARCHAR
      - role: VARCHAR

   persons columns:
      - person_id: VARCHAR
      - name: VARCHAR
      - person_type: VARCHAR
      - val: DOUBLE

   bi_painters columns:
      - bildindex_uri: VARCHAR
      - painter_gnd: VARCHAR


📊 Paintings GND coverage:
   Total paintings: 4594
   With creatorGnds: 2703

✅ Found 3 painters with cross-dataset presence:

  1. Hermann, Franz Georg
     GND: https://d-nb.info/gnd/123546850
     🟦 CbDD Paintings: 51
     🟧 Bildindex Items: 2
     Total cross-references: 53

  2. Harms, Johann Oswald
     GND: https://d-nb.info/gnd/119052520
     🟦 CbDD Paintings: 37
     🟧 Bildindex Items: 1
     Total cross-references: 38

  3. Göding, Heinrich
     GND: https://d-nb.info/gnd/119458446
     🟦 CbDD Paintings: 35
     🟧 Bildind

painter_gnd,person_name,person_id,cbdd_painting_count,bildindex_item_count,total_items
str,str,str,i64,i64,i64
"""https://d-nb.info/gnd/12354685…","""Hermann, Franz Georg""","""20f9622e-6580-44a3-b8fd-1eb3d6…",51,2,53
"""https://d-nb.info/gnd/11905252…","""Harms, Johann Oswald""","""5b74e880-6e0d-11ea-a2e8-4df471…",37,1,38
"""https://d-nb.info/gnd/11945844…","""Göding, Heinrich""","""e7590e67-f15d-4b5d-abcd-770500…",35,1,36


In [ ]:
# =============================================================================
# CELL: Comprehensive Painter Biographies with Timeline Visualization
# =============================================================================
# For each top painter, gather ALL available data from both datasets and visualize

import pandas as pd
from datetime import datetime

print("📜 COMPREHENSIVE PAINTER BIOGRAPHIES")
print("=" * 70)

def gather_painter_biography(painter_gnd, painter_name, person_id):
    """
    Gather all available data for a painter from both CbDD and Bildindex.
    Returns a dictionary with all biographical information.
    """
    bio = {
        'name': painter_name,
        'gnd': painter_gnd,
        'person_id': person_id,
        'cbdd': {},
        'bildindex': {},
        'timeline_events': []
    }
    
    # ===========================================
    # 1. CbDD DATA - Personal Information from persons table
    # ===========================================
    try:
        person_info = con.execute("""
            SELECT 
                person_id,
                name,
                person_type,
                val
            FROM persons
            WHERE person_id = ?
        """, [person_id]).pl()
        
        if len(person_info) > 0:
            info = person_info.row(0, named=True)
            bio['cbdd']['person_type'] = info.get('person_type')
            bio['cbdd']['full_name'] = info.get('name')
    except Exception as e:
        print(f"  ⚠️ Error fetching person info: {e}")
    
    # ===========================================
    # 2. CbDD DATA - All Paintings (using actual column names)
    # ===========================================
    try:
        paintings = con.execute("""
            SELECT DISTINCT
                p.nfdi_uri,
                p.label as painting_label,
                p.year,
                p.year_start,
                p.year_end,
                p.year_is_approximate,
                p.building_name,
                p.building_id,
                p.room_name,
                p.location_state,
                p.painters,
                p.imageUrl as image_url
            FROM paintings p
            JOIN painting_persons pp ON p.nfdi_uri = pp.nfdi_uri
            WHERE pp.person_id = ? AND pp.role = 'PAINTER'
            ORDER BY p.year_start
        """, [person_id]).pl()
        
        bio['cbdd']['paintings'] = paintings.to_dicts()
        bio['cbdd']['painting_count'] = len(paintings)
        
        # Calculate active period from painting years
        years = [p.get('year_start') for p in bio['cbdd']['paintings'] if p.get('year_start')]
        if years:
            bio['cbdd']['active_from'] = min(years)
            end_years = [p.get('year_end') or p.get('year_start') for p in bio['cbdd']['paintings'] if p.get('year_start')]
            bio['cbdd']['active_until'] = max(end_years) if end_years else max(years)
        
        # Add painting events to timeline
        for row in paintings.iter_rows(named=True):
            if row.get('year_start'):
                approx = "~" if row.get('year_is_approximate') else ""
                bio['timeline_events'].append({
                    'year': row['year_start'],
                    'event': f"{approx}Painted: {row['painting_label'][:50]}{'...' if len(row['painting_label']) > 50 else ''}",
                    'type': 'painting',
                    'source': 'CbDD',
                    'location': row.get('building_name', ''),
                    'full_label': row['painting_label']
                })
    except Exception as e:
        print(f"  ⚠️ Error fetching paintings: {e}")
        bio['cbdd']['paintings'] = []
        bio['cbdd']['painting_count'] = 0
    
    # ===========================================
    # 3. CbDD DATA - All Buildings (using actual columns)
    # ===========================================
    try:
        buildings = con.execute("""
            SELECT DISTINCT
                b.building_id,
                b.name as building_name,
                b.function as building_function,
                b.location_state
            FROM buildings b
            JOIN paintings p ON b.building_id = p.building_id
            JOIN painting_persons pp ON p.nfdi_uri = pp.nfdi_uri
            WHERE pp.person_id = ? AND pp.role = 'PAINTER'
        """, [person_id]).pl()
        
        bio['cbdd']['buildings'] = buildings.to_dicts()
        bio['cbdd']['building_count'] = len(buildings)
    except Exception as e:
        print(f"  ⚠️ Error fetching buildings: {e}")
        bio['cbdd']['buildings'] = []
        bio['cbdd']['building_count'] = 0
    
    # ===========================================
    # 4. CbDD DATA - Collaborators (other persons worked with)
    # ===========================================
    try:
        collaborators = con.execute("""
            WITH painter_works AS (
                SELECT DISTINCT p.nfdi_uri
                FROM paintings p
                JOIN painting_persons pp ON p.nfdi_uri = pp.nfdi_uri
                WHERE pp.person_id = ? AND pp.role = 'PAINTER'
            )
            SELECT DISTINCT
                pp2.person_name,
                pp2.person_id,
                pp2.role,
                COUNT(DISTINCT pp2.nfdi_uri) as shared_works
            FROM painter_works pw
            JOIN painting_persons pp2 ON pw.nfdi_uri = pp2.nfdi_uri
            WHERE pp2.person_id != ?
            GROUP BY pp2.person_name, pp2.person_id, pp2.role
            ORDER BY shared_works DESC, pp2.role
        """, [person_id, person_id]).pl()
        
        bio['cbdd']['collaborators'] = collaborators.to_dicts()
        bio['cbdd']['collaborator_count'] = len(collaborators)
    except Exception as e:
        print(f"  ⚠️ Error fetching collaborators: {e}")
        bio['cbdd']['collaborators'] = []
        bio['cbdd']['collaborator_count'] = 0
    
    # ===========================================
    # 5. CbDD DATA - ICONCLASS Subjects
    # ===========================================
    try:
        subjects = con.execute("""
            SELECT DISTINCT
                s.subject_uri,
                s.subject_label,
                s.subject_source,
                COUNT(DISTINCT ps.nfdi_uri) as usage_count
            FROM subjects s
            JOIN painting_subjects ps ON s.subject_uri = ps.subject_uri
            JOIN painting_persons pp ON ps.nfdi_uri = pp.nfdi_uri
            WHERE pp.person_id = ? AND pp.role = 'PAINTER'
              AND s.subject_source = 'ICONCLASS'
            GROUP BY s.subject_uri, s.subject_label, s.subject_source
            ORDER BY usage_count DESC
            LIMIT 20
        """, [person_id]).pl()
        
        bio['cbdd']['subjects'] = subjects.to_dicts()
    except Exception as e:
        print(f"  ⚠️ Error fetching subjects: {e}")
        bio['cbdd']['subjects'] = []
    
    # ===========================================
    # 6. BILDINDEX DATA - All Images
    # ===========================================
    try:
        if painter_gnd:
            bildindex_items = con.execute("""
                SELECT DISTINCT
                    bi.bildindex_uri,
                    bi.bildindex_url,
                    bi.title,
                    bi.tier,
                    bi.collection_method
                FROM bi_items bi
                JOIN bi_painters bp ON bi.bildindex_uri = bp.bildindex_uri
                WHERE bp.painter_gnd = ?
                ORDER BY bi.title
            """, [painter_gnd]).pl()
            
            bio['bildindex']['items'] = bildindex_items.to_dicts()
            bio['bildindex']['item_count'] = len(bildindex_items)
            
            # Add Bildindex items to timeline (without year)
            for row in bildindex_items.iter_rows(named=True):
                bio['timeline_events'].append({
                    'year': None,
                    'event': f"Bildindex: {row['title'][:50] if row.get('title') else 'Untitled'}",
                    'type': 'bildindex_photo',
                    'source': 'Bildindex',
                    'url': row.get('bildindex_url')
                })
        else:
            bio['bildindex']['items'] = []
            bio['bildindex']['item_count'] = 0
    except Exception as e:
        print(f"  ⚠️ Error fetching Bildindex items: {e}")
        bio['bildindex']['items'] = []
        bio['bildindex']['item_count'] = 0
    
    # ===========================================
    # 7. BILDINDEX DATA - Buildings mentioned
    # ===========================================
    try:
        if painter_gnd:
            bi_buildings = con.execute("""
                SELECT DISTINCT
                    bb.building_gnd,
                    COUNT(DISTINCT bb.bildindex_uri) as image_count
                FROM bi_buildings bb
                JOIN bi_painters bp ON bb.bildindex_uri = bp.bildindex_uri
                WHERE bp.painter_gnd = ?
                GROUP BY bb.building_gnd
            """, [painter_gnd]).pl()
            
            bio['bildindex']['buildings'] = bi_buildings.to_dicts()
            bio['bildindex']['building_count'] = len(bi_buildings)
        else:
            bio['bildindex']['buildings'] = []
            bio['bildindex']['building_count'] = 0
    except Exception as e:
        print(f"  ⚠️ Error fetching Bildindex buildings: {e}")
        bio['bildindex']['buildings'] = []
        bio['bildindex']['building_count'] = 0
    
    # ===========================================
    # 8. BILDINDEX DATA - Subjects
    # ===========================================
    try:
        if painter_gnd:
            bi_subjects = con.execute("""
                SELECT DISTINCT
                    bis.iconclass_code,
                    COUNT(DISTINCT bis.bildindex_uri) as usage_count
                FROM bi_subjects bis
                JOIN bi_painters bp ON bis.bildindex_uri = bp.bildindex_uri
                WHERE bp.painter_gnd = ?
                GROUP BY bis.iconclass_code
                ORDER BY usage_count DESC
                LIMIT 20
            """, [painter_gnd]).pl()
            
            bio['bildindex']['subjects'] = bi_subjects.to_dicts()
        else:
            bio['bildindex']['subjects'] = []
    except Exception as e:
        print(f"  ⚠️ Error fetching Bildindex subjects: {e}")
        bio['bildindex']['subjects'] = []
    
    return bio

# Gather biographies for all top painters
PAINTER_BIOGRAPHIES = []

if 'TOP_3_PAINTERS' in dir() and TOP_3_PAINTERS:
    for i, painter in enumerate(TOP_3_PAINTERS, 1):
        print(f"\n{'='*70}")
        print(f"📖 Gathering biography for Painter {i}: {painter['person_name']}")
        print(f"{'='*70}")
        
        bio = gather_painter_biography(
            painter.get('painter_gnd'),
            painter['person_name'],
            painter['person_id']
        )
        PAINTER_BIOGRAPHIES.append(bio)
        
        # Print summary
        print(f"\n  📊 Data Summary for {bio['name']}:")
        print(f"     GND: {bio['gnd']}")
        print(f"     🟦 CbDD Data:")
        print(f"        - Active period: {bio['cbdd'].get('active_from', '?')} - {bio['cbdd'].get('active_until', '?')}")
        print(f"        - Paintings: {bio['cbdd'].get('painting_count', 0)}")
        print(f"        - Buildings: {bio['cbdd'].get('building_count', 0)}")
        print(f"        - Collaborators: {bio['cbdd'].get('collaborator_count', 0)}")
        print(f"        - ICONCLASS subjects: {len(bio['cbdd'].get('subjects', []))}")
        print(f"     🟧 Bildindex Data:")
        print(f"        - Historical images: {bio['bildindex'].get('item_count', 0)}")
        print(f"        - Buildings referenced: {bio['bildindex'].get('building_count', 0)}")
        print(f"        - ICONCLASS subjects: {len(bio['bildindex'].get('subjects', []))}")
        print(f"     📅 Timeline events: {len(bio['timeline_events'])}")
else:
    print("⚠️ TOP_3_PAINTERS not available. Run the previous cell first.")

print(f"\n✅ Collected biographies for {len(PAINTER_BIOGRAPHIES)} painters")

📜 COMPREHENSIVE PAINTER BIOGRAPHIES

📖 Gathering biography for Painter 1: Hermann, Franz Georg

  📊 Data Summary for Hermann, Franz Georg:
     GND: https://d-nb.info/gnd/123546850
     🟦 CbDD Data:
        - Active period: 1729 - 1742
        - Paintings: 54
        - Buildings: 2
        - Collaborators: 3
        - ICONCLASS subjects: 20
     🟧 Bildindex Data:
        - Historical images: 2
        - Buildings referenced: 0
        - ICONCLASS subjects: 2
     📅 Timeline events: 56

📖 Gathering biography for Painter 2: Harms, Johann Oswald

  📊 Data Summary for Harms, Johann Oswald:
     GND: https://d-nb.info/gnd/119052520
     🟦 CbDD Data:
        - Active period: 1569 - 1780
        - Paintings: 119
        - Buildings: 7
        - Collaborators: 8
        - ICONCLASS subjects: 20
     🟧 Bildindex Data:
        - Historical images: 1
        - Buildings referenced: 0
        - ICONCLASS subjects: 1
     📅 Timeline events: 120

📖 Gathering biography for Painter 3: Göding, Heinrich

In [ ]:
# =============================================================================
# CELL: Visualize Painter Biographies - Timeline & Summary Cards
# =============================================================================
# Create comprehensive visual biographies for each painter

from IPython.display import HTML, display

def create_biography_html(bio, index):
    """Create an HTML card showing painter biography with data source indicators."""
    
    name = bio['name']
    gnd = bio['gnd'] or 'Unknown'
    person_id = bio.get('person_id', 'Unknown')
    
    # Active period (derived from paintings)
    active_from = bio['cbdd'].get('active_from', '?')
    active_until = bio['cbdd'].get('active_until', '?')
    active_period = f"{active_from} - {active_until}" if active_from != '?' else "Unknown"
    
    # Build HTML
    html = f"""
    <div style="border: 2px solid #333; border-radius: 10px; padding: 20px; margin: 20px 0; 
                background: linear-gradient(135deg, #f5f7fa 0%, #e4e8eb 100%); color: black;">
        <h2 style="margin-top: 0; color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 10px;">
            🎨 Painter {index}: {name}
        </h2>
        
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
            <!-- Left Column: Personal Info -->
            <div>
                <h3 style="color: #2980b9;">📋 Personal Information</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr><td style="padding: 5px; font-weight: bold;">Active Period:</td><td>{active_period}</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">GND:</td>
                        <td><a href="{gnd}" target="_blank">{gnd.split('/')[-1] if gnd != 'Unknown' else 'N/A'}</a></td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Person ID:</td>
                        <td style="font-size: 0.8em; word-break: break-all;">{person_id[:20]}...</td></tr>
                </table>
                
                <h3 style="color: #2980b9; margin-top: 15px;">📊 Data Summary</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr style="background: #3498db22;">
                        <td style="padding: 8px; font-weight: bold;">Source</td>
                        <td style="padding: 8px; font-weight: bold;">Category</td>
                        <td style="padding: 8px; font-weight: bold; text-align: right;">Count</td>
                    </tr>
                    <tr><td style="padding: 5px;">🟦 CbDD</td><td>Paintings</td>
                        <td style="text-align: right; font-weight: bold;">{bio['cbdd'].get('painting_count', 0)}</td></tr>
                    <tr><td style="padding: 5px;">🟦 CbDD</td><td>Buildings</td>
                        <td style="text-align: right;">{bio['cbdd'].get('building_count', 0)}</td></tr>
                    <tr><td style="padding: 5px;">🟦 CbDD</td><td>Collaborators</td>
                        <td style="text-align: right;">{bio['cbdd'].get('collaborator_count', 0)}</td></tr>
                    <tr style="background: #e67e2222;">
                        <td style="padding: 5px;">🟧 Bildindex</td><td>Historical Images</td>
                        <td style="text-align: right; font-weight: bold;">{bio['bildindex'].get('item_count', 0)}</td></tr>
                    <tr style="background: #e67e2222;">
                        <td style="padding: 5px;">🟧 Bildindex</td><td>Buildings</td>
                        <td style="text-align: right;">{bio['bildindex'].get('building_count', 0)}</td></tr>
                </table>
            </div>
            
            <!-- Right Column: Top Subjects -->
            <div>
                <h3 style="color: #27ae60;">📚 Top ICONCLASS Subjects (CbDD)</h3>
                <ul style="margin: 0; padding-left: 20px; max-height: 150px; overflow-y: auto;">
    """
    
    for subj in bio['cbdd'].get('subjects', [])[:8]:
        html += f"<li><small>{subj['subject_label']} ({subj['usage_count']}x)</small></li>"
    
    if not bio['cbdd'].get('subjects'):
        html += "<li><em>No ICONCLASS subjects found</em></li>"
    
    html += """
                </ul>
                
                <h3 style="color: #e67e22; margin-top: 15px;">📚 Bildindex ICONCLASS</h3>
                <ul style="margin: 0; padding-left: 20px; max-height: 100px; overflow-y: auto;">
    """
    
    for subj in bio['bildindex'].get('subjects', [])[:5]:
        html += f"<li><small>{subj['iconclass_code']} ({subj['usage_count']}x)</small></li>"
    
    if not bio['bildindex'].get('subjects'):
        html += "<li><em>No ICONCLASS subjects found</em></li>"
    
    html += """
                </ul>
            </div>
        </div>
    </div>
    """
    
    return html

def create_paintings_table(bio):
    """Create HTML table of all paintings."""
    paintings = bio['cbdd'].get('paintings', [])
    if not paintings:
        return "<p><em>No paintings found in CbDD</em></p>"
    
    html = """
    <div style="margin: 10px 0;">
        <h4 style="color: #3498db;">🟦 CbDD Paintings ({} total)</h4>
        <div style="max-height: 300px; overflow-y: auto; border: 1px solid #ddd; border-radius: 5px;">
            <table style="width: 100%; border-collapse: collapse; font-size: 0.85em;">
                <tr style="background: #3498db; color: white; position: sticky; top: 0;">
                    <th style="padding: 8px; text-align: left;">Year</th>
                    <th style="padding: 8px; text-align: left;">Painting</th>
                    <th style="padding: 8px; text-align: left;">Building</th>
                    <th style="padding: 8px; text-align: left;">Location</th>
                </tr>
    """.format(len(paintings))
    
    for i, p in enumerate(paintings):
        bg = "#f8f9fa" if i % 2 == 0 else "#ffffff"
        year = p.get('year', '?')
        approx = "~" if p.get('year_is_approximate') else ""
        html += f"""
            <tr style="background: {bg}; color: black;">
                <td style="padding: 6px;">{approx}{year}</td>
                <td style="padding: 6px;">{p.get('painting_label', 'Untitled')[:60]}{'...' if len(p.get('painting_label', '')) > 60 else ''}</td>
                <td style="padding: 6px;">{p.get('building_name', '')[:30]}</td>
                <td style="padding: 6px;">{p.get('location_city', '')}, {p.get('location_state', '')}</td>
            </tr>
        """
    
    html += "</table></div></div>"
    return html

def create_bildindex_table(bio):
    """Create HTML table of all Bildindex items."""
    items = bio['bildindex'].get('items', [])
    if not items:
        return "<p><em>No items found in Bildindex</em></p>"
    
    html = """
    <div style="margin: 10px 0;">
        <h4 style="color: #e67e22;">🟧 Bildindex Historical Images ({} total)</h4>
        <div style="max-height: 250px; overflow-y: auto; border: 1px solid #ddd; border-radius: 5px;">
            <table style="width: 100%; border-collapse: collapse; font-size: 0.85em;">
                <tr style="background: #e67e22; color: white; position: sticky; top: 0;">
                    <th style="padding: 8px; text-align: left;">Title</th>
                    <th style="padding: 8px; text-align: left;">Tier</th>
                    <th style="padding: 8px; text-align: left;">Link</th>
                </tr>
    """.format(len(items))
    
    for i, item in enumerate(items):
        bg = "#fef5e7" if i % 2 == 0 else "#ffffff"
        title = item.get('title', 'Untitled') or 'Untitled'
        url = item.get('bildindex_url', '')
        html += f"""
            <tr style="background: {bg}; color: black;">
                <td style="padding: 6px;">{title[:70]}{'...' if len(title) > 70 else ''}</td>
                <td style="padding: 6px;">{item.get('tier', '?')}</td>
                <td style="padding: 6px;"><a href="{url}" target="_blank">🔗 View</a></td>
            </tr>
        """
    
    html += "</table></div></div>"
    return html

def create_collaborators_table(bio):
    """Create HTML table of collaborators."""
    collabs = bio['cbdd'].get('collaborators', [])
    if not collabs:
        return "<p><em>No collaborators found</em></p>"
    
    html = """
    <div style="margin: 10px 0;">
        <h4 style="color: #9b59b6;">👥 Collaborators ({} persons)</h4>
        <div style="max-height: 200px; overflow-y: auto; border: 1px solid #ddd; border-radius: 5px;">
            <table style="width: 100%; border-collapse: collapse; font-size: 0.85em;">
                <tr style="background: #9b59b6; color: white;">
                    <th style="padding: 8px; text-align: left;">Person</th>
                    <th style="padding: 8px; text-align: left;">Role</th>
                    <th style="padding: 8px; text-align: right;">Shared Works</th>
                </tr>
    """.format(len(collabs))
    
    for i, c in enumerate(collabs[:15]):  # Show top 15
        bg = "#f5f0f7" if i % 2 == 0 else "#ffffff"
        html += f"""
            <tr style="background: {bg}; color: black;">
                <td style="padding: 6px;">{c.get('person_name', 'Unknown')}</td>
                <td style="padding: 6px;">{c.get('role', '?')}</td>
                <td style="padding: 6px; text-align: right;">{c.get('shared_works', 0)}</td>
            </tr>
        """
    
    if len(collabs) > 15:
        html += f"<tr><td colspan='3' style='padding: 6px; text-align: center;'><em>... and {len(collabs) - 15} more</em></td></tr>"
    
    html += "</table></div></div>"
    return html

def create_buildings_table(bio):
    """Create HTML table of buildings from both sources."""
    cbdd_buildings = bio['cbdd'].get('buildings', [])
    bi_buildings = bio['bildindex'].get('buildings', [])
    
    html = """
    <div style="margin: 10px 0;">
        <h4 style="color: #16a085;">🏛️ Buildings</h4>
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px;">
    """
    
    # CbDD Buildings
    html += """
        <div style="border: 1px solid #3498db; border-radius: 5px; padding: 10px; background: #3498db; color: white;">
            <h5 style="color: white; margin-top: 0;">🟦 CbDD Buildings ({})</h5>
            <ul style="margin: 0; padding-left: 20px; max-height: 150px; overflow-y: auto; font-size: 0.85em; color: white;">
    """.format(len(cbdd_buildings))
    
    for b in cbdd_buildings:
        html += f"<li>{b.get('building_name', 'Unknown')} ({b.get('location_city', '')}) - <em>{b.get('building_function', '')}</em></li>"
    
    if not cbdd_buildings:
        html += "<li><em>No buildings found</em></li>"
    
    html += "</ul></div>"
    
    # Bildindex Buildings (just GNDs)
    html += """
        <div style="border: 1px solid #e67e22; border-radius: 5px; padding: 10px; background: #e67e22; color: white;">
            <h5 style="color: white; margin-top: 0;">🟧 Bildindex Building GNDs ({})</h5>
            <ul style="margin: 0; padding-left: 20px; max-height: 150px; overflow-y: auto; font-size: 0.85em; color: white;">
    """.format(len(bi_buildings))
    
    for b in bi_buildings:
        gnd = b.get('building_gnd', '')
        html += f"<li>{gnd} ({b.get('image_count', 0)} images)</li>"
    
    if not bi_buildings:
        html += "<li><em>No building references found</em></li>"
    
    html += "</ul></div></div></div>"
    return html

# ===========================================
# Display Biographies
# ===========================================
print("📜 DISPLAYING COMPREHENSIVE BIOGRAPHIES")
print("=" * 70)

if 'PAINTER_BIOGRAPHIES' in dir() and PAINTER_BIOGRAPHIES:
    for i, bio in enumerate(PAINTER_BIOGRAPHIES, 1):
        # Summary Card
        display(HTML(create_biography_html(bio, i)))
        
        # Detailed Tables
        detail_html = f"""
        <div style="border: 1px solid #ccc; border-radius: 5px; padding: 15px; margin: 10px 0 30px 0; color: black;">
            <h3 style="margin-top: 0; color: black;">📋 Detailed Data for {bio['name']}</h3>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px;">
                <div>{create_paintings_table(bio)}</div>
                <div>{create_bildindex_table(bio)}</div>
            </div>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin-top: 15px;">
                <div>{create_collaborators_table(bio)}</div>
                <div>{create_buildings_table(bio)}</div>
            </div>
        </div>
        """
        display(HTML(detail_html))
else:
    print("⚠️ No biographies available. Run the previous cells first.")

📜 DISPLAYING COMPREHENSIVE BIOGRAPHIES


Year,Painting,Building,Location
1729,Minerva überreicht Ganymed den vom Haus Waldburg-Wolfegg beh...,"Wolfegg, Schloss",", Baden-Württemberg"
1729,Deckengemälde: Verbannung der Laster durch die Kardinaltugen...,"Wolfegg, Schloss",", Baden-Württemberg"
1729,Möglicher Ort des Tondos Christus in Emmaus (Hermann 1729),"Wolfegg, Schloss",", Baden-Württemberg"
~um 1732–1742,Die Verherrlichung des Fürststifts,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Salomon und die Königin von Saba,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Die Trägheit,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Das Erz-Marschallamt,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Jakobs Aussöhnung mit Esau,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Der Tod Joachims,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Die Tollheit,"Kempten, fürstbischöfliche Res",", Bayern"


Year,Painting,Building,Location
~ca. 1569,Himmelfahrt Jesu Christi,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Engel mit leerem Spruchband,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Entwurf für ein Wandgemälde: Opferung der Iphigenie,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Engel mit Flöte,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,David überschreitet den Bach Kidron,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Engel mit Harfe,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Beschneidung Isaaks,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Verklärung Jesu Christi,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,„DOLET.AST.REDOLET“,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Himmelfahrt des Elias,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"


Year,Painting,Building,Location
"~nach 1559, vermutlich 1563",Wand- und Deckenmalerei in Turmzimmer 1,"Stolpen, Burg",", Sachsen"
~nach 1559,Wand- und Deckenmalerei im Turmzimmer 2,"Stolpen, Burg",", Sachsen"
1570-73,"Hasenhaus, nordwestliches Eckhaus, Erdgeschoß, Raum (nicht l...","Augustusburg, Schloss",", Sachsen"
1570-73,"Hasenhaus, Erstes Obergeschoß, Saal, Tür 2, Jagdhörner und R...","Augustusburg, Schloss",", Sachsen"
1570-73,"Hasenhaus, nordwestliches Eckhaus, Erstes Obergeschoß, Raum ...","Augustusburg, Schloss",", Sachsen"
1570-73,"Hasenhaus, Erstes Obergeschoß, Saal, Tür 5, Tanz mit Trommle...","Augustusburg, Schloss",", Sachsen"
1570-73,"Hasenhaus, Erstes Obergeschoß, Saal, Tür 4, Tanz und Pyramid...","Augustusburg, Schloss",", Sachsen"
1570-73,"Hasenhaus, Erstes Obergeschoß, Saal, Tür 6, Tanz mit Kranz","Augustusburg, Schloss",", Sachsen"
1570-73,"Hasenhaus, Erdgeschoß, Stube, Tür 2, Hasen überfallen Jäger","Augustusburg, Schloss",", Sachsen"
1570-73,"Hasenhaus, nordwestliches Eckhaus, Erstes Obergeschoß, Kamme...","Augustusburg, Schloss",", Sachsen"


In [ ]:
# =============================================================================
# CELL: Timeline Visualization for Each Painter
# =============================================================================
# Create visual timelines showing when each painter created works

import pandas as pd

print("📅 PAINTER CAREER TIMELINES")
print("=" * 70)

def create_timeline_chart(bio):
    """Create an Altair timeline visualization for a painter."""
    
    name = bio['name']
    events = bio['timeline_events']
    
    # Filter to events with years (exclude Bildindex items without dates)
    dated_events = [e for e in events if e.get('year') is not None]
    
    if not dated_events:
        return None, "No dated events available"
    
    # Create DataFrame
    timeline_df = pd.DataFrame(dated_events)
    
    # Define colors by type and source
    color_map = {
        'life': '#2c3e50',      # Dark blue-gray for birth/death
        'painting': '#3498db',   # Blue for CbDD paintings
        'bildindex_photo': '#e67e22'  # Orange for Bildindex
    }
    
    # Create a source column for coloring
    timeline_df['color_key'] = timeline_df['type']
    
    # Add jitter for overlapping years
    year_counts = timeline_df.groupby('year').cumcount()
    timeline_df['y_offset'] = year_counts * 0.3
    
    # Get life span for chart bounds
    birth_year = bio['cbdd'].get('birth_year')
    death_year = bio['cbdd'].get('death_year')
    
    min_year = timeline_df['year'].min() - 5
    max_year = timeline_df['year'].max() + 5
    
    if birth_year and birth_year < min_year:
        min_year = birth_year - 5
    if death_year and death_year > max_year:
        max_year = death_year + 5
    
    # Main scatter plot
    points = alt.Chart(timeline_df).mark_circle(size=100).encode(
        x=alt.X('year:Q', 
                scale=alt.Scale(domain=[min_year, max_year]),
                title='Year'),
        y=alt.Y('y_offset:Q', 
                axis=None,
                scale=alt.Scale(domain=[-0.5, max(5, timeline_df['y_offset'].max() + 1)])),
        color=alt.Color('type:N',
                       scale=alt.Scale(
                           domain=['life', 'painting'],
                           range=['#2c3e50', '#3498db']
                       ),
                       title='Event Type'),
        tooltip=[
            alt.Tooltip('year:Q', title='Year'),
            alt.Tooltip('event:N', title='Event'),
            alt.Tooltip('source:N', title='Source'),
            alt.Tooltip('location:N', title='Location')
        ]
    )
    
    # Add life span bar if we have birth/death
    if birth_year and death_year:
        life_df = pd.DataFrame([{
            'start': birth_year,
            'end': death_year,
            'y': -0.3
        }])
        
        life_bar = alt.Chart(life_df).mark_rule(
            color='#95a5a6',
            strokeWidth=8,
            opacity=0.5
        ).encode(
            x='start:Q',
            x2='end:Q',
            y='y:Q'
        )
        
        chart = alt.layer(life_bar, points)
    else:
        chart = points
    
    # Final chart config
    chart = chart.properties(
        title=f"Career Timeline: {name}",
        width=700,
        height=200
    ).configure_view(
        strokeWidth=0
    )
    
    return chart, None

def create_combined_timeline(biographies):
    """Create a combined timeline showing all three painters."""
    
    all_events = []
    for bio in biographies:
        dated_events = [e for e in bio['timeline_events'] if e.get('year') is not None]
        for e in dated_events:
            e['painter'] = bio['name']
        all_events.extend(dated_events)
    
    if not all_events:
        return None
    
    df = pd.DataFrame(all_events)
    
    chart = alt.Chart(df).mark_circle(size=60, opacity=0.7).encode(
        x=alt.X('year:Q', title='Year'),
        y=alt.Y('painter:N', title='Painter'),
        color=alt.Color('type:N',
                       scale=alt.Scale(
                           domain=['life', 'painting'],
                           range=['#e74c3c', '#3498db']
                       ),
                       title='Event Type'),
        tooltip=[
            alt.Tooltip('painter:N', title='Painter'),
            alt.Tooltip('year:Q', title='Year'),
            alt.Tooltip('event:N', title='Event'),
            alt.Tooltip('source:N', title='Source')
        ]
    ).properties(
        title='Combined Career Timelines: Top 3 Painters with GND Overlap',
        width=800,
        height=200
    )
    
    return chart

# ===========================================
# Generate Visualizations
# ===========================================

if 'PAINTER_BIOGRAPHIES' in dir() and PAINTER_BIOGRAPHIES:
    # Combined timeline
    print("\n📊 Combined Timeline (all 3 painters):")
    combined_chart = create_combined_timeline(PAINTER_BIOGRAPHIES)
    if combined_chart:
        ipython_display(combined_chart)
    else:
        print("  ⚠️ Not enough data for combined timeline")
    
    # Individual timelines
    print("\n" + "=" * 70)
    print("📊 Individual Timelines:")
    
    for i, bio in enumerate(PAINTER_BIOGRAPHIES, 1):
        print(f"\n  {i}. {bio['name']}")
        chart, error = create_timeline_chart(bio)
        if chart:
            ipython_display(chart)
        else:
            print(f"     ⚠️ {error}")
else:
    print("⚠️ No biographies available. Run the previous cells first.")

📅 PAINTER CAREER TIMELINES

📊 Combined Timeline (all 3 painters):


alt.Chart(...)


📊 Individual Timelines:

  1. Hermann, Franz Georg


alt.Chart(...)


  2. Harms, Johann Oswald


alt.Chart(...)


  3. Göding, Heinrich


alt.Chart(...)

In [ ]:
# =============================================================================
# CELL: Data Coverage Comparison - CbDD vs Bildindex
# =============================================================================
# Visual comparison of what data each source provides for the top painters

import pandas as pd

print("📊 DATA COVERAGE COMPARISON: CbDD vs Bildindex")
print("=" * 70)

if 'PAINTER_BIOGRAPHIES' in dir() and PAINTER_BIOGRAPHIES:
    # Build comparison data
    comparison_data = []
    
    for bio in PAINTER_BIOGRAPHIES:
        name = bio['name']
        
        # CbDD metrics
        comparison_data.append({
            'painter': name,
            'source': '🟦 CbDD',
            'metric': 'Artworks',
            'count': bio['cbdd'].get('painting_count', 0)
        })
        comparison_data.append({
            'painter': name,
            'source': '🟦 CbDD',
            'metric': 'Buildings',
            'count': bio['cbdd'].get('building_count', 0)
        })
        comparison_data.append({
            'painter': name,
            'source': '🟦 CbDD',
            'metric': 'Collaborators',
            'count': bio['cbdd'].get('collaborator_count', 0)
        })
        comparison_data.append({
            'painter': name,
            'source': '🟦 CbDD',
            'metric': 'Subjects',
            'count': len(bio['cbdd'].get('subjects', []))
        })
        
        # Bildindex metrics
        comparison_data.append({
            'painter': name,
            'source': '🟧 Bildindex',
            'metric': 'Artworks',
            'count': bio['bildindex'].get('item_count', 0)
        })
        comparison_data.append({
            'painter': name,
            'source': '🟧 Bildindex',
            'metric': 'Buildings',
            'count': bio['bildindex'].get('building_count', 0)
        })
        comparison_data.append({
            'painter': name,
            'source': '🟧 Bildindex',
            'metric': 'Collaborators',
            'count': 0  # Not available in Bildindex
        })
        comparison_data.append({
            'painter': name,
            'source': '🟧 Bildindex',
            'metric': 'Subjects',
            'count': len(bio['bildindex'].get('subjects', []))
        })
    
    df = pd.DataFrame(comparison_data)
    
    # Create grouped bar chart
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X('metric:N', title='Data Category', axis=alt.Axis(labelAngle=0)),
        y=alt.Y('count:Q', title='Count'),
        color=alt.Color('source:N', 
                       scale=alt.Scale(
                           domain=['🟦 CbDD', '🟧 Bildindex'],
                           range=['#3498db', '#e67e22']
                       ),
                       title='Data Source'),
        xOffset='source:N',
        tooltip=['painter', 'source', 'metric', 'count']
    ).properties(
        width=150,
        height=200
    ).facet(
        column=alt.Column('painter:N', title='Painter', header=alt.Header(labelAngle=0))
    ).properties(
        title='Data Coverage Comparison: CbDD vs Bildindex by Painter'
    )
    
    ipython_display(chart)
    
    # Summary statistics
    print("\n📋 Summary Statistics:")
    print("-" * 50)
    
    total_cbdd = sum(bio['cbdd'].get('painting_count', 0) for bio in PAINTER_BIOGRAPHIES)
    total_bi = sum(bio['bildindex'].get('item_count', 0) for bio in PAINTER_BIOGRAPHIES)
    
    print(f"   Total CbDD paintings across 3 painters:    {total_cbdd}")
    print(f"   Total Bildindex items across 3 painters:   {total_bi}")
    print(f"   Combined cross-referenced items:           {total_cbdd + total_bi}")
    
    # Create a pie chart showing overall distribution
    pie_data = pd.DataFrame([
        {'source': 'CbDD Paintings', 'count': total_cbdd, 'color': '#3498db'},
        {'source': 'Bildindex Images', 'count': total_bi, 'color': '#e67e22'}
    ])
    
    pie_chart = alt.Chart(pie_data).mark_arc(innerRadius=50).encode(
        theta=alt.Theta('count:Q'),
        color=alt.Color('source:N',
                       scale=alt.Scale(
                           domain=['CbDD Paintings', 'Bildindex Images'],
                           range=['#3498db', '#e67e22']
                       ),
                       title='Source'),
        tooltip=['source', 'count']
    ).properties(
        title='Total Items by Data Source (Top 3 Painters)',
        width=300,
        height=300
    )
    
    ipython_display(pie_chart)
    
else:
    print("⚠️ No biographies available. Run the previous cells first.")

📊 DATA COVERAGE COMPARISON: CbDD vs Bildindex
⚠️ No biographies available. Run the previous cells first.


In [ ]:
# =============================================================================
# CELL 15: Close Connection (run when done)
# =============================================================================

# Uncomment to close the connection
# con.close()
# print("✅ DuckDB connection closed")

print("💡 Tip: The database file 'baroque.duckdb' can be shared with collaborators.")
print("   They can open it with:")
print("   >>> con = duckdb.connect('baroque.duckdb')")
print("   >>> con.execute('SELECT * FROM paintings LIMIT 5').pl()")

💡 Tip: The database file 'baroque.duckdb' can be shared with collaborators.
   They can open it with:
   >>> con = duckdb.connect('baroque.duckdb')
   >>> con.execute('SELECT * FROM paintings LIMIT 5').pl()


# TOP1: Analyze Society and Culture

In [2]:
# =============================================================================
# MAC os:  Setup & Configuration 
# =============================================================================
import duckdb
import polars as pl
import altair as alt
import re
from pathlib import Path
from typing import Tuple, Optional
from IPython.display import display as ipython_display
import os

# --------------------------------------------------
# Project & database paths (Mac)
# --------------------------------------------------

# Current working directory (should be .../src/docs/story/CbDD)
BASE_DIR = Path.cwd()

# DuckDB file is located next to this notebook
DUCKDB_PATH = BASE_DIR / "baroque.duckdb"

print("📁 Working directory:", BASE_DIR)
print("🗄️ DuckDB path:", DUCKDB_PATH)
print("✅ DuckDB exists:", DUCKDB_PATH.exists())

# --------------------------------------------------
# Table definitions (same as original notebook)
# --------------------------------------------------

CBDD_TABLES = [
    'paintings', 'persons', 'buildings', 'rooms', 'ensembles',
    'subjects', 'painting_persons', 'painting_subjects',
    'building_persons', 'room_persons'
]

BILDINDEX_TABLES = [
    'bi_items', 'bi_buildings', 'bi_painters',
    'bi_subjects', 'bi_gnd_overlaps'
]

TABLES = CBDD_TABLES

# Enable Altair for larger datasets
alt.data_transformers.disable_max_rows()


📁 Working directory: /Users/yomo/DataStory_Docker/course-data-stories-baroque/src/docs/story/CbDD
🗄️ DuckDB path: /Users/yomo/DataStory_Docker/course-data-stories-baroque/src/docs/story/CbDD/baroque.duckdb
✅ DuckDB exists: True


DataTransformerRegistry.enable('default')

In [3]:
# =============================================================================
# CELL 1b: Reconnect to Existing Database (macOS)
# =============================================================================

if not DUCKDB_PATH.exists():
    raise FileNotFoundError(f"❌ DuckDB not found at {DUCKDB_PATH}")

con = duckdb.connect(str(DUCKDB_PATH), read_only=True)
print(f"✅ Connected to DuckDB (read-only): {DUCKDB_PATH.name}")

tables_df = con.execute("SHOW TABLES").fetchdf()
print(f"📋 Found {len(tables_df)} tables")

tables_df


✅ Connected to DuckDB (read-only): baroque.duckdb


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

📋 Found 15 tables


,name
0,bi_buildings
1,bi_gnd_overlaps
2,bi_items
3,bi_painters
4,bi_subjects
5,building_persons
6,buildings
7,ensembles
8,painting_persons
9,painting_subjects


In [4]:
## analyze iconclass category 4 - society & culture 

import polars as pl

ICONCLASS_CATEGORIES = {
    '0': 'Abstract, Non-representational Art',
    '1': 'Religion and Magic',
    '2': 'Nature',
    '3': 'Human Being, Man in General',
    '4': 'Society, Civilization, Culture',
    '5': 'Abstract Ideas and Concepts',
    '6': 'History',
    '7': 'Bible',
    '8': 'Literature',
    '9': 'Classical Mythology and Ancient History'
}

society_df = con.execute("""
WITH iconclass_parsed AS (
    SELECT
        ps.nfdi_uri,
        s.subject_uri,
        s.subject_label,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE s.subject_source = 'ICONCLASS'
)
SELECT
    iconclass_code,
    SUBSTRING(iconclass_code, 1, 1) AS category_digit,
    COUNT(DISTINCT nfdi_uri) AS painting_count
FROM iconclass_parsed
WHERE iconclass_code IS NOT NULL
GROUP BY iconclass_code, category_digit
ORDER BY painting_count DESC
""").pl()

# nur Kategorie 4
society_df = society_df.filter(pl.col("category_digit") == "4")

print("Unique ICONCLASS codes in category 4:", society_df.select(pl.count()).item())
print("Paintings with Society & Culture (cat 4):", society_df["painting_count"].sum())
society_df.head(20)


Unique ICONCLASS codes in category 4: 1116
Paintings with Society & Culture (cat 4): 4387


/var/folders/zh/bhwyldrs6916tshqvlz58l2c0000gn/T/ipykernel_35194/915017725.py:42: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  print("Unique ICONCLASS codes in category 4:", society_df.select(pl.count()).item())


iconclass_code,category_digit,painting_count
str,str,i64
"""48C161""","""4""",62
"""48A9876""","""4""",58
"""48A9872""","""4""",57
"""44B191""","""4""",55
"""48A9875""","""4""",54
…,…,…
"""41D26611""","""4""",32
"""49MM32""","""4""",32
"""41A421""","""4""",31


In [5]:
society_subjects = con.execute("""
WITH iconclass_parsed AS (
    SELECT
        ps.nfdi_uri,
        s.subject_label,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE s.subject_source = 'ICONCLASS'
)
SELECT
    iconclass_code,
    subject_label,
    COUNT(DISTINCT nfdi_uri) AS painting_count
FROM iconclass_parsed
WHERE iconclass_code LIKE '4%'
GROUP BY iconclass_code, subject_label
ORDER BY painting_count DESC
LIMIT 30
""").pl()

society_subjects


iconclass_code,subject_label,painting_count
str,str,i64
"""48C161""","""column, pillar ~ architecture""",62
"""48A9876""","""ornament ~ cartouche""",58
"""48A9872""","""ornament ~ grotesque""",57
"""44B191""","""crown (symbol of sovereignty)""",55
"""48A9875""","""ornament ~ festoon, garland""",54
…,…,…
"""49L1""","""letters, alphabet, script""",25
"""43C1147""","""hunting dogs""",25
"""44B112""","""emperor""",25


In [6]:
con.execute("""
SELECT
  COUNT(*) AS n_iconclass,
  SUM(CASE WHEN subject_uri LIKE '%iconclass.org/%' THEN 1 ELSE 0 END) AS n_iconclass_uris
FROM subjects
WHERE subject_source = 'ICONCLASS'
""").fetchdf()


,n_iconclass,n_iconclass_uris
0,4080,4080.0


In [7]:
##how many paintings belong to the class society and culture?

society_summary = con.execute("""
WITH iconclass_parsed AS (
    SELECT
        ps.nfdi_uri,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE s.subject_source = 'ICONCLASS'
)
SELECT
    COUNT(DISTINCT nfdi_uri) AS paintings_society_culture
FROM iconclass_parsed
WHERE iconclass_code LIKE '4%'
""").fetchdf()

society_summary




,paintings_society_culture
0,1417


In [8]:
## which themes dominate in society & culture?

society_top_subjects = con.execute("""
WITH iconclass_parsed AS (
    SELECT
        ps.nfdi_uri,
        s.subject_label,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE s.subject_source = 'ICONCLASS'
)
SELECT
    iconclass_code,
    subject_label,
    COUNT(DISTINCT nfdi_uri) AS painting_count
FROM iconclass_parsed
WHERE iconclass_code LIKE '4%'
GROUP BY iconclass_code, subject_label
ORDER BY painting_count DESC
LIMIT 20
""").pl()

society_top_subjects


iconclass_code,subject_label,painting_count
str,str,i64
"""48C161""","""column, pillar ~ architecture""",62
"""48A9876""","""ornament ~ cartouche""",58
"""48A9872""","""ornament ~ grotesque""",57
"""44B191""","""crown (symbol of sovereignty)""",55
"""48A9875""","""ornament ~ festoon, garland""",54
…,…,…
"""41D26611""","""pearl; pearl necklace""",32
"""49MM32""","""book - MM - book open""",32
"""41A421""","""curtains""",31


In [9]:
##try to get an overview of TOP1 -> top 10 painters

top10_society_artists = con.execute("""
WITH cat4_paintings AS (
  SELECT DISTINCT ps.nfdi_uri
  FROM painting_subjects ps
  JOIN subjects s ON ps.subject_uri = s.subject_uri
  WHERE s.subject_source = 'ICONCLASS'
    AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '4%'
)
SELECT
  pp.person_name AS artist,
  COUNT(DISTINCT pp.nfdi_uri) AS paintings
FROM painting_persons pp
JOIN cat4_paintings c ON c.nfdi_uri = pp.nfdi_uri
WHERE pp.role = 'PAINTER'
GROUP BY pp.person_name
ORDER BY paintings DESC
LIMIT 10
""").fetchdf()

top10_society_artists

,artist,paintings
0,"Hermann, Franz Georg",45
1,"Harms, Johann Oswald",42
2,"Lammers, Seivert",37
3,"Asam, Cosmas Damian",34
4,"Kuen, Franz Martin",33
5,"Kilian, Magnus",31
6,"Wolcker, Johann Georg",29
7,"Castelli, Carlo Ludovico",26
8,"Ritter, Johann Heinrich",21
9,"Paduano, Alessandro",20


In [10]:
society_overview = con.execute("""
WITH cat4_paintings AS (
  SELECT DISTINCT ps.nfdi_uri
  FROM painting_subjects ps
  JOIN subjects s ON ps.subject_uri = s.subject_uri
  WHERE s.subject_source = 'ICONCLASS'
    AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '4%'
),
commissioners AS (
  SELECT nfdi_uri, STRING_AGG(DISTINCT person_name, ', ') AS commissioners
  FROM painting_persons
  WHERE role = 'COMMISSIONER'
  GROUP BY nfdi_uri
),
donors AS (
  SELECT nfdi_uri, STRING_AGG(DISTINCT person_name, ', ') AS donors
  FROM painting_persons
  WHERE role = 'DONOR'
  GROUP BY nfdi_uri
)
SELECT
  pp.person_name AS artist,
  COUNT(DISTINCT p.nfdi_uri) AS paintings,
  STRING_AGG(DISTINCT p.location_state, ', ') AS states,
  STRING_AGG(DISTINCT p.building_name, ' | ') AS buildings,
  STRING_AGG(DISTINCT
    CASE
      WHEN p.year_start IS NOT NULL AND p.year_end IS NOT NULL AND p.year_start <> p.year_end
        THEN CAST(p.year_start AS VARCHAR) || '-' || CAST(p.year_end AS VARCHAR)
      WHEN p.year_start IS NOT NULL
        THEN CAST(p.year_start AS VARCHAR)
      ELSE p.year
    END,
    ', '
  ) AS time_span,
  STRING_AGG(DISTINCT c.commissioners, ' | ') AS commissioners,
  STRING_AGG(DISTINCT d.donors, ' | ') AS donors
FROM painting_persons pp
JOIN cat4_paintings c4 ON c4.nfdi_uri = pp.nfdi_uri
JOIN paintings p ON p.nfdi_uri = pp.nfdi_uri
LEFT JOIN commissioners c ON c.nfdi_uri = p.nfdi_uri
LEFT JOIN donors d ON d.nfdi_uri = p.nfdi_uri
WHERE pp.role = 'PAINTER'
GROUP BY pp.person_name
ORDER BY paintings DESC
LIMIT 10
""").fetchdf()

society_overview

,artist,paintings,states,buildings,time_span,commissioners,donors
0,"Hermann, Franz Georg",45,Bayern,"Kempten, fürstbischöfliche Residenz",1732-1742,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",None
1,"Harms, Johann Oswald",42,"Thüringen, Sachsen-Anhalt, Hessen","Kassel, Orangerie | Eisenberg, Residenzschloss...","1682, 1685-1686, 1701, 1683, 1688, 1685, 1684,...","Johann Adolf I., Herzog von Sachsen-Weißenfels...",None
2,"Lammers, Seivert",37,Thüringen,"Sondershausen, Residenzschloss | Leutenberg, e...","1697, 1698, 1690-1703, 1696-1697","Friedrich II., Sachsen-Gotha-Altenburg, Herzog...",None
3,"Asam, Cosmas Damian",34,"Bayern, Baden-Württemberg","München, St. Johann Nepomuk, sog. Asamkirche |...","1670-1699, 1703-1705, 1943-1948, 1734, 1704, 1738","Königsfeld, Johann Georg Joseph von | Karl Phi...",None
4,"Kuen, Franz Martin",33,Bayern,"Illertissen, Vöhlinschloss, Vorderes Schloss",1751,"Vöhlin von Frickenhausen, Johann Joseph",None
5,"Kilian, Magnus",31,Bayern,Schloss Höchstädt,"1710-1720, 1601","Philipp Ludwig, Pfalz-Neuburg, Pfalzgraf",None
6,"Wolcker, Johann Georg",29,Bayern,"Markt, Schlosskapelle St. Johannes der Täufer","1766, 1739","Fugger von Wellenburg, Joseph Maria",None
7,"Castelli, Carlo Ludovico",26,"Hessen, Thüringen","Saalfeld, Residenzschloss | Altenburg, Residen...","1712, 1720, 1683-1684, 1683, 1718-1722, 1692-1...","Friedrich II., Sachsen-Gotha-Altenburg, Herzog...",None
8,"Ritter, Johann Heinrich",21,Thüringen,"Wechmar, Landhaus Studnitz | Waltershausen, Sc...","1779-1780, 1580, 1733, 1750, 1745-1755, 1719, ...","Studnitz, Hans Adam von | Friedrich III., Sach...",None
9,"Paduano, Alessandro",20,Bayern,"Landshut, Burg Trausnitz","1703-1705, 1704, 1500-1599, 1691-1695, 1578-15...","Wilhelm V., Bayern, Herzog",None


In [11]:
##more detailed analyzation 

import pandas as pd
from IPython.display import Image, display

ICONCLASS_LVL2_MAP = {
    "41": "material aspects of daily life",
    "42": "family, descendance",
    "43": "recreation, amusement",
    "44": "state; law; political life",
    "45": "warfare; military affairs",
    "46": "social & economic life; transport & communication",
    "47": "crafts & industries",
    "48": "art",
    "49": "education, science and learning"
}

def cat4_lvl2_breakdown(artist_name: str, top_n: int = 6):
    """
    1) Für einen Künstler: Verteilung der ICONCLASS-Subkategorien innerhalb Kategorie 4
       (Level-2 Codes: 41..49) inkl. #Paintings.
    """
    sql = f"""
    WITH iconclass_parsed AS (
      SELECT
        ps.nfdi_uri,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code
      FROM painting_subjects ps
      JOIN subjects s ON ps.subject_uri = s.subject_uri
      WHERE s.subject_source = 'ICONCLASS'
    ),
    cat4 AS (
      SELECT DISTINCT nfdi_uri, iconclass_code
      FROM iconclass_parsed
      WHERE iconclass_code LIKE '4%'
    ),
    artist_cat4 AS (
      SELECT DISTINCT
        pp.nfdi_uri,
        REGEXP_EXTRACT(c.iconclass_code, '^(\\d{{2}})', 1) AS lvl2_code
      FROM painting_persons pp
      JOIN cat4 c ON c.nfdi_uri = pp.nfdi_uri
      WHERE pp.role='PAINTER'
        AND pp.person_name = ?
        AND REGEXP_EXTRACT(c.iconclass_code, '^(\\d{{2}})', 1) IS NOT NULL
    )
    SELECT
      lvl2_code,
      COUNT(DISTINCT nfdi_uri) AS painting_count
    FROM artist_cat4
    GROUP BY lvl2_code
    ORDER BY painting_count DESC
    LIMIT {top_n}
    """
    df = con.execute(sql, [artist_name]).fetchdf()
    df["lvl2_name"] = df["lvl2_code"].map(ICONCLASS_LVL2_MAP).fillna("")
    return df[["lvl2_code", "lvl2_name", "painting_count"]]


def paintings_detail_cat4(artist_name: str, limit: int = None):
    """
    2) Pro Painting eine Zeile mit Kontext: wann/wo/gebäude/raum + commissioner/donor
       + zusätzlich: was (Top ICONCLASS labels innerhalb Kategorie 4 als Kurzliste)
    """
    lim = f"LIMIT {int(limit)}" if limit else ""
    
    sql = f"""
    WITH iconclass_parsed AS (
      SELECT
        ps.nfdi_uri,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code,
        s.subject_label
      FROM painting_subjects ps
      JOIN subjects s ON ps.subject_uri = s.subject_uri
      WHERE s.subject_source = 'ICONCLASS'
    ),
    cat4 AS (
      SELECT *
      FROM iconclass_parsed
      WHERE iconclass_code LIKE '4%'
    ),
    painter_hits AS (
      SELECT DISTINCT pp.nfdi_uri
      FROM painting_persons pp
      WHERE pp.role='PAINTER'
        AND pp.person_name = ?
    ),
    painter_cat4 AS (
      SELECT DISTINCT ph.nfdi_uri
      FROM painter_hits ph
      JOIN cat4 c ON c.nfdi_uri = ph.nfdi_uri
    ),
    commissioners AS (
      SELECT nfdi_uri, STRING_AGG(DISTINCT person_name, ', ') AS commissioners
      FROM painting_persons
      WHERE role='COMMISSIONER'
      GROUP BY nfdi_uri
    ),
    donors AS (
      SELECT nfdi_uri, STRING_AGG(DISTINCT person_name, ', ') AS donors
      FROM painting_persons
      WHERE role='DONOR'
      GROUP BY nfdi_uri
    ),
    -- "was": kurze Liste der häufigsten Cat4-Subject Labels pro Painting
    cat4_labels AS (
      SELECT
        nfdi_uri,
        STRING_AGG(DISTINCT subject_label, ' | ') AS cat4_subjects
      FROM cat4
      WHERE subject_label IS NOT NULL
      GROUP BY nfdi_uri
    )
    SELECT
      p.nfdi_uri,
      CASE
        WHEN p.year_start IS NOT NULL AND p.year_end IS NOT NULL AND p.year_start <> p.year_end
          THEN CAST(p.year_start AS VARCHAR) || '-' || CAST(p.year_end AS VARCHAR)
        WHEN p.year_start IS NOT NULL
          THEN CAST(p.year_start AS VARCHAR)
        ELSE p.year
      END AS time,
      p.location_state,
      p.building_name,
      p.room_name,
      COALESCE(c4l.cat4_subjects, '') AS what_cat4_subjects,
      COALESCE(c.commissioners, '') AS commissioners,
      COALESCE(d.donors, '') AS donors,
      p.imageUrl
    FROM painter_cat4 pc
    JOIN paintings p ON p.nfdi_uri = pc.nfdi_uri
    LEFT JOIN commissioners c ON c.nfdi_uri = p.nfdi_uri
    LEFT JOIN donors d ON d.nfdi_uri = p.nfdi_uri
    LEFT JOIN cat4_labels c4l ON c4l.nfdi_uri = p.nfdi_uri
    ORDER BY p.year_start NULLS LAST, p.year_end NULLS LAST, p.building_name
    {lim}
    """
    return con.execute(sql, [artist_name]).fetchdf()


def show_paintings_with_images(df: pd.DataFrame, max_images: int = 10, img_width: int = 350):
    """
    Notebook-Ausgabe: pro Zeile Text + Bild (falls vorhanden).
    max_images begrenzt nur die Anzahl angezeigter Bilder (nicht die DF-Zeilen).
    """
    shown = 0
    for _, row in df.iterrows():
        print(f"🖼 {row.get('building_name','')} | {row.get('room_name','')} | {row.get('time','')} | {row.get('location_state','')}")
        if row.get("what_cat4_subjects"):
            print("What (Cat4):", row["what_cat4_subjects"][:250] + ("..." if len(str(row["what_cat4_subjects"])) > 250 else ""))
        print("Commissioner:", row.get("commissioners",""))
        print("Donor:", row.get("donors",""))
        
        url = row.get("imageUrl","")
        if pd.notna(url) and url and shown < max_images:
            display(Image(url=url, width=img_width))
            shown += 1
        elif not url:
            print("⚠️ No imageUrl")
        
        print("-" * 90)

In [12]:
import pandas as pd

TOP10 = [
    "Hermann, Franz Georg",
    "Harms, Johann Oswald",
    "Lammers, Seivert",
    "Asam, Cosmas Damian",
    "Kuen, Franz Martin",
    "Kilian, Magnus",
    "Wolcker, Johann Georg",
    "Castelli, Carlo Ludovico",
    "Ritter, Johann Heinrich",
    "Paduano, Alessandro",
]

def artist_total_and_cat4_share(artist_name: str):
    sql = """
    WITH artist_paintings AS (
        SELECT DISTINCT nfdi_uri
        FROM painting_persons
        WHERE UPPER(TRIM(role))='PAINTER'
          AND TRIM(person_name)=TRIM(?)
    ),
    cat4_paintings AS (
        SELECT DISTINCT ps.nfdi_uri
        FROM painting_subjects ps
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        WHERE s.subject_source='ICONCLASS'
          AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '4%'
    ),
    artist_cat4 AS (
        SELECT DISTINCT ap.nfdi_uri
        FROM artist_paintings ap
        JOIN cat4_paintings c4 ON c4.nfdi_uri = ap.nfdi_uri
    )
    SELECT
        ? AS artist,
        (SELECT COUNT(*) FROM artist_paintings) AS total_paintings,
        (SELECT COUNT(*) FROM artist_cat4) AS cat4_paintings,
        ROUND(
            (SELECT COUNT(*) FROM artist_cat4) * 100.0 /
            NULLIF((SELECT COUNT(*) FROM artist_paintings), 0),
            2
        ) AS cat4_share_percent
    """
    return con.execute(sql, [artist_name, artist_name]).fetchdf()

# für alle Top10 ausführen
rows = []
for a in TOP10:
    rows.append(artist_total_and_cat4_share(a))

df_top10_share = pd.concat(rows, ignore_index=True).sort_values(
    by=["cat4_share_percent", "cat4_paintings"], ascending=False
)

df_top10_share

,artist,total_paintings,cat4_paintings,cat4_share_percent
5,"Kilian, Magnus",33,31,93.94
0,"Hermann, Franz Georg",54,45,83.33
6,"Wolcker, Johann Georg",35,29,82.86
4,"Kuen, Franz Martin",40,33,82.50
3,"Asam, Cosmas Damian",63,34,53.97
9,"Paduano, Alessandro",38,20,52.63
2,"Lammers, Seivert",82,37,45.12
7,"Castelli, Carlo Ludovico",70,26,37.14
8,"Ritter, Johann Heinrich",58,21,36.21
1,"Harms, Johann Oswald",119,42,35.29


In [13]:
#drill down for painter -> just put the name of the artist in there

artist = "Asam, Cosmas Damian"
cat4_lvl2_breakdown(artist, top_n=10)

,lvl2_code,lvl2_name,painting_count
0,41,material aspects of daily life,16
1,48,art,14
2,44,state; law; political life,13
3,46,social & economic life; transport & communication,12
4,45,warfare; military affairs,9
5,49,"education, science and learning",6
6,42,"family, descendance",5
7,47,crafts & industries,3
8,43,"recreation, amusement",1


In [14]:
import pandas as pd

def map_artist_locations(artist_name: str, only_cat4: bool = False):
    import folium
    from folium.plugins import MarkerCluster
    
    # optional: nur ICONCLASS Kategorie 4 (Society & Culture)
    cat_filter_sql = ""
    if only_cat4:
        cat_filter_sql = """
        AND p.nfdi_uri IN (
          SELECT DISTINCT ps.nfdi_uri
          FROM painting_subjects ps
          JOIN subjects s ON ps.subject_uri = s.subject_uri
          WHERE s.subject_source='ICONCLASS'
            AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '4%'
        )
        """

    df = con.execute(f"""
    SELECT
      p.building_name,
      p.location_state,
      p.lat,
      p.lon,
      COUNT(DISTINCT p.nfdi_uri) AS n_paintings,
      MIN(p.year_start) AS year_min,
      MAX(p.year_end) AS year_max
    FROM painting_persons pp
    JOIN paintings p ON p.nfdi_uri = pp.nfdi_uri
    WHERE pp.role='PAINTER'
      AND pp.person_name = ?
      AND p.lat IS NOT NULL AND p.lon IS NOT NULL
      AND p.building_name IS NOT NULL
      {cat_filter_sql}
    GROUP BY p.building_name, p.location_state, p.lat, p.lon
    ORDER BY n_paintings DESC
    """, [artist_name]).fetchdf()
    
    if df.empty:
        print("⚠️ Keine Geokoordinaten/Buildings gefunden (oder Filter zu streng).")
        return None
    
    # Kartenmittelpunkt
    center_lat = df["lat"].mean()
    center_lon = df["lon"].mean()
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=6, control_scale=True)
    cluster = MarkerCluster(name="Buildings").add_to(m)
    
    for _, r in df.iterrows():
        title = r["building_name"]
        state = r["location_state"] if pd.notna(r["location_state"]) else ""
        n = int(r["n_paintings"])
        y0 = r["year_min"] if pd.notna(r["year_min"]) else ""
        y1 = r["year_max"] if pd.notna(r["year_max"]) else ""
        
        popup_html = f"""
        <b>{title}</b><br>
        <i>{state}</i><br>
        Paintings: <b>{n}</b><br>
        Years: <b>{y0}–{y1}</b>
        """
        
        folium.CircleMarker(
            location=[float(r["lat"]), float(r["lon"])],
            radius=4 + min(n, 20) * 0.6,   # Markergröße ~ Anzahl
            popup=folium.Popup(popup_html, max_width=320),
            tooltip=f"{title} ({n})",
            fill=True,
            fill_opacity=0.7,
            weight=1
        ).add_to(cluster)
    
    folium.LayerControl().add_to(m)
    return m

In [34]:
#drilldown map for all painitngs of painter --> just put name in there

map_artist_locations("Wohlhaupter, Emanuel", only_cat4=False)

In [16]:
#drilldown map only for category 4(society & culture) of painter --> just put name in there

map_artist_locations("Asam, Cosmas Damian", only_cat4=True)

In [17]:
import pandas as pd

def paintings_detail_cat4(artist_name: str, lvl2: str = None, limit: int = None):
    """
    Detail-Liste pro Painting für einen Artist in ICONCLASS Cat4.
    Optional: lvl2='44' (oder '43' etc.) filtert auf Top-Level-2 innerhalb Cat4.
    """
    lim = f"LIMIT {int(limit)}" if limit else ""

    # lvl2 muss 2-stellig sein ("44"), sonst ignorieren wir es
    lvl2_filter = ""
    if lvl2 is not None:
        lvl2 = str(lvl2).strip()
        if len(lvl2) == 2 and lvl2.isdigit():
            lvl2_filter = f"AND SUBSTRING(iconclass_code, 1, 2) = '{lvl2}'"

    sql = f"""
    WITH iconclass_parsed AS (
      SELECT
        ps.nfdi_uri,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code,
        s.subject_label
      FROM painting_subjects ps
      JOIN subjects s ON ps.subject_uri = s.subject_uri
      WHERE s.subject_source = 'ICONCLASS'
    ),
    cat4 AS (
      SELECT *
      FROM iconclass_parsed
      WHERE iconclass_code LIKE '4%'
      {lvl2_filter}
    ),
    painter_hits AS (
      SELECT DISTINCT pp.nfdi_uri
      FROM painting_persons pp
      WHERE UPPER(TRIM(pp.role))='PAINTER'
        AND TRIM(pp.person_name) = TRIM(?)
    ),
    painter_cat4 AS (
      SELECT DISTINCT ph.nfdi_uri
      FROM painter_hits ph
      JOIN cat4 c ON c.nfdi_uri = ph.nfdi_uri
    ),
    commissioners AS (
      SELECT nfdi_uri, STRING_AGG(DISTINCT person_name, ', ') AS commissioners
      FROM painting_persons
      WHERE UPPER(TRIM(role))='COMMISSIONER'
      GROUP BY nfdi_uri
    ),
    donors AS (
      SELECT nfdi_uri, STRING_AGG(DISTINCT person_name, ', ') AS donors
      FROM painting_persons
      WHERE UPPER(TRIM(role))='DONOR'
      GROUP BY nfdi_uri
    ),
    labels AS (
      SELECT
        nfdi_uri,
        STRING_AGG(DISTINCT subject_label, ' | ') AS what_iconclass
      FROM cat4
      WHERE subject_label IS NOT NULL
      GROUP BY nfdi_uri
    )
    SELECT
      p.nfdi_uri,
      CASE
        WHEN p.year_start IS NOT NULL AND p.year_end IS NOT NULL AND p.year_start <> p.year_end
          THEN CAST(p.year_start AS VARCHAR) || '-' || CAST(p.year_end AS VARCHAR)
        WHEN p.year_start IS NOT NULL
          THEN CAST(p.year_start AS VARCHAR)
        ELSE p.year
      END AS time,
      p.location_state,
      p.building_name,
      p.room_name,
      COALESCE(l.what_iconclass, '') AS what_iconclass,
      COALESCE(c.commissioners, '') AS commissioners,
      COALESCE(d.donors, '') AS donors,
      p.imageUrl
    FROM painter_cat4 pc
    JOIN paintings p ON p.nfdi_uri = pc.nfdi_uri
    LEFT JOIN commissioners c ON c.nfdi_uri = p.nfdi_uri
    LEFT JOIN donors d ON d.nfdi_uri = p.nfdi_uri
    LEFT JOIN labels l ON l.nfdi_uri = p.nfdi_uri
    ORDER BY p.year_start NULLS LAST, p.year_end NULLS LAST, p.building_name
    {lim}
    """
    return con.execute(sql, [artist_name]).fetchdf()

In [18]:
#drill down for painter and top 2 level --> change booth for analyzing a different painter or different sublevel

#artist = "Asam, Cosmas Damian"
#df_detail = paintings_detail_cat4(artist, lvl2="4")
#show_paintings_with_images(df_detail, max_images=15, img_width=360)

In [19]:
#drill down for painting -> just put the name of the artist in there


#artist = "Hermann, Franz Georg"

#df_detail = paintings_detail_cat4(artist)   # optional: limit=30
#df_detail.head(10)  # tabellarischer Blick

#show_paintings_with_images(df_detail, max_images=15, img_width=360)

## incorporating top2 level

In [20]:
#Incorporating the next level

society_lvl2 = con.execute("""
WITH iconclass_parsed AS (
  SELECT
    ps.nfdi_uri,
    REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code
  FROM painting_subjects ps
  JOIN subjects s ON ps.subject_uri = s.subject_uri
  WHERE s.subject_source = 'ICONCLASS'
),
level2 AS (
  SELECT
    nfdi_uri,
    iconclass_code,
    REGEXP_EXTRACT(iconclass_code, '^(\\d{2})', 1) AS lvl2_code
  FROM iconclass_parsed
  WHERE iconclass_code LIKE '4%'
),
lvl2_map AS (
  SELECT * FROM (VALUES
    ('41','material aspects of daily life'),
    ('42','family, descendance'),
    ('43','recreation, amusement'),
    ('44','state; law; political life'),
    ('45','warfare; military affairs'),
    ('46','social & economic life; transport & communication'),
    ('47','crafts & industries'),
    ('48','art'),
    ('49','education, science and learning')
  ) AS t(lvl2_code, lvl2_name)
)
SELECT
  l.lvl2_code,
  m.lvl2_name,
  COUNT(DISTINCT l.nfdi_uri) AS painting_count
FROM level2 l
LEFT JOIN lvl2_map m USING(lvl2_code)
WHERE l.lvl2_code IS NOT NULL
GROUP BY l.lvl2_code, m.lvl2_name
ORDER BY painting_count DESC
""").pl()

society_lvl2

lvl2_code,lvl2_name,painting_count
str,str,i64
"""48""","""art""",586
"""41""","""material aspects of daily life""",539
"""44""","""state; law; political life""",269
"""46""","""social & economic life; transp…",261
"""45""","""warfare; military affairs""",249
"""49""","""education, science and learnin…",192
"""43""","""recreation, amusement""",190
"""47""","""crafts & industries""",184
"""42""","""family, descendance""",62


In [21]:
#incorporating next level

society_lvl_deep = con.execute("""
WITH iconclass_parsed AS (
  SELECT
    ps.nfdi_uri,
    REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code
  FROM painting_subjects ps
  JOIN subjects s ON ps.subject_uri = s.subject_uri
  WHERE s.subject_source = 'ICONCLASS'
),
levels AS (
  SELECT
    nfdi_uri,
    iconclass_code,

    -- Level 2: 41..49
    REGEXP_EXTRACT(iconclass_code, '^(\\d{2})', 1) AS lvl2_code,

    -- Level 3: 44A, 44B, 44F, 45A..45M, plus 45(+0)..45(+7)
    CASE
      WHEN REGEXP_MATCHES(iconclass_code, '^\\d{2}[A-Z]') THEN REGEXP_EXTRACT(iconclass_code, '^(\\d{2}[A-Z])', 1)
      WHEN REGEXP_MATCHES(iconclass_code, '^45\\(\\+\\d+\\)') THEN REGEXP_EXTRACT(iconclass_code, '^(45\\(\\+\\d+\\))', 1)
      ELSE NULL
    END AS lvl3_code,

    -- Level 4: 44B0..44B6 and 44F0..44F7
    CASE
      WHEN REGEXP_MATCHES(iconclass_code, '^(44B[0-6])') THEN REGEXP_EXTRACT(iconclass_code, '^(44B[0-6])', 1)
      WHEN REGEXP_MATCHES(iconclass_code, '^(44F[0-7])') THEN REGEXP_EXTRACT(iconclass_code, '^(44F[0-7])', 1)
      ELSE NULL
    END AS lvl4_code

  FROM iconclass_parsed
  WHERE iconclass_code LIKE '4%'
),

lvl2_map AS (
  SELECT * FROM (VALUES
    ('41','material aspects of daily life'),
    ('42','family, descendance'),
    ('43','recreation, amusement'),
    ('44','state; law; political life'),
    ('45','warfare; military affairs'),
    ('46','social & economic life; transport & communication'),
    ('47','crafts & industries'),
    ('48','art'),
    ('49','education, science and learning')
  ) AS t(lvl2_code, lvl2_name)
),

lvl3_map AS (
  SELECT * FROM (VALUES
    -- 44
    ('44A','symbols of the state (in general)'),
    ('44B','government, the State; ''Political''; ''Autorità o Potestà'' (Ripa)'),
    ('44C','the citizen and his rights'),
    ('44D','administrative bodies'),
    ('44E','government services, public services'),
    ('44F','revolution, revolt'),
    ('44G','law and jurisprudence'),
    ('44(+0)','variant'),

    -- 45
    ('45A','war and peace'),
    ('45B','the soldier; the soldier''s life'),
    ('45C','(military) equipment and supply'),
    ('45D','insignia; division of armed forces; ranks'),
    ('45E','communication and transport'),
    ('45F','preparation for battle'),
    ('45G','defensive measures (military)'),
    ('45H','battle, fighting in general'),
    ('45I','after the battle'),
    ('45K','siege, position war'),
    ('45L','occupation ~ war'),
    ('45M','non-combatants ~ war'),
    ('45(+0)','variant'),
    ('45(+1)','during the battle'),
    ('45(+2)','land forces'),
    ('45(+3)','naval force'),
    ('45(+4)','air force'),
    ('45(+5)','ranks; division of armed forces'),
    ('45(+6)','weapons'),
    ('45(+7)','situation of the terrain ~ warfare')
  ) AS t(lvl3_code, lvl3_name)
),

lvl4_map AS (
  SELECT * FROM (VALUES
    -- 44B details
    ('44B0','governmental systems'),
    ('44B1','ruler, sovereign'),
    ('44B2','governmental and representative bodies'),
    ('44B3','water-control administration'),
    ('44B4','diplomacy, diplomat'),
    ('44B5','law-giving power'),
    ('44B6','good and bad government'),
    ('44B(+0)','variant'),

    -- 44F details
    ('44F0','''Seditio''; ''Rebellione''; ''Seditione civile'' (Ripa)'),
    ('44F1','street-fights, riots'),
    ('44F2','civil war'),
    ('44F3','state of emergency'),
    ('44F4','summary justice'),
    ('44F5','revolutionary activities'),
    ('44F6','the old government and its supporters (after revolution)'),
    ('44F7','revolutionary government'),
    ('44F(+0)','variant')
  ) AS t(lvl4_code, lvl4_name)
)

SELECT
  l.lvl2_code,
  l2.lvl2_name,
  l.lvl3_code,
  l3.lvl3_name,
  l.lvl4_code,
  l4.lvl4_name,
  COUNT(DISTINCT l.nfdi_uri) AS painting_count
FROM levels l
LEFT JOIN lvl2_map l2 USING(lvl2_code)
LEFT JOIN lvl3_map l3 USING(lvl3_code)
LEFT JOIN lvl4_map l4 USING(lvl4_code)
WHERE l.lvl2_code IS NOT NULL
GROUP BY
  l.lvl2_code, l2.lvl2_name,
  l.lvl3_code, l3.lvl3_name,
  l.lvl4_code, l4.lvl4_name
ORDER BY painting_count DESC
""").pl()

society_lvl_deep

lvl2_code,lvl2_name,lvl3_code,lvl3_name,lvl4_code,lvl4_name,painting_count
str,str,str,str,str,str,i64
"""48""","""art""","""48C""",null,null,null,324
"""48""","""art""","""48A""",null,null,null,319
"""41""","""material aspects of daily life""","""41A""",null,null,null,310
"""41""","""material aspects of daily life""","""41D""",null,null,null,202
"""44""","""state; law; political life""","""44B""","""government, the State; 'Politi…","""44B1""","""ruler, sovereign""",187
…,…,…,…,…,…,…
"""44""","""state; law; political life""","""44E""","""government services, public se…",null,null,1
"""44""","""state; law; political life""","""44C""","""the citizen and his rights""",null,null,1
"""49""","""education, science and learnin…","""49I""",null,null,null,1


In [22]:

def count_works_for_prefix(prefix):
    return con.execute(f"""
    WITH iconclass_parsed AS (
      SELECT
        ps.nfdi_uri,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\\\.org/([^/]+)', 1) AS iconclass_code
      FROM painting_subjects ps
      JOIN subjects s ON ps.subject_uri = s.subject_uri
      WHERE s.subject_source='ICONCLASS'
    )
    SELECT COUNT(DISTINCT nfdi_uri) AS n_works
    FROM iconclass_parsed
    WHERE iconclass_code LIKE '{prefix}'
    """).fetchone()[0]

In [23]:
# 1) Check: gibt es überhaupt ICONCLASS codes mit 44F?
check_44f = con.execute("""
WITH iconclass_parsed AS (
  SELECT
    ps.nfdi_uri,
    CASE
      WHEN s.subject_uri LIKE '%iconclass.org/%'
        THEN split_part(split_part(s.subject_uri, 'iconclass.org/', 2), '/', 1)
      ELSE NULL
    END AS iconclass_code
  FROM painting_subjects ps
  JOIN subjects s ON ps.subject_uri = s.subject_uri
  WHERE s.subject_source='ICONCLASS'
)
SELECT iconclass_code, COUNT(DISTINCT nfdi_uri) AS n_works
FROM iconclass_parsed
WHERE iconclass_code LIKE '44F%'
   OR iconclass_code LIKE '44F(%'
GROUP BY iconclass_code
ORDER BY n_works DESC
LIMIT 50
""").pl()

check_44f

iconclass_code,n_works
str,i64


In [24]:


TOPN = 5

CATEGORIES = [
  ("44B (government)", "44B%"),
  ("44F (revolution)", "44F%"),
  ("45 (warfare)", "45%")
]

def top_people(prefix, role, topn=5):
    return con.execute(f"""
    WITH iconclass_parsed AS (
      SELECT
        ps.nfdi_uri,
        CASE
          WHEN s.subject_uri LIKE '%iconclass.org/%'
            THEN split_part(split_part(s.subject_uri, 'iconclass.org/', 2), '/', 1)
          ELSE NULL
        END AS iconclass_code
      FROM painting_subjects ps
      JOIN subjects s ON ps.subject_uri = s.subject_uri
      WHERE s.subject_source='ICONCLASS'
    ),
    filtered AS (
      SELECT DISTINCT nfdi_uri
      FROM iconclass_parsed
      WHERE iconclass_code LIKE '{prefix}'
    )
    SELECT
      pp.person_name AS label,
      COUNT(DISTINCT pp.nfdi_uri) AS n
    FROM filtered f
    JOIN painting_persons pp ON pp.nfdi_uri = f.nfdi_uri
    WHERE pp.role = '{role}'
      AND pp.person_name IS NOT NULL AND pp.person_name <> ''
    GROUP BY pp.person_name
    ORDER BY n DESC
    LIMIT {topn}
    """).pl()

def top_states(prefix, topn=5):
    return con.execute(f"""
    WITH iconclass_parsed AS (
      SELECT
        ps.nfdi_uri,
        CASE
          WHEN s.subject_uri LIKE '%iconclass.org/%'
            THEN split_part(split_part(s.subject_uri, 'iconclass.org/', 2), '/', 1)
          ELSE NULL
        END AS iconclass_code
      FROM painting_subjects ps
      JOIN subjects s ON ps.subject_uri = s.subject_uri
      WHERE s.subject_source='ICONCLASS'
    ),
    filtered AS (
      SELECT DISTINCT nfdi_uri
      FROM iconclass_parsed
      WHERE iconclass_code LIKE '{prefix}'
    )
    SELECT
      p.location_state AS label,
      COUNT(DISTINCT p.nfdi_uri) AS n
    FROM filtered f
    JOIN paintings p ON p.nfdi_uri = f.nfdi_uri
    WHERE p.location_state IS NOT NULL AND p.location_state <> ''
    GROUP BY p.location_state
    ORDER BY n DESC
    LIMIT {topn}
    """).pl()

def top_buildings(prefix, topn=5):
    return con.execute(f"""
    WITH iconclass_parsed AS (
      SELECT
        ps.nfdi_uri,
        CASE
          WHEN s.subject_uri LIKE '%iconclass.org/%'
            THEN split_part(split_part(s.subject_uri, 'iconclass.org/', 2), '/', 1)
          ELSE NULL
        END AS iconclass_code
      FROM painting_subjects ps
      JOIN subjects s ON ps.subject_uri = s.subject_uri
      WHERE s.subject_source='ICONCLASS'
    ),
    filtered AS (
      SELECT DISTINCT nfdi_uri
      FROM iconclass_parsed
      WHERE iconclass_code LIKE '{prefix}'
    )
    SELECT
      COALESCE(p.building_name, '(unknown building)') AS label,
      COUNT(DISTINCT p.nfdi_uri) AS n
    FROM filtered f
    JOIN paintings p ON p.nfdi_uri = f.nfdi_uri
    GROUP BY COALESCE(p.building_name, '(unknown building)')
    ORDER BY n DESC
    LIMIT {topn}
    """).pl()

def works_total(prefix):
    return con.execute(f"""
    WITH iconclass_parsed AS (
      SELECT
        ps.nfdi_uri,
        CASE
          WHEN s.subject_uri LIKE '%iconclass.org/%'
            THEN split_part(split_part(s.subject_uri, 'iconclass.org/', 2), '/', 1)
          ELSE NULL
        END AS iconclass_code
      FROM painting_subjects ps
      JOIN subjects s ON ps.subject_uri = s.subject_uri
      WHERE s.subject_source='ICONCLASS'
    )
    SELECT COUNT(DISTINCT nfdi_uri) AS n
    FROM iconclass_parsed
    WHERE iconclass_code LIKE '{prefix}'
    """).pl()[0, "n"]

rows = []

for cat, prefix in CATEGORIES:
    # total
    rows.append({"category": cat, "prefix": prefix, "metric": "works_total", "rank": 1, "label": "(distinct works)", "n": int(works_total(prefix))})

    # top painters / commissioners / states / buildings
    for metric, df in [
        ("painter", top_people(prefix, "PAINTER", TOPN)),
        ("commissioner", top_people(prefix, "COMMISSIONER", TOPN)),
        ("state", top_states(prefix, TOPN)),
        ("building", top_buildings(prefix, TOPN)),
    ]:
        if df.height == 0:
            rows.append({"category": cat, "prefix": prefix, "metric": metric, "rank": 1, "label": "(no data)", "n": 0})
        else:
            for i, r in enumerate(df.iter_rows(named=True), start=1):
                rows.append({"category": cat, "prefix": prefix, "metric": metric, "rank": i, "label": r["label"], "n": int(r["n"])})

import polars as pl
summary_long = pl.DataFrame(rows).sort(["category", "metric", "rank"])
summary_long

category,prefix,metric,rank,label,n
str,str,str,i64,str,i64
"""44B (government)""","""44B%""","""building""",1,"""Gotha, Residenzschloss Frieden…",30
"""44B (government)""","""44B%""","""building""",2,"""Rostock, Herzogliches Theater""",16
"""44B (government)""","""44B%""","""building""",3,"""Fulda, Fürstäbtliches Residenz…",13
"""44B (government)""","""44B%""","""building""",4,"""Kempten, fürstbischöfliche Res…",11
"""44B (government)""","""44B%""","""building""",5,"""Oettingen, Neues Schloss""",9
…,…,…,…,…,…
"""45 (warfare)""","""45%""","""state""",2,"""Thüringen""",47
"""45 (warfare)""","""45%""","""state""",3,"""Baden-Württemberg""",14
"""45 (warfare)""","""45%""","""state""",4,"""Mecklenburg-Vorpommern""",10


In [25]:
import polars as pl

pl.Config.set_tbl_rows(200)      # oder höher als 47
pl.Config.set_tbl_cols(20)       # alle Spalten
pl.Config.set_fmt_str_lengths(200)  # lange Strings nicht abschneiden

summary_long

category,prefix,metric,rank,label,n
str,str,str,i64,str,i64
"""44B (government)""","""44B%""","""building""",1,"""Gotha, Residenzschloss Friedenstein""",30
"""44B (government)""","""44B%""","""building""",2,"""Rostock, Herzogliches Theater""",16
"""44B (government)""","""44B%""","""building""",3,"""Fulda, Fürstäbtliches Residenzschloss""",13
"""44B (government)""","""44B%""","""building""",4,"""Kempten, fürstbischöfliche Residenz""",11
"""44B (government)""","""44B%""","""building""",5,"""Oettingen, Neues Schloss""",9
"""44B (government)""","""44B%""","""commissioner""",1,"""Friedrich I., Sachsen-Gotha-Altenburg, Herzog""",27
"""44B (government)""","""44B%""","""commissioner""",2,"""Christian Ludwig II., Mecklenburg-Schwerin, Herzog""",16
"""44B (government)""","""44B%""","""commissioner""",3,"""Christian Wilhelm, Schwarzburg-Sondershausen, Graf""",8
"""44B (government)""","""44B%""","""commissioner""",4,"""Reichlin-Meldegg, Anselm von, Kempten, Fürstabt""",7


In [26]:
society_overview = con.execute("""
WITH cat4_paintings AS (
  SELECT DISTINCT ps.nfdi_uri
  FROM painting_subjects ps
  JOIN subjects s ON ps.subject_uri = s.subject_uri
  WHERE s.subject_source = 'ICONCLASS'
    AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '45%'
),
commissioners AS (
  SELECT nfdi_uri, STRING_AGG(DISTINCT person_name, ', ') AS commissioners
  FROM painting_persons
  WHERE role = 'COMMISSIONER'
  GROUP BY nfdi_uri
),
donors AS (
  SELECT nfdi_uri, STRING_AGG(DISTINCT person_name, ', ') AS donors
  FROM painting_persons
  WHERE role = 'DONOR'
  GROUP BY nfdi_uri
)
SELECT
  pp.person_name AS artist,
  COUNT(DISTINCT p.nfdi_uri) AS paintings,
  STRING_AGG(DISTINCT p.location_state, ', ') AS states,
  STRING_AGG(DISTINCT p.building_name, ' | ') AS buildings,
  STRING_AGG(DISTINCT
    CASE
      WHEN p.year_start IS NOT NULL AND p.year_end IS NOT NULL AND p.year_start <> p.year_end
        THEN CAST(p.year_start AS VARCHAR) || '-' || CAST(p.year_end AS VARCHAR)
      WHEN p.year_start IS NOT NULL
        THEN CAST(p.year_start AS VARCHAR)
      ELSE p.year
    END,
    ', '
  ) AS time_span,
  STRING_AGG(DISTINCT c.commissioners, ' | ') AS commissioners,
  STRING_AGG(DISTINCT d.donors, ' | ') AS donors
FROM painting_persons pp
JOIN cat4_paintings c4 ON c4.nfdi_uri = pp.nfdi_uri
JOIN paintings p ON p.nfdi_uri = pp.nfdi_uri
LEFT JOIN commissioners c ON c.nfdi_uri = p.nfdi_uri
LEFT JOIN donors d ON d.nfdi_uri = p.nfdi_uri
WHERE pp.role = 'PAINTER'
GROUP BY pp.person_name
ORDER BY paintings DESC
LIMIT 10
""").fetchdf()

society_overview

,artist,paintings,states,buildings,time_span,commissioners,donors
0,"Hermann, Franz Georg",15,Bayern,"Kempten, fürstbischöfliche Residenz",1732-1742,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",None
1,"Lammers, Seivert",13,Thüringen,"Gotha, Residenzschloss Friedenstein | Leutenbe...","1697, 1698, 1696-1697","Friedrich II., Sachsen-Gotha-Altenburg, Herzog...",None
2,"Katzenberger, Balthasar",12,Baden-Württemberg,"Weikersheim, Schloss",1602-1604,"Wolfgang II., Hohenlohe-Weikersheim, Graf",None
3,"Kuen, Franz Martin",12,Bayern,"Illertissen, Vöhlinschloss, Vorderes Schloss",1751,"Vöhlin von Frickenhausen, Johann Joseph",None
4,"Asam, Cosmas Damian",9,Bayern,"München, St. Johann Nepomuk, sog. Asamkirche |...","1703-1705, 1670-1699, 1734, 1704","Asam, Egid Quirin | Königsfeld, Johann Georg J...",None
5,"Speer, Martin",8,Bayern,"Gebelkofen, Schloss | Himmelberg, Sommersitz d...","bis 1765, 1757","Tobiaschu, Adalbert | Lerchenfeld, Maximilian ...",None
6,"Wolcker, Johann Georg",7,Bayern,"Markt, Schlosskapelle St. Johannes der Täufer",1739,"Fugger von Wellenburg, Joseph Maria",None
7,"Asam, Hans Georg",6,Bayern,"Schönach, Schloss | Schmidmühlen, Unteres Schl...","1670-1699, 1704, 1703-1705, 1707","Königsfeld, Johann Georg Joseph von | Vischbac...",None
8,"Paur, Ignaz",5,Bayern,"Hofhegnenberg, Schloss",1750-1751,"Hegnenberg-Dux, Johann Georg Sigismund von",None
9,"Schönfeld, Johann Heinrich",5,Bayern,"Oettingen, Neues Schloss",1683,"Oettingen-Spielberg, Ludovika Rosalie von",None


In [42]:
#Bilder ausgeben lassen mit drilldown (kategorie und künstler)

import re
from IPython.display import Image, display, Markdown

def _list_tables(con):
    # works in DuckDB: SHOW TABLES returns 1 col with table names
    return [r[0] for r in con.execute("SHOW TABLES").fetchall()]

def _table_cols(con, table):
    return [r[1] for r in con.execute(f"PRAGMA table_info('{table}')").fetchall()]

def _find_table_with_cols(con, required_cols, preferred_tables=None):
    tables = _list_tables(con)
    if preferred_tables:
        # preferred first, then the rest
        tables = [t for t in preferred_tables if t in tables] + [t for t in tables if t not in preferred_tables]

    best = None
    best_score = -1
    best_cols = None
    req_set = set(c.lower() for c in required_cols)

    for t in tables:
        cols = _table_cols(con, t)
        cols_l = [c.lower() for c in cols]
        score = sum(1 for c in req_set if c in cols_l)
        if score > best_score:
            best_score = score
            best = t
            best_cols = cols

    return best, best_score, best_cols

def _pick_first_col(cols, candidates):
    cols_l = {c.lower(): c for c in cols}
    for cand in candidates:
        if cand.lower() in cols_l:
            return cols_l[cand.lower()]
    return None

def images_for_artist_and_category(prefix, artist_name, limit=12, debug=True):
    """
    prefix: e.g. '44B%' or '45%'
    artist_name: exact match in painting_persons.person_name
    """

    # 1) Detect "works-like" table by presence of nfdi_uri + some metadata columns
    required = ["nfdi_uri"]
    preferred = ["paintings", "painting", "works", "work", "rooms"]  # rooms is suggested by your error
    works_table, score, cols = _find_table_with_cols(con, required_cols=required, preferred_tables=preferred)

    if works_table is None:
        raise RuntimeError("No tables found. Is the DB loaded?")

    # 2) Choose best-guess columns from that table
    nfdi_col = _pick_first_col(cols, ["nfdi_uri", "uri", "work_uri"])
    title_col = _pick_first_col(cols, ["title", "painting_title", "work_title"])
    ystart_col = _pick_first_col(cols, ["year_start", "date_start", "start_year", "from_year"])
    yend_col   = _pick_first_col(cols, ["year_end", "date_end", "end_year", "to_year"])
    bld_col    = _pick_first_col(cols, ["building_name", "building", "site_name", "location_building"])
    state_col  = _pick_first_col(cols, ["location_state", "state", "bundesland", "region_state"])

    # 3) Detect image url column (very likely NOT in the same table sometimes)
    # Try in works_table first
    img_col = _pick_first_col(cols, ["image_url", "imageurl", "image", "img_url", "depiction", "thumbnail", "thumb_url", "media_url", "file_url"])

    # If not found, search for a table that has (nfdi_uri + image_url-like)
    image_table = works_table
    image_cols = cols
    if img_col is None:
        img_table, img_score, img_cols = _find_table_with_cols(
            con,
            required_cols=["nfdi_uri"],
            preferred_tables=["paintings_with_img", "images", "depictions", "media", "works_with_img", "rooms"]
        )
        # pick image col from the best matching table
        cand_img_col = _pick_first_col(img_cols, ["image_url", "imageurl", "image", "img_url", "depiction", "thumbnail", "thumb_url", "media_url", "file_url"])
        if cand_img_col is not None:
            image_table = img_table
            image_cols = img_cols
            img_col = cand_img_col

    if debug:
        print("Detected works-like table:", works_table, "| nfdi:", nfdi_col, "| image table:", image_table, "| img col:", img_col)

    if nfdi_col is None:
        raise RuntimeError(f"Couldn't find nfdi_uri-like column in table {works_table}. Columns: {cols}")

    if img_col is None:
        raise RuntimeError(
            "Couldn't find an image URL/depiction column in any table.\n"
            "Quick fix: run `SHOW TABLES` and tell me which table contains image URLs (often `imageUrl`)."
        )

    # 4) Build SELECT list safely (only include cols that exist)
    def sel(col, alias):
        return f"w.{col} AS {alias}" if col else f"NULL AS {alias}"

    select_parts = [
        f"ic.nfdi_uri AS nfdi_uri",
        sel(title_col, "title"),
        sel(ystart_col, "year_start"),
        sel(yend_col, "year_end"),
        sel(bld_col, "building_name"),
        sel(state_col, "location_state"),
        f"img.{img_col} AS image_url",
        "ic.iconclass_code AS iconclass_code",
    ]
    select_sql = ",\n      ".join(select_parts)

    # 5) Join strategy:
    # - iconclass_parsed gives nfdi_uri + iconclass_code
    # - painting_persons filters painter
    # - works_table gives metadata
    # - image_table gives image url
    q = f"""
    WITH iconclass_parsed AS (
      SELECT
        ps.nfdi_uri,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code
      FROM painting_subjects ps
      JOIN subjects s ON ps.subject_uri = s.subject_uri
      WHERE s.subject_source = 'ICONCLASS'
    )
    SELECT DISTINCT
      {select_sql}
    FROM iconclass_parsed ic
    JOIN painting_persons pp
      ON ic.nfdi_uri = pp.nfdi_uri
    JOIN {works_table} w
      ON ic.nfdi_uri = w.{nfdi_col}
    JOIN {image_table} img
      ON ic.nfdi_uri = img.nfdi_uri
    WHERE
      ic.iconclass_code LIKE '{prefix}'
      AND pp.role = 'PAINTER'
      AND pp.person_name = '{artist_name}'
      AND img.{img_col} IS NOT NULL
    LIMIT {limit}
    """

    return con.execute(q).pl()

def show_images(df, width=350):
    if df.shape[0] == 0:
        display(Markdown("⚠️ **Keine Treffer** (0 Zeilen). Prüfe `prefix` oder `artist_name` exakt, oder ob Bild-URLs vorhanden sind."))
        return

    for row in df.iter_rows(named=True):
        title = row.get("title") or "(no title)"
        bld = row.get("building_name") or "-"
        st = row.get("location_state") or "-"
        ys = row.get("year_start") or ""
        ye = row.get("year_end") or ""
        code = row.get("iconclass_code") or ""

        display(Markdown(f"**{title}**  \n{bld} ({st})  \n{ys}–{ye}  \n`{code}`"))
        display(Image(url=row["image_url"], width=width))

# --- USAGE (Hier ggfs. ändern) ---
imgs = images_for_artist_and_category(prefix="4%", artist_name="Asam, Cosmas Damian", limit=6, debug=True)
show_images(imgs, width=350)

Detected works-like table: paintings | nfdi: nfdi_uri | image table: paintings | img col: imageUrl


**(no title)**  
Schönach, Schloss (Bayern)  
1703–1705  
`49D5111`

**(no title)**  
München, St. Johann Nepomuk, sog. Asamkirche (Bayern)  
1734–  
`46A15`

**(no title)**  
Schönach, Schloss (Bayern)  
1703–1705  
`48C7341`

**(no title)**  
München, St. Johann Nepomuk, sog. Asamkirche (Bayern)  
1734–  
`41D2666`

**(no title)**  
München, St. Johann Nepomuk, sog. Asamkirche (Bayern)  
1734–  
`44B191`

**(no title)**  
München, St. Johann Nepomuk, sog. Asamkirche (Bayern)  
1734–  
`44G25`

## TOP2: Analyze 44: "state; law; political life"

In [28]:
#incorporating next level

society_lvl_deep = con.execute("""
WITH iconclass_parsed AS (
  SELECT
    ps.nfdi_uri,
    REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code
  FROM painting_subjects ps
  JOIN subjects s ON ps.subject_uri = s.subject_uri
  WHERE s.subject_source = 'ICONCLASS'
),
levels AS (
  SELECT
    nfdi_uri,
    iconclass_code,

    -- Level 2: 41..49
    REGEXP_EXTRACT(iconclass_code, '^(\\d{2})', 1) AS lvl2_code,

    -- Level 3: 44A, 44B, 44F, 45A..45M, plus 45(+0)..45(+7)
    CASE
      WHEN REGEXP_MATCHES(iconclass_code, '^\\d{2}[A-Z]') THEN REGEXP_EXTRACT(iconclass_code, '^(\\d{2}[A-Z])', 1)
      WHEN REGEXP_MATCHES(iconclass_code, '^45\\(\\+\\d+\\)') THEN REGEXP_EXTRACT(iconclass_code, '^(45\\(\\+\\d+\\))', 1)
      ELSE NULL
    END AS lvl3_code,

    -- Level 4: 44B0..44B6 and 44F0..44F7
    CASE
      WHEN REGEXP_MATCHES(iconclass_code, '^(44B[0-6])') THEN REGEXP_EXTRACT(iconclass_code, '^(44B[0-6])', 1)
      WHEN REGEXP_MATCHES(iconclass_code, '^(44F[0-7])') THEN REGEXP_EXTRACT(iconclass_code, '^(44F[0-7])', 1)
      ELSE NULL
    END AS lvl4_code

  FROM iconclass_parsed
  WHERE iconclass_code LIKE '4%'
),

lvl2_map AS (
  SELECT * FROM (VALUES
    ('41','material aspects of daily life'),
    ('42','family, descendance'),
    ('43','recreation, amusement'),
    ('44','state; law; political life'),
    ('45','warfare; military affairs'),
    ('46','social & economic life; transport & communication'),
    ('47','crafts & industries'),
    ('48','art'),
    ('49','education, science and learning')
  ) AS t(lvl2_code, lvl2_name)
),

lvl3_map AS (
  SELECT * FROM (VALUES
    -- 44
    ('44A','symbols of the state (in general)'),
    ('44B','government, the State; ''Political''; ''Autorità o Potestà'' (Ripa)'),
    ('44C','the citizen and his rights'),
    ('44D','administrative bodies'),
    ('44E','government services, public services'),
    ('44F','revolution, revolt'),
    ('44G','law and jurisprudence'),
    ('44(+0)','variant'),

    -- 45
    ('45A','war and peace'),
    ('45B','the soldier; the soldier''s life'),
    ('45C','(military) equipment and supply'),
    ('45D','insignia; division of armed forces; ranks'),
    ('45E','communication and transport'),
    ('45F','preparation for battle'),
    ('45G','defensive measures (military)'),
    ('45H','battle, fighting in general'),
    ('45I','after the battle'),
    ('45K','siege, position war'),
    ('45L','occupation ~ war'),
    ('45M','non-combatants ~ war'),
    ('45(+0)','variant'),
    ('45(+1)','during the battle'),
    ('45(+2)','land forces'),
    ('45(+3)','naval force'),
    ('45(+4)','air force'),
    ('45(+5)','ranks; division of armed forces'),
    ('45(+6)','weapons'),
    ('45(+7)','situation of the terrain ~ warfare')
  ) AS t(lvl3_code, lvl3_name)
),

lvl4_map AS (
  SELECT * FROM (VALUES
    -- 44B details
    ('44B0','governmental systems'),
    ('44B1','ruler, sovereign'),
    ('44B2','governmental and representative bodies'),
    ('44B3','water-control administration'),
    ('44B4','diplomacy, diplomat'),
    ('44B5','law-giving power'),
    ('44B6','good and bad government'),
    ('44B(+0)','variant'),

    -- 44F details
    ('44F0','''Seditio''; ''Rebellione''; ''Seditione civile'' (Ripa)'),
    ('44F1','street-fights, riots'),
    ('44F2','civil war'),
    ('44F3','state of emergency'),
    ('44F4','summary justice'),
    ('44F5','revolutionary activities'),
    ('44F6','the old government and its supporters (after revolution)'),
    ('44F7','revolutionary government'),
    ('44F(+0)','variant')
  ) AS t(lvl4_code, lvl4_name)
)

SELECT
  l.lvl2_code,
  l2.lvl2_name,
  l.lvl3_code,
  l3.lvl3_name,
  l.lvl4_code,
  l4.lvl4_name,
  COUNT(DISTINCT l.nfdi_uri) AS painting_count
FROM levels l
LEFT JOIN lvl2_map l2 USING(lvl2_code)
LEFT JOIN lvl3_map l3 USING(lvl3_code)
LEFT JOIN lvl4_map l4 USING(lvl4_code)
WHERE l.lvl2_code IS NOT NULL
GROUP BY
  l.lvl2_code, l2.lvl2_name,
  l.lvl3_code, l3.lvl3_name,
  l.lvl4_code, l4.lvl4_name
ORDER BY painting_count DESC
""").pl()

society_lvl_deep

lvl2_code,lvl2_name,lvl3_code,lvl3_name,lvl4_code,lvl4_name,painting_count
str,str,str,str,str,str,i64
"""48""","""art""","""48C""",null,null,null,324
"""48""","""art""","""48A""",null,null,null,319
"""41""","""material aspects of daily life""","""41A""",null,null,null,310
"""41""","""material aspects of daily life""","""41D""",null,null,null,202
"""44""","""state; law; political life""","""44B""","""government, the State; 'Political'; 'Autorità o Potestà' (Ripa)""","""44B1""","""ruler, sovereign""",187
"""45""","""warfare; military affairs""","""45C""","""(military) equipment and supply""",null,null,177
"""43""","""recreation, amusement""","""43C""",null,null,null,143
"""47""","""crafts & industries""","""47I""",null,null,null,134
"""46""","""social & economic life; transport & communication""","""46C""",null,null,null,126


In [29]:
# top painter in 44 

state_law_painters = con.execute("""
WITH iconclass_parsed AS (
  SELECT
    ps.nfdi_uri,
    REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code
  FROM painting_subjects ps
  JOIN subjects s ON ps.subject_uri = s.subject_uri
  WHERE s.subject_source = 'ICONCLASS'
),
state_law_paintings AS (
  SELECT DISTINCT nfdi_uri
  FROM iconclass_parsed
  WHERE iconclass_code LIKE '44%'
)
SELECT
  pp.person_name AS painter,
  COUNT(DISTINCT pp.nfdi_uri) AS painting_count
FROM state_law_paintings sl
JOIN painting_persons pp
  ON sl.nfdi_uri = pp.nfdi_uri
WHERE pp.role = 'PAINTER'
  AND pp.person_name IS NOT NULL
GROUP BY pp.person_name
ORDER BY painting_count DESC
LIMIT 20
""").pl()

state_law_painters

painter,painting_count
str,i64
"""Wohlhaupter, Emanuel""",14
"""Asam, Cosmas Damian""",13
"""Lammers, Seivert""",11
"""Hermann, Franz Georg""",10
"""Wolcker, Johann Georg""",9
"""Paduano, Alessandro""",9
"""Wentzing, Heinrich Richard""",8
"""Richter, Johann Wilhelm""",8
"""Asam, Hans Georg""",5


In [30]:
#top auftraggeber für kategorie 44

state_law_commissioners = con.execute("""
WITH iconclass_parsed AS (
  SELECT
    ps.nfdi_uri,
    REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) AS iconclass_code
  FROM painting_subjects ps
  JOIN subjects s ON ps.subject_uri = s.subject_uri
  WHERE s.subject_source = 'ICONCLASS'
),
state_law_paintings AS (
  SELECT DISTINCT nfdi_uri
  FROM iconclass_parsed
  WHERE iconclass_code LIKE '45%'
)
SELECT
  pp.person_name AS commissioner,
  COUNT(DISTINCT pp.nfdi_uri) AS painting_count
FROM state_law_paintings sl
JOIN painting_persons pp
  ON sl.nfdi_uri = pp.nfdi_uri
WHERE pp.role = 'COMMISSIONER'
  AND pp.person_name IS NOT NULL
GROUP BY pp.person_name
ORDER BY painting_count DESC
LIMIT 20
""").pl()

state_law_commissioners

commissioner,painting_count
str,i64
"""Reichlin-Meldegg, Anselm von, Kempten, Fürstabt""",15
"""Vöhlin von Frickenhausen, Johann Joseph""",13
"""Friedrich I., Sachsen-Gotha-Altenburg, Herzog""",12
"""Wolfgang II., Hohenlohe-Weikersheim, Graf""",12
"""Oettingen-Spielberg, Ludovika Rosalie von""",11
"""Fugger von Wellenburg, Joseph Maria""",10
"""Friedrich II., Sachsen-Gotha-Altenburg, Herzog""",10
"""Tobiaschu, Adalbert""",9
"""Christian Ludwig II., Mecklenburg-Schwerin, Herzog""",7


## TOP3: Analyze Painters

In [31]:
import pandas as pd

pp_cols = con.execute("DESCRIBE painting_persons").fetchdf()["column_name"].tolist()
p_cols  = con.execute("DESCRIBE paintings").fetchdf()["column_name"].tolist()
b_cols  = con.execute("DESCRIBE buildings").fetchdf()["column_name"].tolist()

def pick(cols, candidates):
    """return first candidate that exists in cols, else None"""
    for c in candidates:
        if c in cols:
            return c
    return None

# Name-Spalte in painting_persons
PP_NAME = pick(pp_cols, ["person_name", "name"])
PP_ROLE = pick(pp_cols, ["role"])

# Zeitspalten in paintings (je nachdem was es bei dir gibt)
P_TIME = pick(p_cols, ["date_label", "date", "year_label", "year", "year_start", "year_end"])

# Orts-/Gebäude-Spalten in buildings (je nachdem was es bei dir gibt)
B_BUILDING = pick(b_cols, ["building_name", "label", "name"])
B_PLACE    = pick(b_cols, ["place_name", "location_city", "city", "place"])
B_STATE    = pick(b_cols, ["place_state", "location_state", "state"])

print("Detected columns:")
print(" painting_persons:", {"name": PP_NAME, "role": PP_ROLE})
print(" paintings:", {"time": P_TIME})
print(" buildings:", {"building": B_BUILDING, "place": B_PLACE, "state": B_STATE})

Detected columns:
 painting_persons: {'name': 'person_name', 'role': 'role'}
 paintings: {'time': 'year'}
 buildings: {'building': 'name', 'place': None, 'state': 'location_state'}


In [32]:
def paintings_for_artist_cat44(artist_name: str):
    sql = """
    WITH cat44 AS (
      SELECT DISTINCT ps.nfdi_uri
      FROM painting_subjects ps
      JOIN subjects s ON ps.subject_uri = s.subject_uri
      WHERE s.subject_source='ICONCLASS'
        AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '45%'
    ),
    painter_hits AS (
      SELECT DISTINCT pp.nfdi_uri
      FROM painting_persons pp
      JOIN cat44 c ON c.nfdi_uri = pp.nfdi_uri
      WHERE pp.role='PAINTER'
        AND pp.person_name = ?
    ),
    commissioners AS (
      SELECT nfdi_uri, STRING_AGG(DISTINCT person_name, ', ') AS commissioners
      FROM painting_persons
      WHERE role='COMMISSIONER'
      GROUP BY nfdi_uri
    ),
    donors AS (
      SELECT nfdi_uri, STRING_AGG(DISTINCT person_name, ', ') AS donors
      FROM painting_persons
      WHERE role='DONOR'
      GROUP BY nfdi_uri
    )
    SELECT
      p.nfdi_uri,
      -- Zeit: bevorzugt year_start/year_end, sonst year (string)
      CASE
        WHEN p.year_start IS NOT NULL AND p.year_end IS NOT NULL AND p.year_start <> p.year_end
          THEN CAST(p.year_start AS VARCHAR) || '-' || CAST(p.year_end AS VARCHAR)
        WHEN p.year_start IS NOT NULL
          THEN CAST(p.year_start AS VARCHAR)
        ELSE p.year
      END AS time,
      p.location_state,
      p.building_name,
      p.room_name,
      COALESCE(c.commissioners, '') AS commissioners,
      COALESCE(d.donors, '') AS donors,
      p.imageUrl
    FROM painter_hits ph
    JOIN paintings p ON p.nfdi_uri = ph.nfdi_uri
    LEFT JOIN commissioners c ON c.nfdi_uri = p.nfdi_uri
    LEFT JOIN donors d ON d.nfdi_uri = p.nfdi_uri
    ORDER BY p.year_start NULLS LAST, p.year_end NULLS LAST, p.building_name
    """
    return con.execute(sql, [artist_name]).fetchdf()

TOP5 = [
    "Wohlhaupter, Emanuel",
    "Katzenberger, Balthasar",
    "Asam, Cosmas Damian",
    "Lammers, Seivert",
    "Hermann, Franz Georg",
    "Wolcker, Johann Georg",
]

for a in TOP5:
    df = paintings_for_artist_cat44(a)
    print(f"\n=== {a} | ICONCLASS 44 | n={len(df)} ===")
    display(df)



=== Wohlhaupter, Emanuel | ICONCLASS 44 | n=5 ===


,nfdi_uri,time,location_state,building_name,room_name,commissioners,donors,imageUrl
0,https://www.deckenmalerei.eu/8bc4caac-58f8-430...,1727,Hessen,"Fulda, Fürstäbtliches Residenzschloss","Gartensaal, sog. Kaisersaal",,,https://previous.bildindex.de/bilder/fmd100348...
1,https://www.deckenmalerei.eu/9c9db57d-a589-4f2...,1727,Hessen,"Fulda, Fürstäbtliches Residenzschloss","Gartensaal, sog. Kaisersaal",,,https://previous.bildindex.de/bilder/fmd100348...
2,https://www.deckenmalerei.eu/f8fc1987-ad7d-46a...,1727,Hessen,"Fulda, Fürstäbtliches Residenzschloss","Gartensaal, sog. Kaisersaal",,,https://previous.bildindex.de/bilder/fmd100348...
3,https://www.deckenmalerei.eu/ecfe811e-746b-4dc...,1727,Hessen,"Fulda, Fürstäbtliches Residenzschloss","Gartensaal, sog. Kaisersaal",,,https://previous.bildindex.de/bilder/fmd100348...
4,https://www.deckenmalerei.eu/7c2a90ae-4791-43d...,1730-1757,Hessen,"Eichenzell, Lustschloss Fasanerie",Treppenhaus,"Buseck, Amand von",,https://previous.bildindex.de/bilder/fmd100427...



=== Katzenberger, Balthasar | ICONCLASS 44 | n=12 ===


,nfdi_uri,time,location_state,building_name,room_name,commissioners,donors,imageUrl
0,https://www.deckenmalerei.eu/32734a9e-fa16-467...,1602-1604,Baden-Württemberg,"Weikersheim, Schloss",Tafelstube,"Wolfgang II., Hohenlohe-Weikersheim, Graf",,https://previous.bildindex.de/bilder/fmd100058...
1,https://www.deckenmalerei.eu/90940701-9080-4f3...,1602-1604,Baden-Württemberg,"Weikersheim, Schloss",Tafelstube,"Wolfgang II., Hohenlohe-Weikersheim, Graf",,https://previous.bildindex.de/bilder/fmd100058...
2,https://www.deckenmalerei.eu/a524314f-92dc-451...,1602-1604,Baden-Württemberg,"Weikersheim, Schloss",Tafelstube,"Wolfgang II., Hohenlohe-Weikersheim, Graf",,https://previous.bildindex.de/bilder/fmd100058...
3,https://www.deckenmalerei.eu/e956a3f5-0c07-484...,1602-1604,Baden-Württemberg,"Weikersheim, Schloss",Tafelstube,"Wolfgang II., Hohenlohe-Weikersheim, Graf",,https://previous.bildindex.de/bilder/fmd100058...
4,https://www.deckenmalerei.eu/7730ef00-f7ef-495...,1602-1604,Baden-Württemberg,"Weikersheim, Schloss",Tafelstube,"Wolfgang II., Hohenlohe-Weikersheim, Graf",,https://previous.bildindex.de/bilder/fmd100058...
5,https://www.deckenmalerei.eu/58616c52-8677-4e5...,1602-1604,Baden-Württemberg,"Weikersheim, Schloss",Tafelstube,"Wolfgang II., Hohenlohe-Weikersheim, Graf",,https://previous.bildindex.de/bilder/fmd100058...
6,https://www.deckenmalerei.eu/88e57b99-09a5-49b...,1602-1604,Baden-Württemberg,"Weikersheim, Schloss",Tafelstube,"Wolfgang II., Hohenlohe-Weikersheim, Graf",,https://previous.bildindex.de/bilder/fmd100058...
7,https://www.deckenmalerei.eu/b89620b8-eb43-474...,1602-1604,Baden-Württemberg,"Weikersheim, Schloss",Tafelstube,"Wolfgang II., Hohenlohe-Weikersheim, Graf",,https://previous.bildindex.de/bilder/fmd100058...
8,https://www.deckenmalerei.eu/28c7fb56-6c64-471...,1602-1604,Baden-Württemberg,"Weikersheim, Schloss",Tafelstube,"Wolfgang II., Hohenlohe-Weikersheim, Graf",,https://previous.bildindex.de/bilder/fmd100058...
9,https://www.deckenmalerei.eu/bc6d26aa-4877-431...,1602-1604,Baden-Württemberg,"Weikersheim, Schloss",Tafelstube,"Wolfgang II., Hohenlohe-Weikersheim, Graf",,https://previous.bildindex.de/bilder/fmd100058...



=== Asam, Cosmas Damian | ICONCLASS 44 | n=9 ===


...

,nfdi_uri,time,location_state,building_name,room_name,commissioners,donors,imageUrl
0,https://www.deckenmalerei.eu/7b9990e1-bba4-43b...,1696-1697,Thüringen,"Gotha, Residenzschloss Friedenstein",Die Schlosskirche,"Friedrich II., Sachsen-Gotha-Altenburg, Herzog",,https://previous.bildindex.de/bilder/fmd100059...
1,https://www.deckenmalerei.eu/060b7f0c-46ee-457...,1696-1697,Thüringen,"Gotha, Residenzschloss Friedenstein",Die Schlosskirche,"Friedrich II., Sachsen-Gotha-Altenburg, Herzog",,https://previous.bildindex.de/bilder/fmd100059...
2,https://www.deckenmalerei.eu/5cc0365d-c41b-449...,1696-1697,Thüringen,"Gotha, Residenzschloss Friedenstein",Die Schlosskirche,"Friedrich II., Sachsen-Gotha-Altenburg, Herzog",,https://previous.bildindex.de/bilder/fmd100059...
3,https://www.deckenmalerei.eu/ccf7c8b6-55d1-48a...,1696-1697,Thüringen,"Gotha, Residenzschloss Friedenstein",Die Schlosskirche,"Friedrich II., Sachsen-Gotha-Altenburg, Herzog",,https://previous.bildindex.de/bilder/fmd100059...
4,https://www.deckenmalerei.eu/e990e325-7dee-401...,1696-1697,Thüringen,"Gotha, Residenzschloss Friedenstein",Die Schlosskirche,"Friedrich II., Sachsen-Gotha-Altenburg, Herzog",,https://previous.bildindex.de/bilder/fmd100059...
5,https://www.deckenmalerei.eu/4fd00567-b97e-4af...,1697,Thüringen,"Gotha, Residenzschloss Friedenstein",Der Herrschaftsstand,"Friedrich II., Sachsen-Gotha-Altenburg, Herzog",,https://previous.bildindex.de/bilder/fmd100059...
6,https://www.deckenmalerei.eu/c568605b-941a-446...,1697,Thüringen,"Gotha, Residenzschloss Friedenstein",Die Schlosskirche,"Friedrich II., Sachsen-Gotha-Altenburg, Herzog",,https://previous.bildindex.de/bilder/fmd100059...
7,https://www.deckenmalerei.eu/2704a7e1-5661-490...,1697,Thüringen,"Gotha, Residenzschloss Friedenstein",Die Schlosskirche,"Friedrich II., Sachsen-Gotha-Altenburg, Herzog",,https://previous.bildindex.de/bilder/fmd100059...
8,https://www.deckenmalerei.eu/2eb873f5-575c-487...,1698,Thüringen,"Leutenberg, ehem. Schloss ""Friedensburg""",Der so genannte Schlachtensaal,"Albert Anton, Schwarzburg-Rudolstadt, Graf",,https://deckenmalerei-bilder.badw.de/eas/parti...
9,https://www.deckenmalerei.eu/cf9d56be-2913-4cb...,1698,Thüringen,"Leutenberg, ehem. Schloss ""Friedensburg""",Der so genannte Schlachtensaal,"Albert Anton, Schwarzburg-Rudolstadt, Graf",,https://deckenmalerei-bilder.badw.de/eas/parti...



=== Hermann, Franz Georg | ICONCLASS 44 | n=15 ===


,nfdi_uri,time,location_state,building_name,room_name,commissioners,donors,imageUrl
0,https://www.deckenmalerei.eu/8bda7757-ef22-400...,1732-1742,Bayern,"Kempten, fürstbischöfliche Residenz",Tagzimmer,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",,https://previous.bildindex.de/bilder/fmd100146...
1,https://www.deckenmalerei.eu/4ac8d255-ca83-449...,1732-1742,Bayern,"Kempten, fürstbischöfliche Residenz",Vorsaal,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",,https://deckenmalerei-bilder.badw.de/eas/parti...
2,https://www.deckenmalerei.eu/0bc91a69-6131-4d7...,1732-1742,Bayern,"Kempten, fürstbischöfliche Residenz",Audienzzimmer,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",,https://previous.bildindex.de/bilder/fmd100146...
3,https://www.deckenmalerei.eu/0fdffca3-4d3c-465...,1732-1742,Bayern,"Kempten, fürstbischöfliche Residenz",Thronsaal,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",,https://previous.bildindex.de/bilder/fmd100146...
4,https://www.deckenmalerei.eu/68b6a411-9116-418...,1732-1742,Bayern,"Kempten, fürstbischöfliche Residenz",Tagzimmer,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",,https://deckenmalerei-bilder.badw.de/eas/parti...
5,https://www.deckenmalerei.eu/f5796250-3a08-414...,1732-1742,Bayern,"Kempten, fürstbischöfliche Residenz",Vorsaal,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",,URL not found or request failed
6,https://www.deckenmalerei.eu/9b52582b-e780-443...,1732-1742,Bayern,"Kempten, fürstbischöfliche Residenz",Vorsaal,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",,URL not found or request failed
7,https://www.deckenmalerei.eu/acebb5e3-01d3-496...,1732-1742,Bayern,"Kempten, fürstbischöfliche Residenz",Audienzzimmer,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",,https://deckenmalerei-bilder.badw.de/eas/parti...
8,https://www.deckenmalerei.eu/e8203b64-7a70-48f...,1732-1742,Bayern,"Kempten, fürstbischöfliche Residenz",Audienzzimmer,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",,https://previous.bildindex.de/bilder/fmd100146...
9,https://www.deckenmalerei.eu/991bf987-f897-4d3...,1732-1742,Bayern,"Kempten, fürstbischöfliche Residenz",Tagzimmer,"Reichlin-Meldegg, Anselm von, Kempten, Fürstabt",,https://previous.bildindex.de/bilder/fmd100146...



=== Wolcker, Johann Georg | ICONCLASS 44 | n=7 ===


,nfdi_uri,time,location_state,building_name,room_name,commissioners,donors,imageUrl
0,https://www.deckenmalerei.eu/b466bb26-1bca-45c...,1739,Bayern,"Markt, Schlosskapelle St. Johannes der Täufer",Nördlicher Querarm,"Fugger von Wellenburg, Joseph Maria",,https://deckenmalerei-bilder.badw.de/eas/parti...
1,https://www.deckenmalerei.eu/251bb98b-45f8-4a5...,1739,Bayern,"Markt, Schlosskapelle St. Johannes der Täufer",Vierung,"Fugger von Wellenburg, Joseph Maria",,https://deckenmalerei-bilder.badw.de/eas/parti...
2,https://www.deckenmalerei.eu/96b472f5-6b15-416...,1739,Bayern,"Markt, Schlosskapelle St. Johannes der Täufer",Nördlicher Querarm,"Fugger von Wellenburg, Joseph Maria",,https://previous.bildindex.de/bilder/fmd100278...
3,https://www.deckenmalerei.eu/752e99bc-fe8f-45c...,1739,Bayern,"Markt, Schlosskapelle St. Johannes der Täufer",Nördliche Emporen-Erweiterung,"Fugger von Wellenburg, Joseph Maria",,https://deckenmalerei-bilder.badw.de/eas/parti...
4,https://www.deckenmalerei.eu/5e744bda-abca-47c...,1739,Bayern,"Markt, Schlosskapelle St. Johannes der Täufer",Südlicher Querarm,"Fugger von Wellenburg, Joseph Maria",,https://deckenmalerei-bilder.badw.de/eas/parti...
5,https://www.deckenmalerei.eu/e9231f3b-b36e-455...,1739,Bayern,"Markt, Schlosskapelle St. Johannes der Täufer",Vierung,"Fugger von Wellenburg, Joseph Maria",,https://deckenmalerei-bilder.badw.de/eas/parti...
6,https://www.deckenmalerei.eu/8c8ca17b-a10e-4f5...,1739,Bayern,"Markt, Schlosskapelle St. Johannes der Täufer",Nördlicher Querarm,"Fugger von Wellenburg, Joseph Maria",,https://deckenmalerei-bilder.badw.de/eas/parti...


In [33]:
from IPython.display import Image, display
import pandas as pd

for a in TOP5:
    df = paintings_for_artist_cat44(a)
    print(f"\n=== {a} | ICONCLASS 44 | n={len(df)} ===")
    
    for _, row in df.iterrows():
        print(f"🖼 {row['building_name']} | {row['room_name']} | {row['time']} | {row['location_state']}")
        
        if pd.notna(row["imageUrl"]) and row["imageUrl"] != "":
            display(Image(url=row["imageUrl"], width=350))
        else:
            print("⚠️ No image available")
        
        print("Commissioner:", row["commissioners"])
        print("Donor:", row["donors"])
        print("-" * 80)



=== Wohlhaupter, Emanuel | ICONCLASS 44 | n=5 ===
🖼 Fulda, Fürstäbtliches Residenzschloss | Gartensaal, sog. Kaisersaal | 1727 | Hessen


Commissioner: 
Donor: 
--------------------------------------------------------------------------------
🖼 Fulda, Fürstäbtliches Residenzschloss | Gartensaal, sog. Kaisersaal | 1727 | Hessen


Commissioner: 
Donor: 
--------------------------------------------------------------------------------
🖼 Fulda, Fürstäbtliches Residenzschloss | Gartensaal, sog. Kaisersaal | 1727 | Hessen


Commissioner: 
Donor: 
--------------------------------------------------------------------------------
🖼 Fulda, Fürstäbtliches Residenzschloss | Gartensaal, sog. Kaisersaal | 1727 | Hessen


Commissioner: 
Donor: 
--------------------------------------------------------------------------------
🖼 Eichenzell, Lustschloss Fasanerie | Treppenhaus | 1730-1757 | Hessen


...

Commissioner: Fugger von Wellenburg, Joseph Maria
Donor: 
--------------------------------------------------------------------------------
🖼 Markt, Schlosskapelle St. Johannes der Täufer | Nördlicher Querarm | 1739 | Bayern


Commissioner: Fugger von Wellenburg, Joseph Maria
Donor: 
--------------------------------------------------------------------------------
🖼 Markt, Schlosskapelle St. Johannes der Täufer | Nördliche Emporen-Erweiterung | 1739 | Bayern


Commissioner: Fugger von Wellenburg, Joseph Maria
Donor: 
--------------------------------------------------------------------------------
🖼 Markt, Schlosskapelle St. Johannes der Täufer | Südlicher Querarm | 1739 | Bayern


Commissioner: Fugger von Wellenburg, Joseph Maria
Donor: 
--------------------------------------------------------------------------------
🖼 Markt, Schlosskapelle St. Johannes der Täufer | Vierung | 1739 | Bayern


Commissioner: Fugger von Wellenburg, Joseph Maria
Donor: 
--------------------------------------------------------------------------------
